In [ ]:
# IMPORTING LIBRARIES
import spacy
import random
from spacy.util import minibatch, compounding
from spacy.gold import GoldParse
import pandas as pd
from tqdm import tqdm
import datetime
now = datetime.datetime.now()
print ("Model initialization : ", now.strftime("%Y-%m-%d %H:%M:%S"))

train_data= [("adidas Originals   Superstar   Valkoinen   US 5,5.   adidas Originals $ 66.5. The SIZE is us 5,5 , COLOR is valkoinen , GENDER is unisex. 17 muoti ja vaatetus from Caliroots.\r", { 'entities': [(0, 6, 'brand'), (31, 40, 'color'), (43, 49, 'size'), (70, 76, 'price'), (90, 96, 'size'), (108, 117, 'color'), (130, 136, 'gender'), (139, 159, 'provider_category'), (165, 174, 'provider')]}),
("Sc Erna Polvipituinen Hame Sininen Soyaconcept SOYACONCEPT on tanskalainen brändi, joka luo elegantteja vaatteita romanttisilla yksityiskohdilla. Takapituus: 55 cm.  Soyaconcept ä 49.99. The sizeis 36 . for women. 17 muoti ja vaatetus from Boozt.", { 'entities': [(198, 200, 'size'), (207, 212, 'gender'), (240, 245, 'provider'), (214, 234, 'provider_category'), (158, 163, 'size'), (180, 185, 'price'), (35, 46, 'brand'), (47, 58, 'brand'), (22, 26, 'product'), (27, 34, 'color')]}),
("Dana Buchman Silmälasit Taren CARAMEL TORTOISEDana Buchman Taren Silmälasit. Collection:Men. Kehyksen Väri: Tortoise. Kehysmateriaali: Plastic. Koko: 54.  Dana Buchman $ 146.0. for AGE_GROUP of adult men. 13 silmalasit ja piilolinssit Smartbuy Glasses.  \r", { 'entities': [(13, 23, 'product'), (235, 251, 'provider'), (205, 234, 'provider_category'), (200, 203, 'gender'), (194, 199, 'age_group'), (170, 175, 'price'), (150, 152, 'size'), (0, 12, 'brand'), (135, 142, 'product'), (108, 116, 'color'), (88, 91, 'gender'), (38, 46, 'color')]}), 
("Active Sports Woven Shorts B Shortsit Musta PUMA PUMA Active Sports Woven Shorts B  PUMA € 27.0. The size available are 164 cm ', 128 cm, 110 cm, 116 cm, 104 cm, 176 cm. color available is black for kids. 17 muoti ja vaatetus Boozt.", { 'entities': [(38, 43, 'color'), (29, 37, 'product'), (20, 26, 'product'), (199, 203, 'age_group'), (226, 231, 'provider'), (205, 225, 'provider_category'), (189, 194, 'color'), (162, 165, 'size'), (154, 157, 'size'), (146, 149, 'size'), (138, 141, 'size'), (130, 133, 'size'), (120, 123, 'size'), (91, 95, 'price'), (44, 48, 'brand')]}),
("Renata Polvipituinen Hame Musta Fall Winter Spring Summer Fall Winter Spring Summer. A linjainen. Fall Winter Spring Summer €199.0. SIZE : xs, COLOR: jet black, GENDER: women. provider_category: 17 muoti ja vaatetus. provider: Boozt.", { 'entities': [(21, 25, 'product'), (227, 232, 'provider'), (195, 215, 'provider_category'), (169, 174, 'gender'), (150, 159, 'color'), (139, 141, 'size'), (124, 130, 'price'), (26, 31, 'color'), (32, 57, 'brand'), (98, 123, 'brand')]}),
("Hugo Boss 99999 Naisten kello 1502479 Valkoinen/Kullansävytetty", {'entities': [(0, 9, 'brand'), (16, 23, 'gender'), (24, 29, 'product'), (38, 47, 'color'), (48, 63, 'material')]}), 
("Jeffree Star Cosmetics The Gloss Mouthful", {'entities': [(0, 22, 'brand'), (27, 32, 'product')]}),
("Talvisaappaat Geox  D HOSMOS B ABX", {'entities': [(0, 13, 'product'), (14, 18, 'brand')]}), 
("Levis Men Vintage Stripe Yd Boxer B Bokserit Musta Levi´s", {'entities': [(0, 5, 'brand'), (6, 9, 'gender'), (28, 33, 'product'), (45, 50, 'color'), (36, 44, 'product')]}), 
("Salo M Technical Polo Polos Short sleeved Sininen HaltiHalti. Ulkomateriaali: 54% polyesteri, 46% genbrugs polyesteri. Hengittävä materiaali.  Halti 54.9€ Size: s, Color: blue opal, Gender: men. Provider_category is 17 muoti ja vaatetus. Provider : Boozt.", { 'entities': [(22, 33, 'product'), (42, 49, 'color'), (50, 55, 'brand'), (82, 92, 'material'), (98, 117, 'material'), (149, 153, 'price'), (171, 180, 'color'), (190, 193, 'gender'), (216, 236, 'provider_category'), (249, 254, 'provider')]}),
("Tennarit ML850CDNew Balance  tennarit, joiden päällinen mokkaa ja kangasta. Kengissä on logo läpässä ja kannassa. Kumipohja ja kangasvuori.  New Balance ä 64.0 Color is sky blue, Gender is men. PROVIDER_CATEGORY is 17 muoti ja vaatetus. PROVIDER is Stayhard.\r", { 'entities': [(16, 27, 'brand'), (249, 257, 'product'), (215, 235, 'provider_category'), (189, 192, 'gender'), (169, 177, 'color'), (155, 159, 'price'), (0, 8, 'product'), (29, 37, 'product'), (127, 138, 'material'), (114, 123, 'material'), (76, 84, 'product')]}),
("G9 Orange ShinySärkymättämät huippulaadukkaat aurinkolasit oranssilla linssillä. Cat 1.  Gloryfy € 89.0{'gender': ('men') ,16 urheilu ja eraScandinavian Outdoor    ('men')", { 'entities': [(116, 119, 'gender'), (139, 159, 'provider'), (122, 139, 'provider_category'), (99, 103, 'price'), (89, 96, 'brand'), (46, 58, 'product'), (70, 76, 'material'), (3, 9, 'color'), (59, 66, 'color')]}),
("G9 Orange ShinySärkymättämät huippulaadukkaat aurinkolasit oranssilla linssillä. Cat 1.  Gloryfy € 89.0{'gender': ('men') ,16 urheilu ja eraScandinavian Outdoor    ('men')", { 'entities': [(116, 119, 'gender'), (139, 159, 'provider'), (122, 139, 'provider_category'), (99, 103, 'price'), (89, 96, 'brand'), (46, 58, 'product'), (70, 76, 'material'), (3, 9, 'color'), (59, 66, 'color')]}),
("Diesel Aurinkolasit DL0076 90GDiesel DL0076 Aurinkolasit. Collection:Men. Kehyksen Väri: Blue.  Linssin Väri: . Kehysmateriaali: Plastic. Koko: 56.  Diesel €88.0. SIZE : 56. For adult male   blue'. PROVIDER_CATEGORY : 13 silmalasit ja piilolinssit. PROVIDER : Smartbuy Glasses.  \r", { 'entities': [(0, 6, 'brand'),  (157, 161, 'price'), (144, 146, 'size'), (89, 93, 'color'), (69, 72, 'gender'), (184, 188, 'gender'), (178, 183, 'age_group'), (191, 195, 'color'), (96, 102, 'material'), (260, 276, 'provider'), (218, 247, 'provider_category'), (7, 19, 'product'), (129, 136, 'material')]}),
("Dotti Tyyny 50x50cm, MixDotti Tyyny 50x50cm, Mix fron OYOY .  OYOY £ 53.0  with Category is 07 koti ja puutarha from Royal Design.", { 'entities': [(117, 129, 'provider'), (92, 111, 'provider_category'), (69, 73, 'price'), (12, 19, 'size'), (6, 11, 'product'), (54, 58, 'brand')]}),
("Kengät Jonak  DIDLANEOKengät jonak didlaneo black saatavana naisten kokoja 37,38,39,40,41  Jonak 169.0 €. Category: 17 muoti ja vaatetus by Spartoo.", { 'entities': [(0, 6, 'product'), (140, 147, 'provider'), (116, 136, 'provider_category'), (97, 104, 'price'), (87, 89, 'size'), (84, 86, 'size'), (81, 83, 'size'), (78, 80, 'size'), (75, 77, 'size'), (60, 67, 'gender'), (44, 49, 'color')]}),
("Lyhyt mekko See U Soon  SANTINELyhyt mekko see u soon santine black saatavana naisten kokoja eu xs  See U Soon $ 72.79. color: black. size: EU xs. 17 muoti ja vaatetus by Spartoo.", { 'entities': [(12, 22, 'brand'), (96, 98, 'size'), (62, 67, 'color'), (113, 118, 'price'), (78, 85, 'gender'), (171, 178, 'provider'), (6, 11, 'product'), (147, 167, 'provider_category'), (43, 53, 'brand')]}),
("Dsquared2 Silmälasit DQ5126 072Dsquared2 DQ5126 Silmälasit. Collection:Men. Kehyksen Väri: Shiny Rose. Kehysmateriaali: Plastic. Koko: 51. Dsquared 266.0€ {'size': ('51'), 'color': ('shiny rose'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses.  \r", { 'entities': [(273, 289, 'provider'), (243, 272, 'provider_category'), (232, 237, 'age_group'), (209, 213, 'gender'), (182, 193, 'color'), (10, 20, 'product'), (71, 74, 'gender'), (91, 101, 'color'), (120, 127, 'material'), (135, 137, 'size'), (148, 154, 'price'), (0, 9, 'brand'), (139, 147, 'brand')]}),
("Stussy   Basic Tee   Musta   M    Stussy $ 31.5. SIZE: m, COLOR: musta, GENDER: male. 17 muoti ja vaatetus from Caliroots.", { 'entities': [(112, 121, 'provider'), (86, 106, 'provider_category'), (80, 84, 'gender'), (55, 56, 'size'), (21, 22, 'size'), (65, 70, 'color'), (43, 47, 'price'), (0, 6, 'brand')]}),
("Kengät Grisport  Nero Dakar 10309D109GKengät grisport nero dakar 10309d109g multicolour saatavana miesten kokoja 40,41,46  Grisport 94.9 € COLOR: multicolour. 17 muoti ja vaatetus, Spartoo. ", { 'entities': [(181, 188, 'provider'), (159, 179, 'provider_category'), (98, 105, 'gender'), (0, 6, 'product'), (113, 115, 'size'), (116, 118, 'size'), (119, 121, 'size'), (132, 136, 'price'), (7, 15, 'brand'), (76, 87, 'color')]}),
("Vredestein AW ( 15.0/55  17 134A8 TL )Rengas viheriäille ja maaston tyästämiseen  Vahva, joustava nailonrunko  Suuri kosketuspinta  Erityisesti kehitetty profiili peltotäihin sallii renkaan monipuolisen käytän kaikilla pinnoilla. Suosittu profiili korkean kehityksen rengasrakenteella takaa hyvän vakauden joka olosuhteessa. Erityisen vastustuskykyinen.15.0/55  17 134A8 TLVredestein € 407.1. SIZE is 17.0. 02 ajoneuvot Rengas Online ", { 'entities': [(423, 436, 'provider'), (410, 422, 'provider_category'), (404, 408, 'size'), (389, 394, 'price'), (25, 27, 'size'), (0, 10, 'brand'), (38, 44, 'product')]}),
("Ryders Aurinkolasit TRAPPER R02502ARyders TRAPPER Aurinkolasit. Collection:Men. Kehyksen Väri: Brown. Linssin Väri:. Kehysmateriaali: Plastic. Koko: 61. Ryders €75.0 {'size': ('61'), 'color': ('brown'), 'gender': ('male'), 'AGE_GROUP': ('adult'),. 13 silmalasit ja piilolinssit by Smartbuy Glasses.  \r", { 'entities': [(284, 300, 'provider'), (251, 280, 'provider_category'), (239, 244, 'age_group'), (216, 220, 'gender'), (7, 19, 'product'), (135, 142, 'material'), (103, 110, 'material'), (76, 79, 'gender'), (0, 6, 'brand'), (154, 160, 'brand'), (161, 166, 'price'), (150, 152, 'size'), (96, 101, 'color'), (195, 200, 'color')]}),
("Farah Silmälasit FHO 1011 113Farah FHO 1011 Silmälasit. Collection:Men. Kehyksen Väri: Crystal Clear. Kehysmateriaali: Plastic. Koko: 49.  Farah104.0{'size': ('49'), 'color': ('crystal clear'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses('49'),('crystal clear'),('male'),('adult')  \r", { 'entities': [(0, 5, 'brand'), (144, 149, 'price'), (134, 136, 'size'), (87, 100, 'color'), (177, 190, 'color'), (206, 210, 'gender'), (229, 234, 'age_group'), (67, 70, 'gender'), (266, 282, 'provider'), (237, 266, 'provider_category'), (119, 126, 'material'), (6, 16, 'product')]}),
("F 7301 Korolliset Sandaalit Ruskea Wonders Wonder. Ylä: 100% vuohennahka nahka. Ylävuori: 100% siannahka. Pohjallinen: 100% vuohennahka nahka. Välipohja: 100% cardboard. Ulkopohja: 100% synthetic vegan pohja. Neliänmuotoinen kärki. Valmistettu  Espanja. Koron korkeus: 4.5 cm.  Wonders € 119.0. its for women, color is sand and size of 38. 17 muoti ja vaatetus Boozt.", { 'entities': [(361, 366, 'provider'), (340, 360, 'provider_category'), (336, 338, 'size'), (319, 323, 'color'), (303, 308, 'gender'), (288, 293, 'price'), (269, 272, 'size'), (18, 27, 'product'), (28, 34, 'color'), (35, 42, 'brand'), (67, 72, 'material'), (196, 201, 'material'), (186, 194, 'material')]}),
("G4 PRO White mirrorSärkymättämät huippulaadukkaat lasit tummalla linssillä.  Gloryfy159.9{'gender': ('men') ,16 urheilu ja eraScandinavian Outdoor    ('men')", { 'entities': [(125, 145, 'provider'), (108, 125, 'provider_category'), (102, 105, 'gender'), (84, 89, 'price'), (77, 84, 'brand'), (7, 12, 'color'), (65, 71, 'material'), (50, 55, 'product')]}),
("Tennarit CTALYMSINew Balance  nahkatennarit. Kengissä on kullanväriset yksityiskohdat ja läpässä kirjailtu logo. Kumipohja ja kangasvuori.  New Balance47.0{'color': ('8155   black'), 'gender': ('men'), 'PATTERN': ('musta'), 17 muoti ja vaatetusStayhard  ('8155   black'),('men')  ('musta')\r", { 'entities': [(243, 251, 'provider'), (223, 243, 'provider_category'), (151, 155, 'price'), (17, 28, 'brand'), (215, 220, 'color'), (195, 198, 'gender'), (174, 179, 'color'), (0, 8, 'product'), (45, 53, 'product'), (30, 43, 'material'), (57, 70, 'color')]}),
("Sukat Bamboo Socks, 5 pariaCDLP sukat, 5 paria. Pehmeää materiaalia. Varren yläosassa joustinneulosreuna ja logo. Toimitetaan lahjapakkauksessa.  CDLP85.0{'color': ('black'), 'gender': ('men'), 'PATTERN': ('musta'), 17 muoti ja vaatetusStayhard  ('black'),('men')  ('musta')\r", { 'entities': [(187, 190, 'gender'), (166, 171, 'color'), (207, 212, 'color'), (0, 5, 'product'), (32, 37, 'product')]}),
("Cazal Silmälasit 4267 004Cazal 4267 Silmälasit. Collection:Women. Kehyksen Väri: Brown/Gold. Kehysmateriaali: Metal. Koko: 53. Cazal 270.0€ {'size': ('53'), 'color': ('brown/gold'), 'gender': ('female'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses ('53'),('brown/gold'),('female'),('adult')  \r", { 'entities': [(219, 224, 'age_group'), (194, 200, 'gender'), (172, 176, 'color'), (166, 171, 'color'), (230, 259, 'provider_category'), (59, 64, 'gender'), (81, 86, 'color'), (87, 91, 'color'), (133, 139, 'price'), (0, 5, 'brand'), (127, 132, 'brand'), (123, 125, 'size'), (6, 16, 'product'), (110, 115, 'material'), (260, 276, 'provider')]}),
("13294 Bags Small Shoulder Bags   Crossbody Bags Kermanvärinen DEPECHEDEPECHE 13294  Depeche119.0{'size': ('one size'), 'color': ('011 sand'), 'gender': ('women') ,17 muoti ja vaatetusBoozt('one size'),('011 sand'),('women')", { 'entities': [(154, 159, 'gender'), (182, 187, 'provider'), (162, 182, 'provider_category'), (6, 10, 'product'), (84, 91, 'brand'), (62, 69, 'brand'), (91, 96, 'price'), (107, 115, 'size'), (130, 138, 'color')]}),
("Madina W Tee T shirts & Tops Short sleeved Musta 8848 Altitude 8848 Altitude Madina W Tee 8848 Altitude 34.95€ {'size': ('40','42','34'), 'color': ('black'), 'gender': ('women'),. 17 muoti ja vaatetus Boozt", { 'entities': [(104, 110, 'price'), (49, 62, 'brand'), (95, 103, 'brand'), (149, 154, 'color'), (43, 48, 'color'), (122, 134, 'size'), (170, 175, 'gender'), (201, 206, 'provider'), (180, 200, 'provider_category'), (13, 21, 'product'), (24, 28, 'product')]}),
("Superdry Aurinkolasit SDS YATOMI 006Superdry SDS YATOMI Aurinkolasit. Collection:Men. Kehyksen Väri: Blue.  Linssin Väri: . Kehysmateriaali: Plastic. Koko: 59.  Superdry67.0{'size': ('59'), 'color': ('blue'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses.  \r", { 'entities': [(281, 297, 'provider'), (252, 281, 'provider_category'), (244, 249, 'age_group'), (221, 225, 'gender'), (81, 84, 'gender'), (9, 21, 'product'), (141, 148, 'material'), (0, 8, 'brand'), (169, 173, 'price'), (156, 158, 'size'), (101, 105, 'color'), (108, 114, 'material'), (201, 205, 'color')]}),
("Burberry Aurinkolasit BE4181 300187Burberry BE4181 Aurinkolasit. Collection:Men. Kehyksen Väri: Black.  Linssin Väri: . Kehysmateriaali: Plastic. Koko: 58.  Burberry 168.0 €. COLOR is black and size is 58.for adult male. 13 silmalasit ja piilolinssit Smartbuy Glasses.\r", { 'entities': [(0, 8, 'brand'), (166, 171, 'price'), (152, 154, 'size'), (96, 101, 'color'), (184, 189, 'color'), (76, 79, 'gender'), (215, 219, 'gender'), (209, 214, 'age_group'), (251, 267, 'provider'), (221, 250, 'provider_category'), (9, 21, 'product'), (104, 110, 'material'), (137, 144, 'material')]}),
("Ray Kattovalaisin, Large, Valkoinen Ray Kattovalaisin, Large, Valkoinen fron Petite Friture. Petite Friture 260.0€ 07 koti ja puutarha Royal Design", { 'entities': [(135, 147, 'provider'), (115, 134, 'provider_category'), (4, 17, 'product'), (26, 35, 'color'), (108, 114, 'price'), (77, 91, 'brand'), (93, 107, 'brand'), (19, 24, 'size')]}),
("Paradigm Aurinkolasit 19 30 Rose GoldParadigm 19 30 Aurinkolasit. Collection:Men. Kehyksen Väri: Gold.  Linssin Väri: . Kehysmateriaali: Metal. Koko: 53.  Paradigm 120.0€. 53 size, gold color for male adult. Provider_category : 13 silmalasit ja piilolinssit. Provider : Smartbuy Glasses.  \r", { 'entities': [(164, 169, 'price'), (150, 152, 'size'), (181, 185, 'color'), (196, 200, 'gender'), (77, 80, 'gender'), (28, 37, 'color'), (137, 142, 'material'), (270, 286, 'provider'), (228, 257, 'provider_category'), (201, 206, 'age_group'), (9, 21, 'product'), (104, 110, 'material'), (0, 8, 'brand')]}),
("General Tractor V.Ply ( 20.8  38 151A6 10PR TT )  20.8  38 151A6 10PR TTGeneral1712.2{'size': ('38.0'), 02 ajoneuvotRengas Online('38.0')  ", { 'entities': [(96, 100, 'size'), (79, 85, 'price'), (0, 7, 'brand'), (30, 32, 'size'), (115, 128, 'provider'), (103, 115, 'provider_category')]}),
("Sola Dress Polvipituinen Mekko Kermanvärinen Rä©sumä©Rä©sumä©. A linjainen.  Rä©sumä©170.0{'size': ('34'), 'color': ('sand'), 'gender': ('women'), 17 muoti ja vaatetusBoozt.   \r", { 'entities': [(160, 165, 'provider'), (140, 160, 'provider_category'), (132, 137, 'gender'), (79, 84, 'price'), (95, 97, 'size'), (112, 116, 'color'), (45, 51, 'brand'), (25, 30, 'product'), (5, 10, 'product'), (31, 44, 'color')]}),
("Helix teekannu ruostumaton teräs 1 l    Georg Jensen 132.0€ {'color': ('metalli'), 07 koti ja puutarha Makujakauppa  ('metalli')", { 'entities': [(40, 52, 'brand'), (53, 59, 'price'), (27, 32, 'material'), (72, 79, 'color'), (103, 115, 'provider'), (83, 102, 'provider_category'), (6, 14, 'product'), (33, 36, 'size')]}),
("False Lash Effect Velvet Volume Mascara, Black    Max Factor11.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(88, 99, 'provider'), (67, 88, 'provider_category'), (60, 65, 'price'), (50, 60, 'brand'), (41, 46, 'color'), (32, 39, 'product')]}),
("False Lash Effect Velvet Volume Mascara, Black/Br    Max Factor11.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(91, 102, 'provider'), (70, 91, 'provider_category'), (63, 68, 'price'), (53, 63, 'brand'), (41, 46, 'color'), (47, 49, 'color'), (32, 39, 'product')]}),
("ROK Merino Gloves Musta L/XLMerinovillasormikkaat aikuisille.  Lasting19.9{'size': ('l/xl'), 'color': ('musta'), 'gender': ('men, women'), 16 urheilu ja eraScandinavian Outdoor('l/xl'),('musta'),('men, women')", { 'entities': [(130, 135, 'gender'), (125, 128, 'gender'), (155, 175, 'provider'), (138, 155, 'provider_category'), (104, 109, 'color'), (87, 89, 'size'), (11, 17, 'product'), (39, 49, 'product'), (28, 39, 'material'), (4, 10, 'material'), (70, 74, 'price'), (63, 70, 'brand'), (50, 60, 'age_group'), (26, 28, 'size'), (24, 25, 'size'), (18, 23, 'color')]}),
("HP ZBook Studio 360 G5 i9 9880H 15.6inch UHD Touch 16GB DDR4 512GB SSD Nvidia Quadro P2000 4GB Webcam AC+BT 6C Batt FPR W10P 3YW(SE)", {'entities': [(0, 2, 'brand'), (3, 8, 'product'), (32, 40, 'size')]}),
("Calvin Klein 99999 Miesten kello K8M2712N Sininen/Teräs Ø43 mm", {'entities': [(0, 12, 'brand'), (19, 26, 'gender'), (27, 32, 'product'), (42, 49, 'color'), (50, 55, 'material'), (56, 62, 'size')]}), 
("L.A. Girl\xa0 Pro.Coverage Illuminating Foundation\xa0 Soft Honey", {'entities': [(0, 9, 'brand'), (37, 47, 'product')]}),
("Pusero Helena", {'entities': [(0, 6, 'product'), (7, 13, 'brand')]}), 
("Rado True Naisten kello R27059722 Musta/Keraaminen Ø40 mm", {'entities': [(0, 9, 'brand'), (10, 17, 'gender'), (18, 23, 'product'), (34, 39, 'color'), (40, 50, 'material'), (51, 57, 'size')]}),
("Top Bamse P8 T shirts Long sleeved T shirts Valkoinen Lindex", {'entities': [(13, 21, 'product'), (44, 53, 'color'), (54, 60, 'brand')]}), 
("Diesel Silmälasit DL5067 001Diesel DL5067 Silmälasit. Collection:Adult Men. Kehyksen Väri: Shiny Black. Kehysmateriaali: Plastic. Koko: 54.  Diese € l78.0   13 silmalasit ja piilolinssit Smartbuy Glasses.  \r", { 'entities': [(187, 203, 'provider'), (157, 186, 'provider_category'), (0, 6, 'brand'), (150, 154, 'price'), (136, 138, 'size'), (121, 128, 'material'), (65, 70, 'age_group'), (71, 74, 'gender'), (91, 102, 'color'), (7, 17, 'product')]}),
("Small Eye Shadow Beauty WOMEN Makeup Eyes Eyeshadow   Not Palettes Ruskea M.A.C.M.A.C. Small Eye Shadow  M.A.C.20.99{'size': ('1.3 g'), 'color': ('l.e.s artiste'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('1.3 g'),('l.e.s artiste'),('women')", { 'entities': [(176, 181, 'gender'), (204, 209, 'provider'), (184, 204, 'provider_category'), (127, 132, 'size'), (111, 116, 'price'), (74, 80, 'brand'), (42, 51, 'product'), (24, 29, 'gender'), (67, 73, 'color'), (0, 5, 'size')]}),
("Capsule Wool Tights Sukkahousut Valkoinen Mp Denmarkmp Denmark. 80% villa, 17% polyamidi, 3% elastaani. Villa luo hengittävän ja eristävän materiaalin, joka pitää sinut lämpimänä kylminä päivinä..  mp Denmark22.95{'size': ('92', '128', '152', '104', '56 62', '140', '80', '68 74', '164'), 'color': ('432', 'snow white'), 'gender': ('kids'), 17 muoti ja vaatetusBoozt('92', '128', '152', '104', '56 62', '140', '80', '68 74', '164'),('432', 'snow white'),('kids')", { 'entities': [(333, 337, 'age_group'), (307, 317, 'color'), (8, 12, 'material'), (360, 365, 'provider'), (340, 360, 'provider_category'), (282, 285, 'size'), (273, 277, 'size'), (64, 66, 'size'), (260, 263, 'size'), (251, 256, 'size'), (244, 247, 'size'), (237, 240, 'size'), (230, 233, 'size'), (224, 226, 'size'), (20, 31, 'product'), (32, 41, 'color'), (208, 213, 'price'), (52, 62, 'brand'), (42, 52, 'brand')]}),
("Norden 2 osainen veitsisetti    Fiskars118.9{'color': ('ruskea'), 07 koti ja puutarhaMakujakauppa  ('ruskea')", { 'entities': [(65, 84, 'provider_category'), (17, 28, 'product'), (56, 62, 'color'), (39, 44, 'price'), (32, 39, 'brand'), (84, 96, 'provider')]}),
("Marshmallow Hearts Bath & Shower Gel 500 mlTuotekuvaus tulossa pian  Treaclemoon6.94€ 17 muoti ja vaatetusShopping4net", { 'entities': [(69, 80, 'brand'), (80, 84, 'price'), (37, 43, 'size'), (26, 36, 'product'), (106, 118, 'provider'), (86, 106, 'provider_category')]}),
("Wet n Wild Angled Liner BrushTuotekuvaus tulossa pian  Wet n Wild6.99  17 muoti ja vaatetusShopping4net", { 'entities': [(0, 10, 'brand'), (65, 69, 'price'), (71, 91, 'provider_category'), (91, 103, 'provider'), (18, 29, 'product')]}),
("Stella, Shw, Brief Bikinialaosa Sininen ZizziZizzi Stella, Shw, Brief  Zizzi49.95{'size': ('48', '46', '50', '52'), 'color': ('dark blue'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('48', '46', '50', '52'),('dark blue'),('women')", { 'entities': [(40, 45, 'brand'), (76, 81, 'price'), (92, 94, 'size'), (98, 100, 'size'), (104, 106, 'size'), (127, 136, 'color'), (110, 112, 'size'), (32, 39, 'color'), (19, 31, 'product'), (180, 185, 'provider'), (160, 180, 'provider_category'), (152, 157, 'gender')]}),
("Mom Jean Jeans Mom Jeans Sininen Calvin Klein Jeans Calvin Klein Jeans Mom Jean Calvin Klein Jeans 109.9€ {'size': ('31','28','29','32'), 'color': ('light blue yoke'), 'gender': ('women'), 17 muoti ja vaatetus Boozt ('31', '28', '29', '32'),('ab068 light blue yoke'),('women')", { 'entities': [(180, 185, 'gender'), (149, 164, 'color'), (117, 134, 'size'), (212, 217, 'provider'), (191, 211, 'provider_category'), (25, 32, 'color'), (0, 3, 'gender'), (33, 51, 'brand'), (80, 98, 'brand'), (99, 105, 'price'), (9, 14, 'product'), (4, 8, 'product')]}),
("G15 Black MatSärkymättämät huippulaadukkaat aurinkolasit. Cat 3.  Gloryfy119.9{'gender': ('men'), 16 urheilu ja eraScandinavian Outdoor    ('men')", { 'entities': [(66, 73, 'brand'), (73, 78, 'price'), (4, 9, 'color'), (44, 56, 'product'), (114, 134, 'provider'), (97, 114, 'provider_category'), (91, 94, 'gender')]}),
("Colby Sneaker Matalavartiset Sneakerit Tennarit Musta Michael Kors ShoesMichael Kors Shoes. Nauhakiinnitys. Koron korkeus: 0 cm.  Michael Kors Shoes150.0{'size': ('40', '38.5', '37.5'), 'color': ('black'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('40', '38.5', '37.5'),('black'),('women')", { 'entities': [(246, 251, 'provider'), (226, 246, 'provider_category'), (29, 38, 'product'), (6, 13, 'product'), (54, 72, 'brand'), (148, 153, 'price'), (164, 166, 'size'), (170, 174, 'size'), (178, 182, 'size'), (197, 202, 'color'), (39, 47, 'product'), (48, 53, 'color'), (218, 223, 'gender')]}),
("D Bazer Farkut Sininen Diesel MenDiesel Men D Bazer  Diesel Men160.0{'size': ('31', '30'), 'color': ('blue nights'), 'gender': ('men'), 17 muoti ja vaatetusBoozt('31', '30'),('blue nights'),('men')", { 'entities': [(155, 160, 'provider'), (135, 155, 'provider_category'), (102, 113, 'color'), (15, 22, 'color'), (23, 33, 'brand'), (63, 68, 'price'), (79, 81, 'size'), (85, 87, 'size'), (8, 14, 'product'), (40, 44, 'gender'), (129, 132, 'gender')]}),
("Lecco 80 T shirts Short sleeved Valkoinen BOSSLuksusbrändi, BOSS BLACK luo korkealaatuista ja eleganttia muotia tyylikkäille ja itsevarmoille miehille.. 100% puuvillaa. Pyäreä pääntie. Valmistettu  Portugali.  BOSS49.95{'size': ('l', 'm', 'xl', 'xxl', 's'), 'color': ('white'), 'gender': ('men'), 17 muoti ja vaatetusBoozt('l', 'm', 'xl', 'xxl', 's'),('white'),('men')", { 'entities': [(290, 293, 'gender'), (316, 321, 'provider'), (296, 316, 'provider_category'), (246, 249, 'size'), (240, 242, 'size'), (105, 106, 'size'), (25, 26, 'size'), (269, 274, 'color'), (214, 219, 'price'), (42, 46, 'brand'), (9, 17, 'product'), (32, 42, 'color'), (142, 150, 'gender'), (60, 65, 'brand'), (65, 71, 'color')]}),
("G12 Black Shiny PolarizedSärkymättämät huippulaadukkaat aurinkolasit polarisoivilla linsseillä. Cat 3.  Gloryfy €189.9. GENDER: men. 16 urheilu ja era Scandinavian Outdoor.   \r", { 'entities': [(151, 171, 'provider'), (4, 9, 'color'), (56, 68, 'product'), (113, 118, 'price'), (104, 111, 'brand'), (128, 131, 'gender'), (133, 150, 'provider_category'), (84, 91, 'material')]}),
("Suoja Box Kaasugrilliin    Eva Solo 42.93€ Color   musta'. 07 koti ja puutarha from Makujakauppa.", { 'entities': [(84, 96, 'provider'), (59, 78, 'provider_category'), (51, 56, 'color'), (36, 41, 'price'), (27, 35, 'brand'), (10, 23, 'product')]}),
("Verpan Fun 1DM RiippuvalaisinFun 1DM Riippuvalaisin fron Verpan .  Verpan 862.0€   07 koti ja puutarhaRum21", { 'entities': [(102, 107, 'provider'), (83, 102, 'provider_category'), (74, 79, 'price'), (0, 6, 'brand'), (15, 32, 'product')]}),
("Lyhyt mekko Betty London  LADYLyhyt mekko betty london lady red saatavana naisten kokoja s,m,l  Betty London $34.49. 17 muoti ja vaatetus Spartoo", { 'entities': [(138, 145, 'provider'), (117, 137, 'provider_category'), (110, 115, 'price'), (12, 24, 'brand'), (48, 49, 'size'), (64, 65, 'size'), (6, 11, 'product'), (60, 63, 'color'), (74, 81, 'gender')]}),
("Coat Rack Naulakko, Musta/RautaModern and practical hanger that does not occupy any floor area.  House Doctor €171.0.  07 koti ja puutarha Royal Design", { 'entities': [(139, 151, 'provider'), (119, 138, 'provider_category'), (111, 116, 'price'), (97, 109, 'brand'), (20, 25, 'color'), (52, 58, 'product'), (0, 9, 'product'), (10, 18, 'product')]}),
("Chelsea Päytävalaisin 31x48cm, MustaChelsea päytävalaisin on modernin yksinkertainen, ja yhdistelee upeasti minimalismia ja ylellisiä yksityiskohtia.  By On €166.0  07 koti ja puutarha Royal Design", { 'entities': [(185, 197, 'provider'), (165, 184, 'provider_category'), (158, 163, 'price'), (151, 156, 'brand'), (22, 29, 'size'), (8, 21, 'product'), (31, 36, 'color')]}),
("Reppu Vans  WM REALM PLUS BACKPAReppu vans wm realm plus backpa bordeaux saatavana miesten kokoja yksi koko  Vans £ 44.79. 'color': 'bordeaux'. 17 muoti ja vaatetus Spartoo. ", { 'entities': [(165, 172, 'provider'), (98, 107, 'size'), (6, 10, 'brand'), (116, 121, 'price'), (64, 72, 'color'), (144, 164, 'provider_category'), (83, 90, 'gender'), (46, 51, 'product'), (15, 20, 'product')]}),
("Lash Sensational Waterproof Mascara, Black    Maybelline £ 12.95  06 kauneus ja terveys   Hajuvesi.fi", { 'entities': [(90, 101, 'provider'), (66, 87, 'provider_category'), (28, 35, 'product'), (37, 42, 'color'), (46, 56, 'brand'), (59, 64, 'price')]}),
("Tilda Dress Polvipituinen Mekko Sininen Boob Boob. A linjainen. Global Organic Textile Standard (GOTS) standardi varmistaa, että lopputuotteen koko tuotantoketju on luonnonmukainen. Tämä poissulkee myrkylliset torjunta aineet ja lannoitteet, joita käytetään tavallisen puuvillan viljelyssä. Standardi auttaa pitämään maaperän terveenä ja vähentää vesistäjen saastumista, mikä parantaa samalla tyäntekijäiden turvallisuutta. Valitsemalla luonnonmukaista tuet aiempaa terveellisempää puuvillan viljelyä. Luomu. Valmistettu Portugali. Boob 84.95€ {'size': ('xl', 'm', 's', 'xs'), 'color': ('thunder blue'), 'gender': ('women'), 17 muoti ja vaatetus Boozt ('xl', 'm', 's', 'xs'),('thunder blue'),('women')", { 'entities': [(116, 117, 'size'), (557, 576, 'size'), (6, 11, 'product'), (32, 39, 'color'), (26, 31, 'product'), (590, 602, 'color'), (40, 44, 'brand'), (534, 538, 'brand'), (539, 545, 'price'), (649, 654, 'provider'), (627, 647, 'provider_category'), (617, 622, 'gender')]}),
("Origami Päytävalaisin, Valkoinen Japanilaisista origami paperiteoksista inspiraationsa saanut päytävalaisin on valmistettu paperista ja mangopuusta. Yksityiskohtainen, ja kaunis varjostin pysyy paikoillaan magneettien avulla. By On 108.0€ 07 koti ja puutarha Royal Design", { 'entities': [(259, 271, 'provider'), (239, 258, 'provider_category'), (232, 238, 'price'), (226, 231, 'brand'), (8, 21, 'product'), (55, 70, 'material'), (122, 132, 'material'), (135, 146, 'material'),(23, 31, 'color')]}),
("Absolute Woman, EdT 40ml     Bruno Banani € 22.95   06 kauneus ja terveys. Hajuvesi.fi", { 'entities': [(75, 86, 'provider'), (52, 73, 'provider_category'), (44, 49, 'price'), (29, 36, 'brand'), (20, 24, 'size'), (9, 14, 'gender')]}),
("Moss Kynttilänjalka 8.5 cm Blue Mirage    A Simple Mess 9.9€  07 koti ja puutarha Makujakauppa", { 'entities': [(82, 94, 'provider'), (62, 81, 'provider_category'), (56, 59, 'price'), (27, 31, 'color'), (20, 26, 'size'), (5, 19, 'product'), (42, 55, 'brand')]}),
("Le Parfum, Deospray 100ml    Elie Saab 31.95€  06 kauneus ja terveys Hajuvesi.fi", { 'entities': [(69, 80, 'provider'), (47, 68, 'provider_category'), (39, 44, 'price'), (20, 25, 'size'), (29, 38, 'brand'), (11, 19, 'product')]}),
("Vredestein Flotation + ( 400/70  20 150A8 TL )Ideaali rengas vaativaan ajoon.  Vankka diagonaalirakenne.  Ainutlaatuinen proofili.  Suuri kosketuspinta. Korkean kuormituskyvyn avulla ja melko vähäisellä rengaspaineella Flotation+ on ideaali rengas raskaaseen ajoon. Maksimoitu leveys ja suuri ilmavolyymi tekee renkaasta erityisen sopivan maatalouskoneille. Modernin runkorakenteensa ja ainutlaatuisen suunnittelunsa avulla Flotation+ selviytyy joka tehtävästä. Korkean ajomukavuuden lisäksi rengas omaa erinomaisen itsepuhdistusominaisuuden. Tämä mahdollistaa renkaan pyärimisen kaikissa olosuhteissa ja takaa optimoidun sivuvakauden.400/70  20 150A8 TLVredestein717.4{'size': ('20.0'), 02 ajoneuvotRengas Online('20.0')  ", { 'entities': [(683, 687, 'size'), (33, 35, 'size'), (0, 10, 'brand'), (667, 672, 'price'), (702, 715, 'provider'), (690, 702, 'provider_category'), (54, 61, 'product')]}),
("VG10 Leipäveitsi 20cm EGO 150.0€ 07 koti ja puutarha Makujakauppa", { 'entities': [(53, 65, 'provider'), (33, 52, 'provider_category'), (17, 21, 'size'), (5, 16, 'product'), (26, 32, 'price'), (22, 25, 'brand')]}),
("Mitas TD 19 ( 12.4  24 120A6 8PR TT kaksoistunnus  112A8 )  12.4  24 120A6 8PR TT kaksoistunnus  112A8Mitas456.3{'size': ('24.0'), 02 ajoneuvotRengas Online('24.0')  ", { 'entities': [(123, 127, 'size'), (142, 155, 'provider'), (130, 142, 'provider_category'), (107, 112, 'price'), (0, 5, 'brand'), (20, 22, 'size')]}),
("All Day Pique Polo Shirt Polos Short sleeved GAP", {'entities': [(19, 24, 'product'), (45, 48, 'brand')]}),
("Rotalla Setula 4 Season RA03 ( 205/50 R17 93W XL )", {'entities': [(0, 14, 'brand'), (31, 48, 'size')]}),
("Trunk Bokserit Valkoinen Tommy Hilfiger", {'entities': [(25, 39, 'brand'), (15, 24, 'color'), (6, 14, 'product')]}), 
("Sc Dollie Kilpikonnakaulus Poolopaita Oranssi Soyaconcept", {'entities': [(0, 9, 'brand'), (32, 37, 'product'), (38, 45, 'color')]}), 
("Sandals Shoes Summer Shoes Sandals Kermanvärinen Bisgaard", {'entities': [(0, 7, 'product'), (8, 13, 'product'), (49, 57, 'brand')]}),
("Gap Logo Joggers Collegehousut Olohousut Harmaa GAP GAP. Reilusti valmistettu. GAP 36.0€ {'size': ('l', 'xl'), 'color': ('light heather grey'), 'gender': ('women'), 17 muoti ja vaatetus Boozt ('l', 'xl'),('light heather grey'),('women')", { 'entities': [(41, 47, 'color'), (100, 106, 'size'), (187, 192, 'provider'), (166, 186, 'provider_category'), (155, 160, 'gender'), (48, 51, 'brand'), (83, 88, 'price'), (31, 40, 'product'), (24, 30, 'product'), (0, 3, 'brand'), (79, 82, 'brand')]}),
("Diesel Silmälasit DL5050 050Diesel DL5050 Silmälasit. Collection:Men. Kehyksen Väri: Black. Kehysmateriaali: Plastic. Koko: 52. Diesel 100  {'size': ('52'), 'color': ('black'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses ('52'),('black'),('male'),('adult')  \r", { 'entities': [(135, 130, 'price'), (0, 6, 'brand'), (128, 134, 'brand'), (124, 126, 'size'), (168, 173, 'color'), (85, 90, 'color'), (7, 17, 'product'), (109, 116, 'material'), (65, 68, 'gender'), (189, 193, 'gender'), (212, 217, 'age_group'), (223,252,'provider_category'), (253,269, 'provider')]}),
("Kate Spade Aurinkolasit Avaline2/S HT8/0JKate Spade Avaline2/S Aurinkolasit. Collection:Women. Kehyksen Väri: Gold. Linssin Väri: . Kehysmateriaali: Metal. Koko: 58. Kate Spade 146.0€ {'size': ('58'), 'color': ('gold'), 'gender': ('female'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses ('58'),('gold'),('female'),('adult')  \r", { 'entities': [(0, 10, 'brand'), (174, 180, 'price'), (162, 164, 'size'), (211, 215, 'color'), (109, 113, 'color'), (11, 23, 'product'), (166, 172, 'product'), (265, 294, 'provider_category'), (295, 311, 'provider'), (88, 93, 'gender'), (229, 235, 'gender'), (254, 259, 'age_group'), (149, 154, 'material'), (116, 123, 'material')]}),
("Ozka Agro 10 Traction ( 280/85 R24 115A8 TL kaksoistunnus 112B) 280/85 R24 115A8 TL kaksoistunnus 112B Ozka 613.8€ {'size': ('24.0'), 02 ajoneuvot Rengas Online ('24.0')  ", { 'entities': [(136, 148, 'provider_category'), (149, 162, 'provider'), (126, 130, 'size'), (31, 34, 'size'), (108, 114, 'price'), (0, 4, 'brand'), (103, 107, 'brand')]}),
("Gloryfy Gi15 St Pauli Havanna ShinySärkymättämät aurinkolasit. Cat 2.  Gloryfy149.9{'gender': ('men, women'), 16 urheilu ja eraScandinavian Outdoor    ('men, women')", { 'entities': [(101, 106, 'gender'), (96, 99, 'gender'), (126, 146, 'provider'), (109, 126, 'provider_category'), (49, 61, 'product'), (0, 7, 'brand'), (78, 83, 'price')]}),
("Select Warm Shortsit w/Lycra MustaHienot Select Warm shortsit jotka on tehty 2mm paksusta SBR neopreenistä sekä elastaanista. Optimoitu istuvuus takaa mukavuuden sekä lämmän jotka estävät nivus  sekä lukuisia muita lihasvammoja.  Select35.95{'size': ('x large'), 'color': ('musta'), 'gender': ('male'), 16 urheilu ja eraUnisport('x large'),('musta'),('male')", { 'entities': [(236, 241, 'price'), (252, 259, 'size'), (274, 279, 'color'), (12, 21, 'product'), (29, 34, 'color'), (77, 81, 'size'), (319, 327, 'provider'), (302, 319, 'provider_category'), (295, 299, 'gender'), (0, 7, 'brand')]}),
("Retrosuperfuture Aurinkolasit DREW MAMA PANNA IAXN SVXRetrosuperfuture DREW MAMA PANNA Aurinkolasit. Collection:Men. Kehyksen Väri: White. Linssin Väri: . Kehysmateriaali: Plastic. Koko: 51. Retrosuperfuture 128.0€ {'size': ('51'), 'color': ('white'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses ('51'),('white'),('male'),('adult')  \r", { 'entities': [(17, 29, 'product'), (187, 189, 'size'), (242, 247, 'color'), (132, 137, 'color'), (0, 16, 'brand'), (191, 207, 'brand'), (264, 268, 'gender'), (287, 292, 'age_group'), (328, 344, 'provider'), (298, 327, 'provider_category'), (172, 179, 'material'), (139, 146, 'material'), (208, 214, 'price'), (112, 115, 'gender')]}),
("Umbra Dress Maksimekko Juhlamekko Vaaleanpunainen Rä©sumä©Rä©sumä© Umbra Dress  Rä©sumä©125.0{'size': ('s', 'm', 'xs', 'l'), 'color': ('pink'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('s', 'm', 'xs', 'l'),('pink'),('women')", { 'entities': [(150, 155, 'gender'), (178, 183, 'provider'), (158, 178, 'provider_category'), (6, 11, 'product'), (23, 33, 'product'), (12, 22, 'product'), (50, 56, 'brand'), (82, 87, 'price'), (34, 49, 'color'), (108, 110, 'size'), (130, 134, 'color')]}),
("Harvey Jacket Outerwear Jackets Anoraks Sininen Lexington ClothingLexington Clothing. Vettähylkivä.  Lexington Clothing199.0{'size': ('m', 's'), 'color': ('dark blue'), 'gender': ('men'), 17 muoti ja vaatetusBoozt('m', 's'),('dark blue'),('men')", { 'entities': [(156, 165, 'color'), (135, 136, 'size'), (24, 31, 'product'), (40, 47, 'color'), (119, 124, 'price'), (48, 66, 'brand'), (207, 212, 'provider'), (187, 207, 'provider_category'), (181, 184, 'gender'), (7, 13, 'product')]}),
("Superdry Silmälasit SDO YUMI 104Superdry SDO YUMI Silmälasit. Collection:Men. Kehyksen Väri: Black. Kehysmateriaali: Plastic. Koko: 52.  Superdry133.0{'size': ('52'), 'color': ('black'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses('52'),('black'),('male'),('adult')  \r", { 'entities': [(222, 227, 'age_group'), (199, 203, 'gender'), (259, 275, 'provider'), (230, 259, 'provider_category'), (9, 20, 'product'), (117, 124, 'material'), (132, 134, 'size'), (93, 98, 'color'), (0, 9, 'brand'), (145, 150, 'price'), (178, 183, 'color'), (73, 76, 'gender')]}),
("Sophie Aurora Dress Maksimekko Juhlamekko Sininen Bruuns Bazaar Bruuns Bazaar Sophie Aurora Dress Bruuns Bazaar 169.95€. Size of 34, 42, COLOR night sky, 'GENDER women 17 muoti ja vaatetus Boozt", { 'entities': [(166, 186, 'provider_category'), (160, 165, 'gender'), (140, 150, 'color'), (132, 134, 'size'), (128, 130, 'size'), (111, 118, 'price'), (20, 30, 'product'), (42, 49, 'color'), (14, 19, 'product'), (188, 193, 'provider'), (50, 63, 'brand'), (97, 110, 'brand'), (31, 41, 'product')]}),
("Norr Leipäveitsi 21 cm    Fiskars49.97{'color': ('ruskea'), 07 koti ja puutarhaMakujakauppa  ('ruskea')", { 'entities': [(50, 56, 'color'), (78, 90, 'provider'), (17, 20, 'size'), (33, 38, 'price'), (26, 33, 'brand'), (5, 16, 'product'), (59, 78, 'provider_category')]}),
("Essential for Her, EdT 20ml    Replay16.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(65, 76, 'provider'), (44, 65, 'provider_category'), (37, 42, 'price'), (31, 37, 'brand'), (23, 27, 'size')]}),
("Festina Miesten kello F16374 8 Musta/Teräs Ø40 mm", {'entities': [(0, 7, 'brand'), (8, 15, 'gender'), (16, 21, 'product'), (31, 36, 'color'), (37, 42, 'material'), (43, 49, 'size')]}), 
("Iskunvaimennin   Sv 450008", {'entities': [(0, 14, 'product')]}), 
("Gubi Turbo Kattovalaisin Ø62cm, Valkoinen", {'entities': [(0, 10, 'brand'), (16, 24, 'product'), (25, 30, 'size'), (32, 41, 'color')]}),
("Star Trading Knot Lampshade 50 cm, Gray", {'entities': [(0, 12, 'brand'), (18, 27, 'product'), (28, 33, 'size'), (35, 39, 'color')]}),
("BF Goodrich Route Control T ( 285/70 R19.5 150/148J )", {'entities': [(30, 51, 'size'), (0, 27, 'brand')]}),
("Bralette Night & Underwear Underwear Tops Valkoinen Abercrombie & Fitch", {'entities': [(17, 26, 'product'), (42, 51, 'color'), (52, 71, 'brand')]}),
("Olivia Night & Underwear Underwear Tops Punainen Molo", {'entities': [(49, 53, 'brand'), (40, 48, 'color'), (15, 24, 'product')]}),
("Warm Up Anorak Outerwear Jackets Anoraks Musta Lyle & Scott", {'entities': [(25, 32, 'product'), (41, 46, 'color'), (47, 59, 'brand')]}),
("Essential for Her, EdT 40ml    Replay29.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(65, 76, 'provider'), (44, 65, 'provider_category'), (37, 42, 'price'), (31, 37, 'brand'), (23, 27, 'size')]}),
("FF Veitsitukki 5 veistä valkoinen    Fiskars 69.9€ category is 07 koti ja puutarha from Makujakauppa", { 'entities': [(88, 100, 'provider'), (63, 82, 'provider_category'), (45, 49, 'price'), (37, 44, 'brand'), (24, 33, 'color'), (3, 14, 'product')]}),
("GI21 Dante Red ärimmäisen härskinnäkäiset särkymättämät aurinkolasit.  Gloryfy €99.0  16 urheilu ja eraScandinavian Outdoor", { 'entities': [(103, 123, 'provider'), (86, 103, 'provider_category'), (80, 84, 'price'), (71, 78, 'brand'), (56, 68, 'product')]}),
("Lyhythihainen t paita BOTD ESTOILALyhythihainen t paita botd estoila white saatavana miesten kokoja 3xl BOTD 9.99€ {'color': ('white'), 17 muoti ja vaatetus Spartoo  ('white')", { 'entities': [(69, 74, 'color'), (159, 166, 'provider'), (138, 158, 'provider_category'), (109, 114, 'price'), (22, 26, 'brand'), (100, 103, 'size'), (85, 92, 'gender'), (14, 21, 'product'), (57, 62, 'brand'), (104, 108, 'brand')]}),
("Kengät Big Star  Shoes Big Top FF274343Kengät big star shoes big top ff274343 multicolour saatavana naisten kokoja 36,37,38,39,40,41  Big Star44.99{'color': ('multicolour'), 17 muoti ja vaatetusSpartoo  ('multicolour')", { 'entities': [(78, 89, 'color'), (193, 200, 'provider'), (0, 6, 'product'), (55, 60, 'product'), (115, 117, 'size'), (118, 120, 'size'), (121, 123, 'size'), (124, 126, 'size'), (127, 129, 'size'), (130, 132, 'size'), (7, 15, 'brand'), (17, 22, 'product'), (100, 108, 'gender'), (173, 193, 'provider_category'), (46, 54, 'brand'), (142, 147, 'price')]}),
("Starmaxx TR 110 ( 340/85 R38 133A8 TL kaksoistunnus  130B )  340/85 R38 133A8 TL kaksoistunnus  130BStarmaxx713.0{'size': ('38.0'), 02 ajoneuvotRengas Online('38.0')  ", { 'entities': [(124, 128, 'size'), (143, 156, 'provider'), (131, 143, 'provider_category'), (26, 28, 'size'), (108, 113, 'price'), (0, 8, 'brand')]}),
("Dana Buchman Silmälasit Poppi BLACKDana Buchman Poppi Silmälasit. Collection:Men. Kehyksen Väri: Black. Kehysmateriaali: Metal. Koko: 52.  Dana Buchman146.0{'size': ('52'), 'color': ('black'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses('52'),('black'),('male'),('adult')  \r", { 'entities': [(151, 156, 'price'), (13, 23, 'product'), (0, 12, 'brand'), (97, 102, 'color'), (134, 136, 'size'), (184, 189, 'color'), (236, 265, 'provider_category'), (265, 281, 'provider'), (30, 35, 'color'), (121, 126, 'material'), (77, 80, 'gender'), (205, 209, 'gender'), (228, 233, 'age_group')]}),
("Roberto Cavalli Silmälasit RC 5052 FOIANO 016Roberto Cavalli RC 5052 FOIANO Silmälasit. Collection:Women. Kehyksen Väri: Silver. Kehysmateriaali: Metal. Koko: 54.  Roberto Cavalli89.0{'size': ('54'), 'color': ('silver'), 'gender': ('female'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses('54'),('silver'),('female'),('adult')  \r", { 'entities': [(258, 263, 'age_group'), (233, 239, 'gender'), (295, 311, 'provider'), (266, 295, 'provider_category'), (16, 26, 'product'), (146, 151, 'material'), (99, 104, 'gender'), (179, 183, 'price'), (159, 161, 'size'), (211, 217, 'color'), (1, 15, 'brand')]}),
("Roberto Cavalli Silmälasit RC 5053 FOLLONICA A56Roberto Cavalli RC 5053 FOLLONICA Silmälasit. Collection:Women. Kehyksen Väri: Tortoise. Kehysmateriaali: Plastic. Koko: 49.  Roberto Cavalli138.0{'size': ('49'), 'color': ('tortoise'), 'gender': ('female'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses('49'),('tortoise'),('female'),('adult')  \r", { 'entities': [(16, 26, 'product'), (127, 135, 'color'), (169, 171, 'size'), (222, 230, 'color'), (308, 324, 'provider'), (279, 308, 'provider_category'), (271, 276, 'age_group'), (246, 252, 'gender'), (105, 110, 'gender'), (154, 161, 'material'), (0, 15, 'brand')]}),
("Femme, EdP 100ml Ed Hardy27.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(54, 65, 'provider'), (33, 54, 'provider_category'), (26, 31, 'price'), (12, 17, 'size'), (18, 26, 'brand')]}),
("Paradise Bay, EdP 85ml    Karl Lagerfeld37.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(68, 79, 'provider'), (47, 68, 'provider_category'), (40, 45, 'price'), (26, 40, 'brand'), (18, 22, 'size')]}),
("Rannekoru A1859 Leather Bracelet Black    Thomas Sabo € 169.0{'color': ('black'), 'gender': ('men'), 17 muoti ja vaatetus, Stayhard.\r", { 'entities': [(123, 131, 'provider'), (101, 121, 'provider_category'), (94, 97, 'gender'), (73, 78, 'color'), (33, 39, 'color'), (16, 23, 'age_group'), (24, 32, 'product'), (42, 53, 'brand'), (56, 61, 'price'), (0, 9, 'product')]}),
("Striped Flower Pussilakana 220x220cm, Lt AquaStriped Flower Pussilakana 220x220cm, Lt Aqua fron Simply Scandinavian .  Simply Scandinavian 123.0€  07 koti ja puutarha, Royal Design", { 'entities': [(168, 180, 'provider'), (147, 166, 'provider_category'), (27, 36, 'size'), (96, 115, 'brand'), (139, 144, 'price'), (15, 26, 'product')]}),
("Funktion 4 Matala Kattila ja kansi 140 cl High quality casserole with lid from WMF that fits all cookers including induction.  WMF $159.01,  07 koti ja puutarha   Royal Design", { 'entities': [(163, 175, 'provider'), (132, 138, 'price'), (141, 160, 'provider_category'), (79, 82, 'brand'), (35, 41, 'size'), (18, 25, 'product')]}),
("Mosakb Tie Dress Polvipituinen Mekko Monivärinen/Kuvioitu Karen By SimonsenKaren By Simonsen. A linjainen. V pääntie. 3/4 pituiset hihat. Etupituus: 106 cm. Takapituus: 118 cm.  Karen By Simonsen175.0{'size': ('34'), 'color': ('desert palm'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('34'),('desert palm'),('women')", { 'entities': [(149, 155, 'size'), (228, 239, 'color'), (31, 37, 'product'), (11, 17, 'product'), (37, 48, 'color'), (58, 75, 'brand'), (211, 213, 'size'), (255, 260, 'gender'), (283, 288, 'provider'), (263, 282, 'provider_category'), (195, 200, 'price'), (169, 175, 'size')]}),
("Bitz kulho  14 cm musta Musta meripihka    Bitz10.0  07 koti ja puutarhaNordic Nest", { 'entities': [(73, 84, 'provider'), (54, 73, 'provider_category'), (48, 52, 'price'), (25, 30, 'color'), (19, 25, 'color'), (13, 18, 'size'), (0, 4, 'brand'), (5, 11, 'product')]}),
("Akribos XXIV 99999 Miesten kello AK849SSB Musta/Teräs Ø45 mm", {'entities': [(0, 12, 'brand'), (19, 26, 'gender'), (27, 32, 'product'), (42, 47, 'color'), (48, 53, 'material'), (54, 60, 'size')]}),  
("Vidreamers Pure T Shirt Noos T shirts & Tops Short sleeved Valkoinen Vila", {'entities': [(16, 23, 'product'), (29, 37, 'product'), (59, 68, 'color'), (69, 73, 'brand')]}),
("Etujousen Norra", {'entities': [(10, 15, 'brand'), (0, 9, 'product')]}), 
("Blue Polo Short Sleeve Popover Shirt Polos Short sleeved Sininen Eton", {'entities': [(57, 64, 'color'), (65, 69, 'brand'), (31, 36, 'product'), (0, 4, 'color')]}), 
("Cultor RD 01 (320/85 R32 142A8 TL kaksoistunnus 142B) 320/85 R32 142A8 TL kaksoistunnus 142B Cultor 630€. SIZE:32.0. 02 ajoneuvot, Rengas Online  ", { 'entities': [(130, 143, 'provider'), (116, 128, 'provider_category'), (110, 114, 'size'), (99, 103, 'price'), (0, 6, 'brand'), (92, 98, 'brand'), (15, 17, 'size')]}),
("DBKD Oblong Ruukku/Maljakko Small, MoleOblong Ruukku/Maljakko Small, Mole fron DBKD .  DBKD $ 23.99  07 koti ja puutarha, Rum21", { 'entities': [(122, 127, 'provider'), (101, 120, 'provider_category'), (94, 99, 'price'), (28, 33, 'size'), (12, 18, 'product'), (19, 27, 'product'), (0, 4, 'brand')]}),
("Poikien sandaalit Geox  JR SANDAL STRADAPoikien sandaalit geox jr sandal strada blue saatavana poikien kokoja 29  Geox53.59{'color': ('blue'), 17 muoti ja vaatetus, Spartoo  ('blue')", { 'entities': [(118, 123, 'price'), (18, 22, 'brand'), (110, 113, 'size'), (95, 102, 'gender'), (63, 66, 'gender'), (80, 85, 'color'), (66, 73, 'product'), (8, 18, 'product'), (0, 8, 'gender'), (24, 27, 'gender'), (27, 34, 'product'), (142, 162, 'provider_category'), (164, 171, 'provider')]}),
("Moncler Aurinkolasit ML0073 01AMoncler ML0073 Aurinkolasit. Collection:Men. Kehyksen Väri: Black.  Linssin Väri: . Kehysmateriaali: Plastic. Koko: 51.  Moncler146.0{'size': ('51'), 'color': ('black'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses('51'),('black'),('male'),('adult')  \r", { 'entities': [(8, 20, 'product'), (91, 96, 'color'), (192, 197, 'color'), (273, 289, 'provider'), (213, 217, 'gender'), (236, 241, 'age_group'), (244, 273, 'provider_category'), (147, 149, 'size'), (159, 164, 'price'), (0, 7, 'brand'), (99, 105, 'material'), (132, 139, 'material'), (71, 74, 'gender')]}),
("Police Aurinkolasit SPL997 ORIGINS LITE 3 0722 Police SPL997 ORIGINS LITE 3 Aurinkolasit. Collection:Men. Kehyksen Väri: Dark Havana. Linssin Väri:. Kehysmateriaali: Plastic. Koko: 53. Police 94.0€ {'size': ('53'), 'color': ('dark havana'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses ('53'),('dark havana'),('male'),('adult')  \r", { 'entities': [(264, 269, 'age_group'), (305, 321, 'provider'), (275, 304, 'provider_category'), (241, 245, 'gender'), (100, 103, 'gender'), (164, 171, 'material'), (133, 140, 'material'), (120, 131, 'color'), (190, 195, 'price'), (225, 236, 'color'), (7, 19, 'product'), (179, 181, 'size'), (0, 6, 'brand'), (183, 189, 'brand')]}),
("Tommy Girl Neon Brights, EdT 50ml Tommy Hilfiger 16.95€ 06 kauneus ja terveys Hajuvesi.fi", { 'entities': [(77, 88, 'provider'), (55, 76, 'provider_category'), (49, 54, 'price'), (29, 33, 'size'), (0, 10, 'brand'), (34, 48, 'brand'), (11, 23, 'color')]}),
("Mini Foodprocessor 0,95 litraa Punainen    KitchenAid122.15{'color': ('punainen'), 07 koti ja puutarhaMakujakauppa  ('punainen')", { 'entities': [(71, 79, 'color'), (101, 113, 'provider'), (82, 101, 'provider_category'), (31, 39, 'color'), (19, 24, 'size'), (5, 18, 'product'), (0, 5, 'size'), (53, 59, 'price'), (43, 53, 'brand')]}),
("SP700+ Maidonvaahdotin, jossa 13 Ohjelmaa Severin 114.3€ {'color': ('metalli'), 07 koti ja puutarha Makujakauppa  ('metalli')", { 'entities': [(102, 114, 'provider'), (82, 101, 'provider_category'), (69, 76, 'color'), (50, 56, 'price'), (42, 49, 'brand')]}),
("Aromatics Elixir, Perfume Spray 100ml    Clinique68.95  06 kauneus ja terveysHajuvesi.fi", { 'entities': [(77, 88, 'provider'), (18, 31, 'product'), (32, 37, 'size'), (49, 54, 'price'), (41, 49, 'brand'), (56, 77, 'provider_category')]}),
("Ozka Agro 10 Traction ( 480/70 R30 141A8 TL )  480/70 R30 141A8 TLOzka1104.9{'size': ('30.0'), 02 ajoneuvotRengas Online('30.0')  ", { 'entities': [(106, 118, 'provider'), (94, 106, 'provider_category'), (87, 91, 'size'), (70, 76, 'price'), (0, 4, 'brand'), (32, 34, 'size')]}),
("G3 SmaragdSärkymättämät huippulaadukkaat lasit neutraalin harmaalla linssillä. Cat 3.  Gloryfy €159.9 for men. 16 urheilu ja eraScandinavian Outdoor.   \r", { 'entities': [(128, 148, 'provider'), (111, 128, 'provider_category'), (106, 109, 'gender'), (96, 101, 'price'), (87, 94, 'brand'), (68, 77, 'material'), (41, 46, 'product'), (58, 68, 'color')]}),
("Pleated Georgette Hihaton Pusero Paita Ganni", {'entities': [(39, 44, 'brand'), (33, 38, 'product'), (26, 32, 'product')]}), 
("Pusero Charlie Tank", {'entities': [(0, 6, 'product'), (7, 19, 'brand')]}),
("gForce Action lever Belt, 11mm, black", {'entities': [(20, 24, 'product'), (26, 30, 'size'), (32, 37, 'color'), (0, 6, 'brand')]}),
("Clara Velvet Skirt Polvipituinen Hame Oranssi Just Female ust Female.  Just Female95.0{'size': ('m', 'l', 'xs', 's'), 'color': ('pumpkin spice'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('m', 'l', 'xs', 's'),('pumpkin spice'),('women')", { 'entities': [(158, 163, 'gender'), (129, 142, 'color'), (186, 191, 'provider'), (166, 186, 'provider_category'), (33, 37, 'product'), (13, 18, 'product'), (107, 109, 'size'), (82, 86, 'price'), (46, 57, 'brand'), (38, 45, 'color')]}),
("Leivänpaahdin Cube, 2 viipaleelle    OBH Nordica 47.48€. Metalli color. 07 koti ja puutarha, Makujakauppa", { 'entities': [(93, 105, 'provider'), (72, 91, 'provider_category'), (57, 64, 'color'), (49, 54, 'price'), (37, 48, 'brand'), (0, 18, 'product')]}),
("Bkt Tr 135 ( 12.4  36 127A6 8Pr Tt )AS ristikudosrengas12.4  36 127A6 8PR TTBKT580.4{'size': ('36.0'), 02 ajoneuvotRengas Online('36.0')  ", { 'entities': [(95, 99, 'size'), (114, 127, 'provider'), (102, 114, 'provider_category'), (79, 84, 'price'), (0, 3, 'brand'), (39, 55, 'product'), (19, 22, 'size')]}),
("G2 Cuba LibreSärkymättämät huippulaadukkaat aurinkolasit lähes kirkkailla linsseillä.  Gloryfy139.9{'gender': ('men'), 16 urheilu ja eraScandinavian Outdoor   \r", { 'entities': [(135, 155, 'provider'), (118, 135, 'provider_category'), (94, 99, 'price'), (87, 94, 'brand'), (44, 56, 'product'), (74, 81, 'material'), (112, 115, 'gender')]}),
("Aromatics Elixir, Perfume Spray 10ml    Clinique31.95  06 kauneus ja terveysHajuvesi.fi", { 'entities': [(76, 87, 'provider'), (55, 76, 'provider_category'), (48, 53, 'price'), (40, 47, 'brand'), (32, 36, 'size'), (18, 31, 'product')]}),
("Korkokengät Etro  3074Korkokengät etro 3074 brown saatavana naisten kokoja 37,38  Etro €372.8. 'color': 'brown'. 17 muoti ja vaatetus, Spartoo. ", { 'entities': [(135, 142, 'provider'), (113, 133, 'provider_category'), (44, 49, 'color'), (88, 93, 'price'), (78, 80, 'size'), (75, 77, 'size'), (0, 11, 'product'), (60, 68, 'gender'), (12, 16, 'brand')]}),
("Sandaalit Pikolinos ALGAR W0XSandaalit pikolinos algar w0x brown saatavana naisten kokoja 36,38 Pikolinos 64.39€ brown 17 muoti ja vaatetus Spartoo. ", { 'entities': [(59, 64, 'color'), (90, 92, 'size'), (93, 95, 'size'), (10, 19, 'brand'), (96, 105, 'brand'), (106, 112, 'price'), (0, 9, 'product'), (39, 48, 'brand'), (75, 82, 'gender'), (119, 139, 'provider_category'), (140, 147, 'provider')]}),
("Kengät Les Tropä©ziennes par M Belarbi  AMELIEKengät les tropä©ziennes par m belarbi amelie black saatavana naisten kokoja 36,38  Les Tropä©ziennes par M Belarbi81.6{'color': ('black'), 17 muoti ja vaatetusSpartoo  ('black')", { 'entities': [(90, 95, 'color'), (202, 209, 'provider'), (182, 202, 'provider_category'), (158, 162, 'price'), (106, 114, 'gender'), (124, 126, 'size'), (121, 123, 'size'), (7, 37, 'brand'), (0, 6, 'product'), (52, 82, 'brand')]}),
("Kengät Everlast  EVL SHADOW MIDKengät everlast evl shadow mid white saatavana miesten kokoja 43,44  Everlast 53.59€, 17 muoti ja vaatetus, Spartoo", { 'entities': [(0, 6, 'product'), (93, 95, 'size'), (7, 15, 'brand'), (139, 146, 'provider'), (117, 137, 'provider_category'), (109, 114, 'price'), (78, 86, 'gender'), (62, 68, 'color'), (38, 47, 'brand')]}),
("Farah Aurinkolasit FHS 5015 001Farah FHS 5015 Aurinkolasit. Collection:Men. Kehyksen Väri: Gold.  Linssin Väri: . Kehysmateriaali: Metal. Koko: 56.  Farah ä59.0   gold male adult. 13 silmalasit ja piilolinssit, Smartbuy Glasses  \r", { 'entities': [(211, 227, 'provider'), (180, 209, 'provider_category'), (173, 178, 'age_group'), (168, 173, 'gender'), (131, 136, 'material'), (71, 74, 'gender'), (6, 18, 'product'), (144, 146, 'size'), (91, 95, 'color'), (0, 5, 'brand'), (156, 160, 'price'), (98, 105, 'material')]}),
("Ceat Farmax R85 ( 380/85 R34 137A8 TL kaksoistunnus  137B )  380/85 R34 137A8 TL kaksoistunnus  137B Ceat 882.3 size: 34.0. 02 ajoneuvot Rengas Online  ", { 'entities': [(137, 150, 'provider'), (124, 136, 'provider_category'), (118, 122, 'size'), (106, 111, 'price'), (0, 4, 'brand'), (26, 28, 'size')]}),
("Diesel Silmälasit DL5050 B05Diesel DL5050 Silmälasit. Collection:Men. Kehyksen Väri: Black. Kehysmateriaali: Plastic. Koko: 52.  Diesel €100.0 SIZE: 52, COLOR: black, GENDER: male, AGE_GROUP: adult. 13 silmalasit ja piilolinssit   Smartbuy Glasses  \r", { 'entities': [(231, 247, 'provider'), (0, 6, 'brand'), (7, 17, 'product'), (85, 90, 'color'), (124, 126, 'size'), (137, 142, 'price'), (160, 165, 'color'), (109, 116, 'material'), (65, 68, 'gender'), (175, 179, 'gender'), (192, 197, 'age_group'), (199, 228, 'provider_category')]}),
("Gi2 DeJaVu Angel VioletSärkymättämät huippulaadukkaat aurinkolasit savulinsseillä.  Gloryfy79.0{'gender': ('men'), 16 urheilu ja eraScandinavian Outdoor  \r", { 'entities': [(131, 151, 'provider'), (114, 131, 'provider_category'), (108, 111, 'gender'), (54, 66, 'product'), (84, 91, 'brand'), (91, 95, 'price')]}),
("Nonogarden T shirts & Tops Short sleeved Sininen American VintageAmerican Vintage. V pääntie.  American Vintage85.0{'size': ('m', 's'), 'color': ('hurricane'), 'gender': ('women'), 17 muoti ja vaatetus Boozt   \r", { 'entities': [(201, 206, 'provider'), (180, 200, 'provider_category'), (172, 177, 'gender'), (147, 156, 'color'), (111, 115, 'price'), (49, 65, 'brand'), (41, 48, 'color'), (22, 26, 'product'), (11, 19, 'product')]}),
("Crewneck T Shirt T shirts Short sleeved Valkoinen ChampionChampion. Yksityiskohdat: athletic jersey combed (180gr). Ulkomateriaali: 100% puuvillaa.  Champion40.0{'size': ('s', 'l', 'xl', 'xxl'), 'color': ('white'), 'gender': ('men'), 17 muoti ja vaatetusBoozt('s', 'l', 'xl', 'xxl'),('white'),('men')", { 'entities': [(157, 161, 'price'), (50, 58, 'brand'), (0, 9, 'brand'), (188, 191, 'size'), (182, 184, 'size'), (33, 34, 'size'), (108, 113, 'size'), (253, 258, 'provider'), (233, 253, 'provider_category'), (40, 50, 'color'), (206, 211, 'color'), (227, 230, 'gender'), (9, 16, 'product'), (17, 25, 'product')]}),
("Classic Chino Suoralahkeiset Housut Vaaleanpunainen GANT GANT. Suorat lahkeet.  Gant 99.9{'size': ('38', '44'), 'color': ('rapture rose'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('38', '44'),('rapture rose'),('women')", { 'entities': [(151, 156, 'gender'), (123, 135, 'color'), (100, 102, 'size'), (106, 108, 'size'), (29, 36, 'product'), (36, 52, 'color'), (52, 57, 'brand'), (80, 85, 'brand'), (179, 184, 'provider'), (159, 179, 'provider_category')]}),
("All About Shadow Single, Purple Pumps Beauty WOMEN Makeup Eyes Eyeshadow   Not Palettes Liila CliniqueClinique All About Shadow Single, Purple Pumps  Clinique26.99{'size': ('2.20g'), 'color': ('purple pumps'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('2.20g'),('purple pumps'),('women')", { 'entities': [(158, 163, 'price'), (94, 102, 'brand'), (174, 179, 'size'), (194, 206, 'color'), (25, 37, 'color'), (88, 93, 'color'), (250, 255, 'provider'), (230, 249, 'provider_category'), (63, 73, 'product'), (45, 51, 'gender'), (222, 227, 'gender')]}),
("G12 Titan PolarizedSärkymättämät huippulaadukkaat aurinkolasit polarisoivilla linsseillä ja vaihtuvalla tummennuksella 2 3.  Gloryfy199.0{'gender': ('men'), 16 urheilu ja eraScandinavian Outdoor   \r", { 'entities': [(173, 193, 'provider'), (156, 173, 'provider_category'), (125, 132, 'brand'), (132, 137, 'price'), (107, 110, 'gender'), (50, 63, 'product'), (78, 85, 'material')]}),
("Mitas TS06 (6.50/80 15 94A8 6PR TT kaksoistunnus 82A8) 6.50/80 15 94A8 6PR TT kaksoistunnus 82A8 Mitas 132.5€ {'size': ('15.0'), 02 ajoneuvot Rengas Online ('15.0')  ", { 'entities': [(124, 128, 'size'), (130, 142, 'provider_category'), (143, 156, 'provider'), (102, 108, 'price'), (0, 5, 'brand'), (96, 101, 'brand'), (120, 124, 'size')]}),
("Scoop Teepurkki MittalusikallaScoop is an elegant and practical tea canister with a specially designed measuring scoop that is integrated into the lid so you always have it on hand when you need it.  Stelton36.0  07 koti ja puutarhaRoyal Design", { 'entities': [(232, 244, 'provider'), (213, 232, 'provider_category'), (207, 211, 'price'), (200, 207, 'brand'), (6, 15, 'product')]}),
("Lyhythihainen t paita adidas  TREFOIL T SHIRTLyhythihainen t paita adidas trefoil t shirt red saatavana miesten kokoja xl  adidas €39.99. 17 muoti ja vaatetus, Spartoo", { 'entities': [(160, 167, 'provider'), (138, 158, 'provider_category'), (131, 136, 'price'), (119, 122, 'size'), (22, 29, 'brand'), (82, 89, 'product'), (90, 94, 'color'), (104, 112, 'gender'), (14, 21, 'product'), (38, 45, 'product')]}),
("Wool Slipper Infant Aamutossut Sisäkengät Harmaa Hummel Hummel Wool Slipper Infant  Hummel29.95{'size': ('24', '27', '26', '29', '30', '25', '28', '22'), 'color': ('alloy'), 'gender': ('kids'), 17 muoti ja vaatetusBoozt('24', '27', '26', '29', '30', '25', '28', '22'),('alloy'),('kids')", { 'entities': [(186, 190, 'age_group'), (165, 170, 'color'), (148, 150, 'size'), (142, 144, 'size'), (136, 138, 'size'), (118, 120, 'size'), (112, 114, 'size'), (106, 108, 'size'), (130, 132, 'size'), (5, 12, 'product'), (0, 4, 'material'), (13, 19, 'age_group'), (31, 42, 'product'), (42, 49, 'color'), (90, 95, 'price'), (49, 55, 'brand'), (213, 218, 'provider'), (193, 213, 'provider_category')]}),
("Olkalaukut Desigual  MEX KIEV MINIOlkalaukut desigual mex kiev mini black saatavana naisten kokoja yksi koko  Desigual65.59{'color': ('black'), 17 muoti ja vaatetusSpartoo  ('black')", { 'entities': [(68, 73, 'color'), (99, 108, 'size'), (84, 92, 'gender'), (0, 10, 'product'), (30, 44, 'product'), (45, 53, 'brand'), (11, 19, 'brand'), (118, 123, 'price'), (163, 170, 'provider'), (143, 163, 'provider_category')]}),
("Mitas FL08 ( 18x7.00  8 121A5 14PR TT NHS )  18x7.00  8 121A5 14PR TT NHSMitas112.2{'size': ('8.0'), 02 ajoneuvotRengas Online('8.0')  ", { 'entities': [(112, 125, 'provider'), (100, 112, 'provider_category'), (94, 97, 'size'), (78, 83, 'price'), (9, 11, 'size'), (0, 6, 'brand')]}),
("Mid Rise True Skinny Jeans Skinny Farkut Sininen GAPGAP Mid Rise True Skinny Jeans  GAP65.0{'size': ('34', '32'), 'color': ('medium indigo 8'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('34', '32'),('medium indigo 8'),('women')", { 'entities': [(184, 189, 'provider'), (164, 184, 'provider_category'), (108, 110, 'size'), (102, 104, 'size'), (125, 140, 'color'), (87, 91, 'price'), (49, 52, 'brand'), (21, 26, 'product'), (34, 41, 'product'), (41, 49, 'color'), (14, 20, 'material'), (156, 161, 'gender')]}),
("Metsästäjän Sukat 37 39 Paksut ja mukvat sukat joissa 75% villaa.  Tesema8.9{'size': ('37 39'), 'gender': ('men'), 16 urheilu ja eraScandinavian Outdoor('37 39')  ('men')", { 'entities': [(18, 23, 'size'), (108, 111, 'gender'), (12, 17, 'product'), (41, 46, 'product'), (58, 64, 'material'), (114, 131, 'provider_category'), (131, 151, 'provider'), (73, 76, 'price'), (67, 73, 'brand')]}),
("Aromatics Elixir, Perfume Spray 25ml    Clinique28.95  06 kauneus ja terveysHajuvesi.fi", { 'entities': [(76, 87, 'provider'), (55, 76, 'provider_category'), (48, 53, 'price'), (40, 48, 'brand'), (32, 36, 'size'), (18, 31, 'product')]}),
("Bisgaard Jacob Tennarit Sneakerit Kengät Vihreä BisgaardBisgaard. Sisävuori: nahka. Ylävuori: nahka. Pohjallinen: nahka. Ulkopohja: tpr. Vegetable tanned leather is toxic free making it recyclable. Extracts from raw materials are used giving it warm and natural tones that mature with use over time.. Vähemmän kemikaaleja. Tarrakiinnitys. Valmistettu  Portugali.  Bisgaard104.95{'color': ('mint'), 'gender': ('kids'), 17 muoti ja vaatetusBoozt  ('mint'),('kids')", { 'entities': [(410, 414, 'age_group'), (390, 394, 'color'), (437, 442, 'provider'), (417, 437, 'provider_category'), (372, 378, 'price'), (0, 8, 'brand'), (41, 48, 'color'), (15, 24, 'product'), (24, 34, 'product'), (34, 41, 'product'), (154, 162, 'material'), (77, 82, 'material')]}),
("GI Slow Juice new horizontal Hurom 379.9€ {'color': ('harmaa'), 07 koti ja puutarha Makujakauppa  ('harmaa')", { 'entities': [(54, 60, 'color'), (86, 98, 'provider'), (66, 85, 'provider_category'), (29, 34, 'brand'), (35, 41, 'price')]}),
("Bvlgari Aurinkolasit BV5047Q 202273Bvlgari BV5047Q Aurinkolasit. Collection:Men. Kehyksen Väri: Gold. Linssin Väri: . Kehysmateriaali: Metal. Koko: 59. Bvlgari 244€ {'size': ('59'), 'color': ('gold'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses ('59'),('gold'),('male'),('adult')  \r", { 'entities': [(236, 241, 'age_group'), (213, 217, 'gender'), (135, 140, 'material'), (102, 109, 'material'), (96, 100, 'color'), (76, 79, 'gender'), (8, 20, 'product'), (51, 63, 'product'), (194, 198, 'color'), (148, 150, 'size'), (277, 293, 'provider'), (247, 276, 'provider_category'), (160, 163, 'price'), (152, 159, 'brand'), (0, 7, 'brand')]}),
("Le Klint 6 Varjostin 21 cm, MuoviLe Klint 6 Varjostin 21 cm, Muovi fron Le Klint .  Le Klint 175€  07 koti ja puutarhaRoyal Design", { 'entities': [(118, 130, 'provider'), (99, 118, 'provider_category'), (93, 97, 'price'), (21, 24, 'size'), (28, 33, 'product'), (0, 8, 'brand')]}),
("General Tractor V.Ply ( 14.9  28 137A6 12PR TT )  14.9  28 137A6 12PR TTGeneral819.3{'size': ('28.0'), 02 ajoneuvotRengas Online('28.0')  ", { 'entities': [(95, 99, 'size'), (114, 127, 'provider'), (102, 114, 'provider_category'), (79, 84, 'price'), (0, 7, 'brand'), (30, 33, 'size')]}),
("Diesel Silmälasit DL5045 090Diesel DL5045 Silmälasit. Collection:Men. Kehyksen Väri: Shiny Blue. Kehysmateriaali: Plastic. Koko: 54.  Diesel108.0{'size': ('54'), 'color': ('shiny blue'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses('54'),('shiny blue'),('male'),('adult')  \r", { 'entities': [(0, 6, 'brand'), (140, 145, 'price'), (230, 259, 'provider_category'), (259, 275, 'provider'), (129, 131, 'size'), (114, 121, 'material'), (7, 17, 'product'), (85, 95, 'color'), (173, 183, 'color'), (65, 68, 'gender'), (199, 203, 'gender'), (222, 227, 'age_group')]}),
("Clubman Classic Vanilla After Shave 177 mlAfter shave maskuliinisella tuoksulla Clubmanilta  Clubman9.95  17 muoti ja vaatetusShopping4net", { 'entities': [(42, 53, 'product'), (36, 42, 'size'), (24, 35, 'product'), (126, 138, 'provider'), (106, 126, 'provider_category'), (100, 104, 'price'), (0, 7, 'brand')]}),
("Cordoba Rug 160x230 cm, IvoryCordoba Rug 160x230 cm, Ivory fron Linie Design .  Linie Design583.0  07 koti ja puutarhaRoyal Design", { 'entities': [(118, 130, 'provider'), (99, 118, 'provider_category'), (92, 97, 'price'), (64, 76, 'brand'), (24, 29, 'color'), (0, 11, 'product'), (12, 19, 'size')]}),
("Refine Tyynynpäällinen RaidVaikutteina olleet matkat Marokkoon, Intiaan, Afrikkaan, Kiinaan, Indonesiaan ja Karibialle.  DAY Home €69.99  17 muoti ja vaatetus   Shopping4net", { 'entities': [(7, 22, 'product'), (130, 136, 'price'), (121, 129, 'brand'), (161, 173, 'provider'), (138, 158, 'provider_category')]}),
("BKT Maglift LIP (15x4.50 8 109A5 TL) 15x4.50 8 109A5 TL BKT €104.3 {'size': ('8.0'), 02 ajoneuvot Rengas Online  ", { 'entities': [(97, 110, 'provider'), (84, 96, 'provider_category'), (75, 78, 'size'), (58, 64, 'price'), (0, 3, 'brand'), (54, 57, 'brand'), (18, 25, 'size')]}),
("Lyhythihainen t paita Gant LE MANS BLOCKLyhythihainen t paita gant le mans block blue saatavana miesten kokoja s Gant 39.99€ {'color': ('blue'), 17 muoti ja vaatetus Spartoo", { 'entities': [(168, 175, 'provider'), (147, 167, 'provider_category'), (81, 85, 'color'), (119, 125, 'price'), (22, 26, 'brand'), (62, 66, 'brand'), (114, 118, 'brand'), (14, 21, 'product'), (30, 34, 'gender'), (70, 74, 'gender'), (96, 103, 'gender'), (112, 113, 'size')]}),
("Day Paris Bag Bags Small Shoulder Bags   Crossbody Bags Musta DAY EtDAY et. Hakaskiinnitys. Säädettävä olkain. Täysvuorattu. Mitat: 18 x 11 x 6 cm.  Day Et110.0{'size': ('one size'), 'color': ('black'), 'gender': ('women'), 17 muoti ja vaatetusBoozt('one size'),('black'),('women')", { 'entities': [(215, 220, 'gender'), (194, 199, 'color'), (171, 179, 'size'), (132, 146, 'size'), (149, 155, 'brand'), (68, 74, 'brand'), (62, 68, 'brand'), (56, 61, 'color'), (14, 18, 'product'), (10, 13, 'product'), (155, 160, 'price'), (243, 248, 'provider'), (223, 243, 'provider_category')]}),
("Komfort XXL Kolgrillsvagn    Landmann345.9  07 koti ja puutarhaMakujakauppa", { 'entities': [(63, 75, 'provider'), (44, 63, 'provider_category'), (37, 42, 'price'), (29, 37, 'brand'), (8, 11, 'size'), (12, 25, 'product')]}),
("Jane Top T shirts & Tops Short sleeved Valkoinen Ida SjästedtIda Sjästedt. Sisävuori: 100% polyesteri. Ulkomateriaali: 100% nailonia. Tyylikästä hienostuneisuutta modernilla vivahteella. Etupituus: 54 cm. Takapituus: 54 cm.  Ida Sjästedt187.0{'size': ('44'), 'color': ('ivory'), 'gender': ('women'), 17 muoti ja vaatetus Boozt", { 'entities': [(320, 325, 'provider'), (299, 319, 'provider_category'), (237, 242, 'price'), (49, 61, 'brand'), (198, 200, 'size'), (91, 101, 'material'), (253, 255, 'size'), (291, 296, 'gender'), (39, 49, 'color'), (270, 275, 'color'), (124, 132, 'material'), (9, 17, 'product'), (20, 24, 'product')]}),
("CLASSIC Veitsitukki Setti 7 osaa    Wästhof414.9  07 koti ja puutarhaMakujakauppa", { 'entities': [(69, 81, 'provider'), (50, 69, 'provider_category'), (43, 48, 'price'), (35, 43, 'brand'), (8, 19, 'product')]}),
("Aromatics Elixir, Perfume Spray 45ml    Clinique 40.95€  06 kauneus ja terveys   Hajuvesi.fi", { 'entities': [(81, 92, 'provider'), (57, 78, 'provider_category'), (49, 54, 'price'), (40, 48, 'brand'), (32, 36, 'size'), (18, 31, 'product')]}),
("Puma Silmälasit PJ0029O Kids 001Puma PJ0029O Kids Silmälasit. Collection:Kids. Kehyksen Väri: Black. Kehysmateriaali: Plastic. Koko: 50.  PUMA46.0{'size': ('50'), 'color': ('black'), 'gender': ('unisex'), 'AGE_GROUP': ('kids'), 13 silmalasit ja piilolinssit Smartbuy Glasses  \r", { 'entities': [(257, 273, 'provider'), (227, 256, 'provider_category'), (174, 179, 'color'), (133, 135, 'size'), (142, 146, 'price'), (138, 142, 'brand'), (5, 15, 'product'), (0, 4, 'brand'), (94, 99, 'color'), (24, 28, 'age_group'), (195, 201, 'gender'), (118, 125, 'material'), (220, 224, 'age_group')]}),
("CLASSIC Veitsitukki Setti 9 osaa Musta    Wästhof390.9  07 koti ja puutarhaMakujakauppa", { 'entities': [(75, 87, 'provider'), (56, 75, 'provider_category'), (49, 54, 'price'), (42, 49, 'brand'), (33, 38, 'color'), (8, 19, 'product')]}),
("Kokkiveitsi Red    Brabantia 13.12€ COLOR: punainen, 07 koti ja puutarha   Makujakauppa", { 'entities': [(75, 87, 'provider'), (53, 72, 'provider_category'), (43, 51, 'color'), (29, 34, 'price'), (19, 28, 'brand'), (12, 16, 'color'), (0, 11, 'product')]}),
("Cultor AS Front 08 ( 6.50 16 91A6 8PR TT kaksoistunnus 83A8) 6.50 16 91A6 8PR TT kaksoistunnus 83A8 Cultor 293.6€ {'size': ('16.0'), 02 ajoneuvot Rengas Online ('16.0')  ", { 'entities': [(148, 161, 'provider'), (135, 147, 'provider_category'), (125, 129, 'size'), (107, 113, 'price'), (0, 6, 'brand'), (100, 106, 'brand')]}),
("Balleriinat Ikks  ANABELBalleriinat ikks anabel white saatavana tyttäjen kokoja 24,25,27 varsi: nahkaa  Ikks 64.79€ 17 muoti ja vaatetus   Spartoo", { 'entities': [(139, 146, 'provider'), (116, 136, 'provider_category'), (109, 115, 'price'), (12, 16, 'brand'), (86, 88, 'size'), (83, 85, 'size'), (80, 82, 'size'), (48, 54, 'color'), (36, 40, 'brand'), (0, 12, 'product'), (64, 72, 'gender'), (96, 102, 'material')]}),
("Bleiseri Le Temps des Cerises  MIMOBleiseri le temps des cerises mimo beige saatavana naisten kokoja l,xs  Le Temps des Cerises65.59{'color': ('beige'), 17 muoti ja vaatetus. Spartoo", { 'entities': [(174, 181, 'provider'), (152, 172, 'provider_category'), (70, 75, 'color'), (103, 105, 'size'), (127, 132, 'price'), (9, 29, 'brand'), (44, 64, 'brand'), (86, 94, 'gender'), (0, 9, 'product')]}),
("Tiffany & Co. Silmälasit TF2175 8134Tiffany & Co. TF2175 Silmälasit. Collection:Women. Kehyksen Väri: Havana/Blue. Kehysmateriaali: Plastic. Koko: 52. Tiffany & Co. 162.0€ {'size': ('52'), 'color': ('havana/blue'), 'gender': ('female'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses  \r", { 'entities': [(294, 310, 'provider'), (263, 292, 'provider_category'), (207, 211, 'color'), (200, 206, 'color'), (102, 108, 'color'), (109, 113, 'color'), (165, 171, 'price'), (147, 149, 'size'), (0, 13, 'brand'), (151, 164, 'brand'), (227, 233, 'gender'), (252, 257, 'age_group'), (132, 139, 'material'), (80, 85, 'gender'), (14, 24, 'product')]}),
("Cazal Silmälasit 4268 003Cazal 4268 Silmälasit. Collection:Women. Kehyksen Väri: Cream/Gold. Kehysmateriaali: Metal. Koko: 53.  Cazal270.0{'size': ('53'), 'color': ('cream/gold'), 'gender': ('female'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit, Smartbuy Glasses \r", { 'entities': [(256, 272, 'provider'), (225, 254, 'provider_category'), (123, 125, 'size'), (166, 171, 'color'), (172, 176, 'color'), (133, 138, 'price'), (0, 5, 'brand'), (81, 86, 'color'), (87, 91, 'color'), (110, 115, 'material'), (217, 222, 'age_group'), (59, 64, 'gender'), (192, 198, 'gender'), (6, 16, 'product')]}),
("Timberland Silmälasit TB1652 002Timberland TB1652 Silmälasit. Collection:Men. Kehyksen Väri: Matte Black. Kehysmateriaali: Metal. Koko: 50. Timberland 78€ {'size': ('50'), 'color': ('matte black'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses  \r", { 'entities': [(275, 291, 'provider'), (245, 274, 'provider_category'), (94, 105, 'color'), (137, 139, 'size'), (183, 194, 'color'), (0, 10, 'brand'), (141, 151, 'brand'), (152, 154, 'price'), (124, 129, 'material'), (74, 77, 'gender'), (211, 215, 'gender'), (234, 239, 'age_group'), (11, 21, 'product')]}),
("Coolmax Saumaton Musta/pinkki 34   36Matalat Coolmax sukat sporttikäyttään.  Tesema3.9{'size': ('34   36'), 'color': ('musta/pinkki'), 'gender': ('men, women'), 16 urheilu ja eraScandinavian Outdoor('34   36'),('musta/pinkki'),('men, women')", { 'entities': [(152, 157, 'gender'), (147, 150, 'gender'), (17, 22, 'color'), (23, 29, 'color'), (30, 37, 'size'), (53, 58, 'product'), (119, 124, 'color'), (177, 197, 'provider'), (160, 177, 'provider_category'), (83, 86, 'price'), (77, 83, 'brand')]}),
("Urban Mini Talvikengät Tarroilla Musta ECCOECCO Urban Mini  Ecco89.9{'size': ('21', '20'), 'color': ('night sky', 'night sky', 'scarlet'), 'gender': ('kids'), 17 muoti ja vaatetus. Boozt", { 'entities': [(180, 185, 'provider'), (158, 178, 'provider_category'), (64, 68, 'price'), (60, 64, 'brand'), (79, 81, 'size'), (85, 87, 'size'), (102, 111, 'color'), (128, 135, 'color'), (11, 22, 'product'), (39, 43, 'brand'), (33, 39, 'color'), (151, 155, 'age_group')]}),
("Stockholm Vase 22,5 cm, LignumAluminum vase with artistic pattern from Stelton.  Stelton195.0€ 07 koti ja puutarha, Royal Design", { 'entities': [(116, 128, 'provider'), (95, 114, 'provider_category'), (88, 93, 'price'), (71, 78, 'brand'), (15, 20, 'size'), (10, 14, 'product'), (39, 43, 'product'), (30, 38, 'material')]}),
("Idun Hanskat Käsineet Ruskea HestraHestra Idun  Hestra85.0{'size': ('7', '8', '6', '9'), 'color': ('chocolate'), 'gender': ('women'), 17 muoti ja vaatetus Boozt", { 'entities': [(22, 29, 'color'), (5, 13, 'product'), (13, 22, 'product'), (69, 70, 'size'), (84, 85, 'size'), (79, 80, 'size'), (100, 109, 'color'), (125, 130, 'gender'), (155, 160, 'provider'), (134, 154, 'provider_category'), (54, 58, 'price'), (29, 35, 'brand')]}),
("Beauty, Shower Gel 200ml Calvin Klein 11.95€ 06 kauneus ja terveys Hajuvesi.fi", { 'entities': [(67, 78, 'provider'), (45, 66, 'provider_category'), (38, 44, 'price'), (25, 37, 'brand'), (19, 24, 'size'), (8, 18, 'product')]}),
("Lr Gilmore T shirts & Tops Long sleeved Monivärinen/Kuvioitu Levete RoomLevete Room. Etupituus: 65 cm. Takapituus: 65 cm.  Levete Room59.99{'size': ('m', 's', 'l', 'xs'), 'color': ('l500c   lavendula combi'), 'gender': ('women'), 17 muoti ja vaatetusBoozt", { 'entities': [(249, 254, 'provider'), (229, 249, 'provider_category'), (165, 167, 'size'), (96, 101, 'size'), (40, 51, 'color'), (190, 205, 'color'), (61, 72, 'brand'), (134, 139, 'price'), (11, 19, 'product'), (22, 26, 'product'), (221, 226, 'gender')]}),
("Lyhythihainen t paita Vans  FLYING V CREW TEELyhythihainen t paita vans flying v crew tee pink saatavana naisten kokoja s,m,l,xl,xs  Vans26.34{'color': ('pink'), 17 muoti ja vaatetusSpartoo  ('pink')", { 'entities': [(90, 94, 'color'), (129, 132, 'size'), (126, 128, 'size'), (73, 74, 'size'), (122, 123, 'size'), (105, 113, 'gender'), (14, 21, 'product'), (137, 142, 'price'), (22, 26, 'brand'), (67, 71, 'brand'), (181, 188, 'provider'), (161, 181, 'provider_category')]}),
("Bobo Regular C Nk Top Ss T shirts & Tops Short sleeved Sininen Tommy Hilfiger Amerikkalainen brändi, Tommy Hilfiger luo ikonisia vaatteita, jotka ovat klassisia ja preppyjä. Luomu. Tommy Hilfiger 40.9€ {'size': ('l', 's', 'm', 'xs'), 'color': ('desert sky'), 'gender': ('women'), 17 muoti ja vaatetus Boozt", { 'entities': [(302, 307, 'provider'), (281, 301, 'provider_category'), (195, 200, 'price'), (101, 115, 'brand'), (180, 194, 'brand'), (212, 229, 'size'), (128, 137, 'product'), (35, 39, 'product'), (24, 32, 'product'), (18, 22, 'product'), (54, 61, 'color'), (243, 253, 'color'), (270, 275, 'gender')]}),
("Malhotra MRL ML2 455 ( 10  16.5 134A2 10PR TL kaksoistunnus  126A3 )  10  16.5 134A2 10PR TL kaksoistunnus  126A3Malhotra MRL289.8{'size': ('16.5'), 02 ajoneuvotRengas Online('16.5')  ", { 'entities': [(27, 31, 'size'), (160, 173, 'provider'), (148, 160, 'provider_category'), (125, 130, 'price'), (0, 12, 'brand')]}),
("Brow Defining Kit 1 setTäydellinen setti kulmakarvoille Ardellilta  Ardell13.95€ 17 muoti ja vaatetusShopping4net", { 'entities': [(101, 113, 'provider'), (81, 101, 'provider_category'), (74, 79, 'price'), (56, 62, 'brand'), (41, 55, 'product')]}),
("Extreme Length Sealer 50ml    Redken19.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(64, 75, 'provider'), (43, 64, 'provider_category'), (36, 41, 'price'), (30, 36, 'brand'), (22, 26, 'size'), (15, 21, 'product')]}),
("Prada Aurinkolasit PR53VS 1AB1I0Prada PR53VS Aurinkolasit. Collection:Men. Kehyksen Väri: Black.  Linssin Väri: . Kehysmateriaali: Metal. Koko: 59.  Prada197.0{'size': ('59'), 'color': ('black'), 'gender': ('male'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses('59'),('black'),('male'),('adult')  \r", { 'entities': [(268, 284, 'provider'), (187, 192, 'color'), (6, 18, 'product'), (0, 5, 'brand'), (154, 159, 'price'), (144, 146, 'size'), (90, 95, 'color'), (131, 136, 'material'), (98, 105, 'material'), (70, 73, 'gender'), (208, 212, 'gender'), (231, 236, 'age_group'), (239, 268, 'provider_category')]}),
("Lyhythihainen t paita Desigual  UNDERLyhythihainen t paita desigual under multicolour saatavana naisten kokoja l  Desigual29.59{'color': ('multicolour'), 17 muoti ja vaatetusSpartoo  ('multicolour')", { 'entities': [(74, 85, 'color'), (122, 127, 'price'), (114, 122, 'brand'), (211, 112, 'size'), (59, 67, 'brand'), (173, 180, 'provider'), (153, 173, 'provider_category'), (14, 21, 'product'), (96, 104, 'gender')]}),
("Police Silmälasit VK079 LAPIS JR 2 Kids 06QSPolice VK079 LAPIS JR 2 Kids Silmälasit. Collection:Kids. Kehyksen Väri: Matte Blue. Kehysmateriaali: Plastic. Koko: 48.  Police55.2{'size': ('48'), 'color': ('matte blue'), 'gender': ('unisex'), 'AGE_GROUP': ('kids'), 13 silmalasit ja piilolinssitSmartbuy Glasses('48'),('matte blue'),('unisex'),('kids')  \r", { 'entities': [(255, 259, 'age_group'), (230, 236, 'gender'), (204, 214, 'color'), (161, 163, 'size'), (35, 40, 'age_group'), (117, 127, 'color'), (24, 29, 'age_group'), (30, 32, 'age_group'), (146, 153, 'material'), (172, 176, 'price'), (0, 6, 'brand'), (291, 307, 'provider'), (262, 291, 'provider_category'), (7, 17, 'product')]}),
("Coolmax Musta 3436Monitoimisukat, materiaali kosteutta siirtävää keinokuitua.  Tesema3.5{'size': ('3436'), 'color': ('musta'), 'gender': ('men'), 16 urheilu ja eraScandinavian Outdoor('3436'),('musta'),('men')", { 'entities': [(8, 14, 'color'), (162, 182, 'provider'), (145, 162, 'provider_category'), (85, 88, 'price'), (79, 85, 'brand'), (118, 123, 'color'), (14, 18, 'size'), (18, 32, 'product'), (65, 76, 'material'), (139, 142, 'gender')]}),
("Extreme Length Primer 150ml    Redken20.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(65, 76, 'provider'), (44, 65, 'provider_category'), (37, 42, 'price'), (31, 37, 'brand'), (22, 27, 'size'), (15, 21, 'product')]}),
("Malhotra MRL MTF 222 (4.50 16 73A6 4PR TT kaksoistunnus 65A8) 4.50 16 73A6 4PR TT kaksoistunnus 65A8 Malhotra MRL 66.9€ {'size': ('16.0'), 02 ajoneuvot Rengas Online ('16.0')  ", { 'entities': [(134, 138, 'size'), (114, 119, 'price'), (0, 12, 'brand'), (101, 113, 'brand'), (27, 29, 'size'), (154, 167, 'provider'), (141, 153, 'provider_category')]}),
("Frizz Dismiss Conditioner 250ml    Redken15.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(69, 80, 'provider'), (48, 69, 'provider_category'), (41, 46, 'price'), (35, 41, 'brand'), (26, 31, 'size'), (14, 25, 'product')]}),
("Pullover Long Sleeve Kilpikonnakaulus Poolopaita Punainen TaifunTaifun Pullover Long Sleeve  Taifun129.99{'size': ('l', 'xl'), 'color': ('lipstick red'), 'gender': ('women'). 17 muoti ja vaatetus Boozt \r", { 'entities': [(196, 201, 'provider'), (175, 195, 'provider_category'), (166, 171, 'gender'), (138, 150, 'color'), (58, 64, 'brand'), (99, 105, 'price'), (121, 123, 'size'), (49, 57, 'color'), (0, 8, 'product'), (38, 48, 'product')]}),
("Sweater Forest Aop Dots Svetari Collegepaita Beige LindexLindex Sweater Forest Aop Dots  Lindex   14.99€    'size': ('110', '98', '128'), 'color': ('beige'), 'gender': ('kids'). 17 muoti ja vaatetus   Boozt", { 'entities': [(201, 206, 'provider'), (178, 198, 'provider_category'), (51, 57, 'brand'), (98, 103, 'price'), (118, 121, 'size'), (125, 127, 'size'), (131, 134, 'size'), (0, 7, 'product'), (39, 44, 'product'), (45, 51, 'color'), (149, 154, 'color'), (170, 174, 'age_group'), (24, 32, 'product')]}),
("Frizz Dismiss Conditioner 500ml    Redken26.95€ 06 kauneus ja terveys, Hajuvesi.fi", { 'entities': [(71, 82, 'provider'), (48, 69, 'provider_category'), (41, 46, 'price'), (35, 41, 'brand'), (26, 31, 'size'), (14, 25, 'product')]}),
("Wella SP Balance Scalp Shampoo 250 mlHerkän hiuspohjan shampoo Wella SP:ltä.  Wella SP $13.95  17 muoti ja vaatetus Shopping4net", { 'entities': [(0, 8, 'brand'), (88, 93, 'price'), (17, 30, 'product'), (55, 62, 'product'), (31, 37, 'size'), (116, 128, 'provider'), (95, 115, 'provider_category')]}),
("Vogue Eyewear Silmälasit VO5224 2636Vogue Eyewear VO5224 Silmälasit. Collection:Women. Kehyksen Väri: Red. Kehysmateriaali: Plastic. Koko: 51.  Vogue Eyewear73.0{'size': ('51'), 'color': ('red'), 'gender': ('female'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssitSmartbuy Glasses  \r", { 'entities': [(270, 286, 'provider'), (241, 270, 'provider_category'), (157, 161, 'price'), (0, 13, 'brand'), (102, 105, 'color'), (139, 141, 'size'), (80, 85, 'gender'), (233, 238, 'age_group'), (208, 214, 'gender'), (189, 192, 'color'), (124, 131, 'material'), (14, 24, 'product')]}),
("Adi Core 1/4 Z Svetari Collegepaita Musta Adidas Golfadidas Golf Adi Core 1/4 Z  adidas Golf79.95{'size': ('l', 'xl', 's'), 'color': ('black'), 'gender': ('men'), 17 muoti ja vaatetusBoozt('l', 'xl', 's'),('black'),('men')", { 'entities': [(156, 159, 'gender'), (135, 140, 'color'), (113, 115, 'size'), (182, 187, 'provider'), (162, 182, 'provider_category'), (92, 97, 'price'), (53, 64, 'provider'), (42, 53, 'brand'), (36, 41, 'color'), (15, 23, 'product'), (30, 35, 'product')]}),
("Korkokengät Irregular Choice MARMA LADIES Korkokengät irregular choice marma ladies beige saatavana naisten kokoja 38,42 Irregular Choice 86.4€ {'color': ('beige'), 17 muoti ja vaatetus Spartoo  ('beige')", { 'entities': [(84, 89, 'color'), (115, 120, 'size'), (100, 107, 'gender'), (77, 84, 'gender'), (35, 41, 'gender'), (0, 11, 'product'), (138, 143, 'price'), (12, 28, 'brand'), (121, 137, 'brand'), (188, 195, 'provider'), (167, 187, 'provider_category')]}),
("Emilio Pucci Aurinkolasit EP0138 01AEmilio Pucci EP0138 Aurinkolasit. Collection:Women. Kehyksen Väri: Shiny Black. Linssin Väri: . Kehysmateriaali: Plastic. Koko: 52. Emilio Pucci 158.0€ {'size': ('52'), 'color': ('shiny black'), 'gender': ('female'), 'AGE_GROUP': ('adult'), 13 silmalasit ja piilolinssit Smartbuy Glasses ('52'),('shiny black'),('female'),('adult')  \r", { 'entities': [(309, 325, 'provider'), (279, 308, 'provider_category'), (181, 187, 'price'), (0, 12, 'brand'), (168, 180, 'brand'), (149, 156, 'material'), (13, 25, 'product'), (81, 86, 'gender'), (103, 114, 'color'), (116, 123, 'material'), (215, 226, 'color'), (243, 249, 'gender'), (268, 273, 'age_group'), (164, 166, 'size')]}),
("Biolage SmoothProof Serum 89ml    Biolage15.95€ 06 kauneus ja terveysHajuvesi.fi", { 'entities': [(69, 80, 'provider'), (48, 69, 'provider_category'), (41, 46, 'price'), (0, 7, 'brand'), (26, 30, 'size'), (20, 25, 'product')]}),
("Cugy Lyhyt Hame Vaaleanpunainen Vanessa Bruno Vanessa Bruno. A linjainen. Vanessa Bruno 195.0€ {'size': ('38', '42'), 'color': ('blush'), 'gender': ('women'), 17 muoti ja vaatetus Boozt('38', '42'),('blush'),('women')", { 'entities': [(182, 187, 'provider'), (161, 181, 'provider_category'), (16, 31, 'color'), (11, 15, 'product'), (106, 108, 'size'), (112, 114, 'size'), (150, 155, 'gender'), (88, 94, 'price'), (46, 59, 'brand'), (74, 87, 'brand')]}),
("Roar Skirt Polvipituinen Hame Musta Lollys LaundryLollys Laundry. A linjainen.  Lollys Laundry85.0{'color': ('washed black'), 'gender': ('women'), 17 muoti ja vaatetusBoozt  ('washed black'),('women')", { 'entities': [(138, 143, 'gender'), (110, 122, 'color'), (30, 36, 'color'), (25, 30, 'product'), (5, 11, 'product'), (166, 171, 'provider'), (146, 166, 'provider_category'), (94, 98, 'price'), (36, 50, 'brand')]}),
("Zonnebril Alensa Retro Black    Alensa8.99  13 silmalasit ja piilolinssitAlensa", { 'entities': [(44, 73, 'provider_category'), (38, 42, 'price'), (23, 29, 'color'), (10, 16, 'brand')]}),
("Traverse Shorts Cumin LSlim fit  shortsit miehille, materiaalina joustava tekninen kangas.  Rab35.0{'size': ('l'), 'color': ('cumin'), 'gender': ('men'), 16 urheilu ja eraScandinavian Outdoor('l'),('cumin'),('men')", { 'entities': [(147, 150, 'gender'), (24, 25, 'size'), (42, 50, 'gender'), (83, 89, 'material'), (33, 42, 'product'), (126, 131, 'color'), (95, 99, 'price'), (92, 95, 'brand'), (170, 190, 'provider'), (153, 170, 'provider_category'), (16, 22, 'color')]}),
("Flairville Low Laceshoes Shoes Business Laced Shoes Ruskea GANTGANT Flairville Low Laceshoes  Gant169.9{'size': ('40', '43', '42', '44', '41', '45', '46'), 'color': ('tobacco brown'), 'gender': ('men'), 17 muoti ja vaatetusBoozt('40', '43', '42', '44', '41', '45', '46'),('tobacco brown'),('men')", { 'entities': [(222, 227, 'provider'), (202, 222, 'provider_category'), (144, 146, 'size'), (114, 116, 'size'), (150, 152, 'size'), (138, 140, 'size'), (132, 134, 'size'), (126, 128, 'size'), (120, 122, 'size'), (98, 103, 'price'), (59, 63, 'brand'), (94, 98, 'brand'), (25, 30, 'product'), (40, 51, 'product'), (52, 59, 'color'), (196, 199, 'gender'), (167, 180, 'color')]}),
("https://www.boozt.com/fi/fi/bjorn borg  bjorn borg footwear boozt /mio high m_18078193/18078196?navId=59819&group=brandwall&position=1000000 \r", { 'entities': [(60, 65, 'provider'), (40, 50, 'brand'), (51, 59, 'product')]}),
("https://www.boozt.com/fi/fi/adidas originals/vint mini bag_26303593/211791666?navId=68158&group=listing&position=1000000\r", { 'entities': [(12, 17, 'provider'), (28, 34, 'brand'), (55, 58, 'product')]}),
("<script type=\'application/ld+json\'> ({\'@context\': \'http://schema.org\', \'@type\': \'Product\', \'name\': \'VINT MINI BAG\', \'brand\': \'adidas Originals\', \'sku\': \'ADIGD4782\', \'gtin8\': \'4061612417101\', \'url\': \'https://www.boozt.com/fi/fi/adidas originals/vint mini bag_26303593/211791666\', \'description\': \'100% polyuretaani Vetoketjukiinnitys yläosassa Säädettävä olkain\', \'image\': \'https://ean images.booztcdn.com/adidas originals232x303/adigd4782_cblack.jpg\', \'color\': \'BLACK\', \'offers\': ({\'@type\': \'Offer\', \'gtin8\': \'4061612417101\', \'url\': \'https://www.boozt.com/fi/fi/adidas originals/vint mini bag_26303593/211791666\', \'price\': \'29.95\', \'priceCurrency\': \'EUR\', \'itemCondition\': \'https://schema.org/NewCondition\', \'availability\': \'http://schema.org/InStock\', AdditionalProperty\': {\'@type\': \'PropertyValue\', \'name\': \'Size\', \'value\': \'ONE SIZE\'}}), \'aggregateRating\': (), \'review\': ()]}),</script>\r", { 'entities': [(826, 834, 'size'), (623, 628, 'price'), (461, 466, 'color'), (211, 216, 'provider'), (126, 142, 'brand'), (227, 243, 'brand'), (100, 113, 'product'), (244, 257, 'product'), (439, 444, 'color'), (300, 312, 'material')]}),
("<script nonce=\'b85235826bf328ee358e02bdd3c3094d\'> var eventParams = {'event': \'productView\', 'productBrand': \'adidas Originals\', 'productName': \'VINT MINI BAG\', 'productActionList': \'listing\', 'productDescription': \'ILMAINEN TOIMITUS \\x2d adidas Originals Vint Mini Bag \\x28Black\\x29 Boozt\\x2ecom\\x3aissa\\x2e Uusi adidas Originals kokoelma 2020\\x21 Tarjoamme nopean toimituksen ja helpon palautuksen\\x2e\', 'productPrice': '29.95', 'productUrl' : 'https://www.boozt.com/fi/fi/adidas originals/vint mini bag_26303593/211791666', 'productImageUrl' : 'https://ean images.booztcdn.com/adidas originals/183x239/adigd4782_cblack.jpg', 'currentCategoryName' : 'Olkalaukut', 'currencyCode' : 'EUR', 'variantId': '597735340', 'hasFlixstockModel': 'false','hasBrandModel': 'false' }; </script>\r", { 'entities': [(110, 126, 'brand'), (145, 158, 'product'), (256, 269, 'product'), (423, 428, 'price'), (653, 663, 'product'), (616, 621, 'color'), (492, 505, 'product'), (475, 491, 'brand'), (459, 464, 'provider')]}),
("Lacoste Silmälasit L2840 220", { 'entities': [(0, 7, 'brand'), (8, 18, 'product')]}),
("Beyond Perf. Foundation + Concealer, 09 Neutral", { 'entities': [(13, 35, 'product'), (40, 47, 'color')]}),
("Silk Screen Refining Powder Makeup, 24 Meikkivoide Meikki Origins", { 'entities': [(21, 34, 'product'), (39, 50, 'product'), (58, 65, 'brand')]}),
("Calvin Klein Earth Miesten kello K5Y31XWL Vihreä/Nahka 44 mm", { 'entities': [(0, 12, 'brand'), (19, 26, 'gender'), (27, 32, 'product'), (42, 48, 'color'), (49, 54, 'material'), (55, 61, 'size')]}),
("Just Cavalli Silmälasit JC 0817 086", { 'entities': [(0, 12, 'brand'), (13, 23, 'product')]}),
("Alliance Forestry 333 ( 320/85  24 127A8 14PR TL kaksoistunnus 124B )", { 'entities': [(0, 8, 'brand'), (24, 34, 'size')]}),
("ITP Bajacross ( 26x10.00 R14 TL 81D )", { 'entities': [(0, 3, 'brand'), (16, 24, 'size')]}),
("Polaroid Aurinkolasit PLD 7017/S Polarized 807/5Z", { 'entities': [(0, 8, 'brand'), (9, 21, 'product')]}),
("Dobsom Lanna Jacket", { 'entities': [(0, 6, 'brand'), (13, 19, 'product')]}),
("Vogue Eyewear Silmälasit VO2714 IN VOGUE 2693 Material: Plastic", { 'entities': [(56,63,'material'), (0, 13, 'brand'), (14, 24, 'product')]}),
("Arnette Aurinkolasit AN4265 POST MALONE X ARNETTE 259080", { 'entities': [(0, 7, 'brand'), (8, 20, 'product')]}),
("Breguet Classique Miesten kello 5717BR EU 9ZU Hopea/Nahka 44 mm", { 'entities': [(0, 7, 'brand'), (18, 25, 'gender'), (26, 31, 'product'), (46, 51, 'color'), (52, 57, 'material'), (58, 62, 'size')]}),
("Yokohama W.drive (V905),( 285/35 R21 105V XL, RPB )", { 'entities': [(0, 8, 'brand'), (26, 36, 'size'), (42, 44, 'size')]}),
("Beyond Perf. Foundation + Concealer, 11 Honey", { 'entities': [(13, 35, 'product'), (40, 45, 'color')]}),
("Q10 120 mg 60 kapselia", { 'entities': [(0, 3, 'product'), (4, 10, 'size')]}),
("Lastenkengät adidas EQT SUPPORT ADV J", { 'entities': [(0, 12, 'product'), (13, 19, 'brand')]}),
("Calvin Klein Silmälasit CK7932 240 Material: Plastic", { 'entities': [(45,52,'material'), (0, 12, 'brand'), (13, 23, 'product')]}),
("Beyond Perf. Foundation + Concealer, 18 Sand", { 'entities': [(13, 35, 'product'), (40, 44, 'color')]}),
("Lastenkengät adidas U_PATH RUN J", { 'entities': [(0, 12, 'product'), (13, 19, 'brand')]}),
("Pitkähihainen paitapusero Eleven Paris VRAPP MEN", { 'entities': [(14, 25, 'product'), (26, 38, 'brand'), (46, 49, 'gender')]}),
("Tory Burch Silmälasit TY2063 1726", { 'entities': [(0, 10, 'brand'), (11, 21, 'product')]}),
("Versace Aurinkolasit VE4359 GB1/1A", { 'entities': [(0, 7, 'brand'), (8, 20, 'product')]}),
("B&P Valle Steel HV 32g 12/70 Haulikon Patruuna", { 'entities': [(0, 3, 'brand'), (4, 15, 'product'), (19, 22, 'size')]}),
("Beyond Perf. Foundation + Concealer, 23 Ginger", { 'entities': [(13, 35, 'product'), (40, 46, 'color')]}),
("Etnia Barcelona Silmälasit Greenwich BKBZ", { 'entities': [(0, 15, 'brand'), (16, 26, 'product')]}),
("Didak Ananas Uimapatja", { 'entities': [(0, 5, 'brand'), (13, 22, 'product')]}),
("Farkut Leon", { 'entities': [(0, 6, 'product')]}),
("MOMO Shampoo, 1000ml", { 'entities': [(5, 12, 'product'), (14, 20, 'size')]}),
("Westward Leaning Aurinkolasit Voyager 30", { 'entities': [(0, 16, 'brand'), (17, 29, 'product')]}),
("MODO Silmälasit 4528 Nude Crystal", { 'entities': [(0, 4, 'brand'), (5, 15, 'product'), (21, 33, 'color')]}),
("Roberto Cavalli Silmälasit RC 756 REETHI 001", { 'entities': [(0, 15, 'brand'), (16, 26, 'product')]}),
("Flora Shoes Summer Shoes Sandals Hopea ECCO", { 'entities': [(6, 11, 'product'), (12, 24, 'product'), (25, 32, 'product'), (33, 38, 'color'), (39, 43, 'brand')]}),
("Beyond Perf. Foundation + Concealer, 48 Oat", { 'entities': [(13, 35, 'product'), (40, 43, 'color')]}),
("adidas Säärisuojat Predator League Inflight   Valkoinen/Hopea/Musta", { 'entities': [(0, 6, 'brand'), (7, 18, 'product'), (46, 55, 'color'), (56, 61, 'color'), (62, 67, 'color')]}),
("Michelin Alpin 6 ( 195/60 R16 89T )", { 'entities': [(0, 8, 'brand'), (19, 29, 'size')]}),
("Chinot River", { 'entities': [(7, 12, 'brand')]}),
("Zac Posen Aurinkolasit ALEK Tortoise", { 'entities': [(0, 9, 'brand'), (10, 22, 'product'), (28, 36, 'color')]}),
("Classic Tie Solmio Kravatti Monivärinen/Kuvioitu Amanda Christensen", { 'entities': [(8, 11, 'product'), (12, 18, 'product'), (19, 27, 'product'), (28, 39, 'color'), (40, 48, 'color'), (49, 67, 'brand')]}),
("Calvin Klein Aurinkolasit CK4354S 239", { 'entities': [(0, 12, 'brand'), (13, 25, 'product')]}),
("Westward Leaning Aurinkolasit Concorde 22", { 'entities': [(0, 16, 'brand'), (17, 29, 'product')]}),
("Retrosuperfuture Aurinkolasit Tuttolente Drew Mama Black IUKA HFW", { 'entities': [(0, 16, 'brand'), (17, 29, 'product'), (51, 56, 'color')]}),
("Cultor RD 01 ( 460/85 R34 147A8 TL kaksoistunnus 144B )", { 'entities': [(0, 6, 'brand'), (15, 25, 'size')]}),
("Rival Fleece Jogger Collegehousut Olohousut Sininen Under Armour", { 'entities': [(6, 12, 'material'), (20, 33, 'product'), (34, 43, 'product'), (44, 51, 'color'), (52, 64, 'brand')]}),
("Viiru & Pesonen lasten huopa Tervehdys", { 'entities': [(16, 22, 'gender'), (23, 28, 'product')]}),
("Ceat Farmax RC ( 380/90 R46 165A8 TL kaksoistunnus 165B )", { 'entities': [(0, 4, 'brand'), (17, 27, 'size')]}),
("Miki Swim Pants Swimwear Nappie Briefs Sininen Soft Gallery", { 'entities': [(5, 15, 'product'), (16, 24, 'product'), (39, 46, 'color'), (47, 59, 'brand')]}),
("Kengät Hunter MENS ORIGINAL TALL", { 'entities': [(0, 6, 'product'), (7, 13, 'brand'), (15, 17, 'gender')]}),
("Croon Silmälasit Leesbril Classic Pink Black Duo Material: Plastic", { 'entities': [(59, 66, 'material'), (0, 5, 'brand'), (6, 16, 'product'), (34, 38, 'color'), (39, 44, 'color')]}),
("Gucci GG0025O 002", { 'entities': [(0, 5, 'brand')]}),
("BKT AW 705 ( 400/55  22.5 154A8 18PR TL kaksoistunnus 150B )", { 'entities': [(0, 3, 'brand'), (13, 25, 'size')]}),
("Tom Ford Georgia FT0498 69T", { 'entities': [(0, 8, 'brand')]}),
("Tom Ford Georgia FT0498 69T", { 'entities': [(0, 8, 'brand')]}),
("Fendi Silmälasit FF M0005 SCL", { 'entities': [(0, 5, 'brand'), (6, 16, 'product')]}),
("6219   Star Napoli Normal Bleiseri Puvun Takki Harmaa SAND", { 'entities': [(26, 34, 'product'), (35, 46, 'product'), (47, 53, 'color'), (54, 58, 'brand')]}),
("Blouses Woven Pitkähihainen Pusero Paita Valkoinen Esprit Collection", { 'entities': [(0, 7, 'product'), (28, 40, 'product'), (41, 50, 'color'), (51, 57, 'brand')]}),
("Dress Long Sleeve Polvipituinen Mekko Oranssi Noa Noa", { 'entities': [(0, 5, 'product'), (32, 37, 'product'), (38, 45, 'color'), (46, 53, 'brand')]}),
("got2b Twisted Anti Frizz Styling Spray 200 ml", { 'entities': [(25, 38, 'product'), (39, 45, 'size')]}),
("Croon Silmälasit Leesbril Madonna Havana Red Transparent", { 'entities': [(0, 5, 'brand'), (6, 16, 'product'), (41, 56, 'color')]}),
("Alliance Farm Pro 70 ( 520/70 R30 145A8 TL kaksoistunnus 145B )", { 'entities': [(0, 8, 'brand'), (23, 33, 'size')]}),
("Poikien sandaalit Andrä© MILAN", { 'entities': [(8, 17, 'product'), (18, 23, 'brand')]}),
("Paradise Swimsuit Uimapuku Uima asut Monivärinen/Kuvioitu Notes Du Nord", { 'entities': [(9, 17, 'product'), (18, 26, 'product'), (27, 36, 'product'), (37, 48, 'color'), (49, 57, 'color'), (58, 71, 'brand')]}),
("AJ päytälamppu Musta", { 'entities': [(3, 14, 'product'), (15, 20, 'color')]}),
("INTEX Swan Ride On", { 'entities': [(0, 5, 'brand')]}),
("Beyond Perf. Foundation + Concealer, 6,5 Buttermilk", { 'entities': [(13, 35, 'product'), (41, 51, 'color')]}),
("Guess GU7459 01B", { 'entities': [(0, 5, 'brand')]}),
("Cannot Stop would not Stop Contour Concealer Peitevoide Meikki NYX PROFESSIONAL MAKEUP", { 'entities': [(22, 39, 'product'), (40, 57, 'product'), (58, 81, 'brand')]}),
("Paita objSifka Top 107", { 'entities': [(0, 5, 'product')]}),
("Korkokengät Rupert Sanderson CLIPPER", { 'entities': [(0, 11, 'product'), (12, 28, 'brand')]}),
("Ray Ban Silmälasit RB2132F New Wayfarer Asian Fit Polarized 901/58", { 'entities': [(0, 7, 'brand'), (8, 18, 'product')]}),
("Bvlgari Aurinkolasit BV6104 20144Z Material: Plastic", { 'entities': [(45, 52, 'material'), (0, 7, 'brand'), (8, 20, 'product')]}),
("Isabel Alushousut Brief Tangat Ruskea Love Stories", { 'entities': [(7, 17, 'product'), (18, 30, 'product'), (31, 37, 'color'), (38, 50, 'brand')]}),
("Cr7 Boys Trunk 2 Pack Alushousut Musta CR7", { 'entities': [(0, 3, 'brand'), (22, 32, 'product'), (33, 38, 'color'), (39, 42, 'brand')]}),
("Seha Agro 10 ( 420/70 R24 130A8 TL )", { 'entities': [(0, 4, 'brand'), (15, 25, 'size')]}),
("Carlisle Turf Saver ( 18x8.50  8 74A4 4PR TL NHS )", { 'entities': [(0, 8, 'brand'), (22, 32, 'size')]}),
("Kengät Ippon Vintage BEACH PRINCE", { 'entities': [(0, 6, 'product'), (7, 20, 'brand')]}),
("High Lengths Mascara, Black", { 'entities': [(13, 20, 'product'), (22, 27, 'color')]}),
("Self Sun Face Tinted Lotion 50ml", { 'entities': [(21, 27, 'product'), (28, 32, 'size')]}),
("Cargo Loose Suoralahkeiset Housut Vihreä Please Jeans", { 'entities': [(27, 33, 'product'), (34, 40, 'color'), (41, 53, 'brand')]}),
("Rally Sneakers Matalavartiset Sneakerit Tennarit Musta FitFlop", { 'entities': [(6, 14, 'product'), (40, 48, 'product'), (49, 54, 'color'), (55, 62, 'brand')]}),
("Ceat Lift Pro ( 460/70 R24 159A8 TL kaksoistunnus 159B, T.R.A. I3 )", { 'entities': [(0, 4, 'brand'), (16, 26, 'size')]}),
("Farkut Rex", { 'entities': [(0, 6, 'product')]}),
("IsaDora Double Ended Eye Shadow Applicator", { 'entities': [(0, 7, 'brand'), (21, 42, 'product')]}),
("Day Dream On Messinki", { 'entities': [(13, 21, 'material')]}),
("Ceat Farmax R1 ( 15.5  38 137A6 10PR TT )", { 'entities': [(0, 4, 'brand'), (17, 25, 'size')]}),
("Radley Silmälasit RDO NATALLIA 003 Material: Plastic", { 'entities': [(45, 52, 'material'), (0, 6, 'brand'), (7, 17, 'product')]}),
("Vredestein V 61 ( 250/65  14.5 121A8 12PR TL )", { 'entities': [(0, 10, 'brand'), (18, 30, 'size')]}),
("Superdefense Moisturizer SPF20 50ml (Very Dry/Comb)", { 'entities': [(13, 24, 'product'), (31, 35, 'size')]}),
("Quickliner For Eyes, Blue Grey", { 'entities': [(0, 19, 'product'), (21, 30, 'color')]}),
("Vogue Eyewear Silmälasit VO4094 5138 Material: Plastic", { 'entities': [(47, 54, 'material'), (0, 13, 'brand'), (14, 24, 'product')]}),
("Tennarit UA Classic Slip On", { 'entities': [(0, 8, 'product')]}),
("Dolce & Gabbana Aurinkolasit DG2213 02/73", { 'entities': [(0, 15, 'brand'), (16, 28, 'product')]}),
("Viceiw Blazer Bleiseri Puvun Takki Ruskea InWear", { 'entities': [(7, 13, 'product'), (14, 22, 'product'), (23, 34, 'product'), (35, 41, 'color'), (42, 48, 'brand')]}),
("Washed Linen Pussilakana Tuplapeittoon, Navy Blue", { 'entities': [(13, 38, 'product'), (40, 49, 'color')]}),
("Starmaxx TR 110 ( 200/70 R16 94A8 TL )", { 'entities': [(0, 8, 'brand'), (18, 28, 'size')]}),
("Kate Spade Aurinkolasit Kahli/S WR7/HA", { 'entities': [(0, 10, 'brand'), (11, 23, 'product')]}),
("lacoste silmälasit l2840 220", {'entities': [(0, 7, 'brand'), (8, 18, 'product')]}), 
("beyond perf. foundation + concealer, 09 neutral", {'entities': [(13, 35, 'product'), (40, 47, 'color')]}), 
("silk screen refining powder makeup, 24 meikkivoide meikki origins", {'entities': [(21, 34, 'product'), (39, 50, 'product'), (58, 65, 'brand')]}), 
("calvin klein earth miesten kello k5y31xwl vihreä/nahka ø44 mm", {'entities': [(0, 12, 'brand'), (19, 26, 'gender'), (27, 32, 'product'), (42, 48, 'color'), (49, 54, 'material'), (56, 61, 'size')]}), 
("just cavalli silmälasit jc 0817 086", {'entities': [(0, 12, 'brand'), (13, 23, 'product')]}), 
("alliance forestry 333 ( 320/85  24 127a8 14pr tl kaksoistunnus\xa0 124b )", {'entities': [(0, 8, 'brand'), (24, 48, 'size')]}), 
("itp bajacross ( 26x10.00 r14 tl 81d )", {'entities': [(0, 3, 'brand'), (16, 35, 'size')]}), 
("polaroid aurinkolasit pld 7017/s polarized 807/5z", {'entities': [(0, 8, 'brand'), (9, 21, 'product')]}), 
("dobsom lanna jacket", {'entities': [(0, 6, 'brand'), (13, 19, 'product')]}), 
("vogue eyewear silmälasit vo2714 in vogue 2693", {'entities': [(0, 13, 'brand'), (14, 24, 'product')]}), 
("arnette aurinkolasit an4265 post malone x arnette 259080", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("breguet classique miesten kello 5717br eu 9zu hopea/nahka ø44 mm", {'entities': [(0, 7, 'brand'), (18, 25, 'gender'), (26, 31, 'product'), (46, 51, 'color'), (52, 57, 'material'), (59, 64, 'size')]}), 
("yokohama w.drive (v905),( 285/35 r21 105v xl, rpb )", {'entities': [(0, 8, 'brand'), (26, 49, 'size')]}), 
("beyond perf. foundation + concealer, 11 honey", {'entities': [(13, 35, 'product'), (40, 45, 'color')]}), 
("q10 120 mg 60 kapselia", {'entities': [(0, 3, 'product'), (4, 10, 'size')]}), 
("lastenkengät adidas\xa0 eqt support adv j", {'entities': [(0, 12, 'product'), (13, 19, 'brand')]}), 
("calvin klein silmälasit ck7932 240", {'entities': [(0, 12, 'brand'), (13, 23, 'product')]}), 
("beyond perf. foundation + concealer, 18 sand", {'entities': [(13, 35, 'product'), (40, 44, 'color')]}), 
("lastenkengät adidas\xa0 u_path run j", {'entities': [(0, 12, 'product'), (13, 19, 'brand')]}), 
("pitkähihainen paitapusero eleven paris\xa0 vrapp men", {'entities': [(14, 25, 'product'), (26, 38, 'brand'), (46, 49, 'gender')]}), 
("tory burch silmälasit ty2063 1726", {'entities': [(0, 10, 'brand'), (11, 21, 'product')]}), 
("versace aurinkolasit ve4359 gb1/1a", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("b&p valle steel hv 32g 12/70 haulikon patruuna", {'entities': [(0, 3, 'brand'), (4, 15, 'product'), (19, 22, 'size')]}), 
("beyond perf. foundation + concealer, 23 ginger", {'entities': [(13, 35, 'product'), (40, 46, 'color')]}), 
("etnia barcelona silmälasit greenwich bkbz", {'entities': [(0, 15, 'brand'), (16, 26, 'product')]}), 
("didak ananas uimapatja", {'entities': [(0, 5, 'brand'), (13, 22, 'product')]}), 
("farkut leon", {'entities': [(0, 6, 'product')]}), 
("momo shampoo, 1000ml", {'entities': [(5, 12, 'product'), (14, 20, 'size')]}), 
("westward leaning aurinkolasit voyager 30", {'entities': [(0, 16, 'brand'), (17, 29, 'product')]}), 
("modo silmälasit 4528 nude crystal", {'entities': [(0, 4, 'brand'), (5, 15, 'product'), (21, 33, 'color')]}), 
("roberto cavalli silmälasit rc 756 reethi 001", {'entities': [(0, 15, 'brand'), (16, 26, 'product')]}), 
("flora shoes summer shoes sandals hopea ecco", {'entities': [(6, 11, 'product'), (12, 24, 'product'), (25, 32, 'product'), (33, 38, 'color'), (39, 43, 'brand')]}), 
("beyond perf. foundation + concealer, 48 oat", {'entities': [(13, 35, 'product'), (40, 43, 'color')]}), 
("adidas säärisuojat predator league inflight   valkoinen/hopea/musta", {'entities': [(0, 6, 'brand'), (7, 18, 'product'), (46, 55, 'color'), (56, 61, 'color'), (62, 67, 'color')]}), 
("michelin alpin 6 ( 195/60 r16 89t )", {'entities': [(0, 8, 'brand'), (19, 33, 'size')]}), 
("chinot river", {'entities': [(7, 12, 'brand')]}), 
("zac posen aurinkolasit alek tortoise", {'entities': [(0, 9, 'brand'), (10, 22, 'product'), (28, 36, 'color')]}), 
("classic tie solmio kravatti monivärinen/kuvioitu amanda christensen", {'entities': [(8, 11, 'product'), (12, 18, 'product'), (19, 27, 'product'), (28, 39, 'color'), (40, 48, 'color'), (49, 67, 'brand')]}), 
("calvin klein aurinkolasit ck4354s 239", {'entities': [(0, 12, 'brand'), (13, 25, 'product')]}), 
("westward leaning aurinkolasit concorde 22", {'entities': [(0, 16, 'brand'), (17, 29, 'product')]}), 
("retrosuperfuture aurinkolasit tuttolente drew mama black iuka hfw", {'entities': [(0, 16, 'brand'), (17, 29, 'product'), (51, 56, 'color')]}), 
("cultor rd 01 ( 460/85 r34 147a8 tl kaksoistunnus\xa0 144b )", {'entities': [(0, 6, 'brand'), (15, 34, 'size')]}), 
("rival fleece jogger collegehousut olohousut sininen under armour", {'entities': [(6, 12, 'material'), (20, 33, 'product'), (34, 43, 'product'), (44, 51, 'color'), (52, 64, 'brand')]}), 
("viiru & pesonen lasten huopa tervehdys", {'entities': [(23, 28, 'product')]}), 
("ceat farmax rc ( 380/90 r46 165a8 tl kaksoistunnus\xa0 165b )", {'entities': [(0, 4, 'brand'), (17, 36, 'size')]}), 
("miki swim pants swimwear nappie briefs sininen soft gallery", {'entities': [(5, 15, 'product'), (16, 24, 'product'), (39, 46, 'color'), (47, 59, 'brand')]}), 
("kengät hunter\xa0 mens original tall", {'entities': [(0, 6, 'product'), (7, 13, 'brand'), (15, 19, 'gender')]}), 
("croon silmälasit leesbril classic pink black duo", {'entities': [(0, 5, 'brand'), (6, 16, 'product'), (34, 38, 'color'), (39, 44, 'color')]}), 
("gucci gg0025o 002", {'entities': [(0, 5, 'brand')]}), 
("bkt aw 705 ( 400/55  22.5 154a8 18pr tl kaksoistunnus\xa0 150b )", {'entities': [(0, 3, 'brand'), (13, 39, 'size')]}), 
("tom ford georgia ft0498 69t", {'entities': [(0, 8, 'brand')]}), 
("tom ford georgia ft0498 69t", {'entities': [(0, 8, 'brand')]}), 
("viking protech newgen ( 235/60 r18 107w xl )", {'entities': [(0, 6, 'brand'), (7, 21, 'product'), (24, 42, 'size')]}), 
("seventh street silmälasit 7a032 2f7", {'entities': [(0, 14, 'brand'), (15, 25, 'product')]}), 
("matkakassi roxy feel happy", {'entities': [(0, 10, 'product'), (11, 15, 'brand')]}), 
("vyö albert", {'entities': [(0, 3, 'product')]}), 
("healing volume thickening conditioner, 1000ml", {'entities': [(26, 37, 'product'), (39, 45, 'size')]}), 
("maxxis vansmart snow wl2 ( 215/65 r15c 104/102t )", {'entities': [(0, 6, 'brand'), (27, 47, 'size')]}), 
("healing volume thickening conditioner, 250ml", {'entities': [(26, 37, 'product'), (39, 44, 'size')]}), 
("love moschino mol524 807", {'entities': [(5, 13, 'brand')]}), 
("sunday somewhere aurinkolasit jesse brg", {'entities': [(0, 16, 'brand'), (17, 29, 'product')]}), 
("healing style beach spray 100ml", {'entities': [(14, 25, 'product'), (26, 31, 'size')]}), 
("healing strength neem plant silk serum 100ml", {'entities': [(28, 38, 'product'), (39, 44, 'size')]}), 
("matador mp47 hectorra 3 ( 175/70 r13 82t )", {'entities': [(0, 7, 'brand'), (26, 40, 'size'), (13, 21, 'product')]}), 
("padded select reppu laukku musta eastpak", {'entities': [(14, 19, 'product'), (20, 26, 'product'), (27, 32, 'color'), (33, 40, 'brand')]}), 
("sc pamela hihaton pusero paita vaaleanpunainen soyaconcept", {'entities': [(10, 17, 'product'), (18, 24, 'product'), (25, 30, 'product'), (31, 46, 'color'), (47, 58, 'brand')]}), 
("vyö ashton", {'entities': [(0, 3, 'product')]}), 
("pelican hat accessories headwear hats musta sways", {'entities': [(8, 11, 'product'), (33, 37, 'product'), (38, 43, 'color'), (44, 49, 'brand'), (24, 32, 'product')]}), 
("vyö arthur", {'entities': [(0, 3, 'product')]}), 
("absolu rouge 106 look huulipuna meikki beige lancôme", {'entities': [(22, 31, 'product'), (39, 44, 'color'), (45, 52, 'brand')]}), 
("d1. the gant spring varsity bag bags weekend & gym bags sininen gant", {'entities': [(28, 31, 'product'), (32, 36, 'product'), (56, 63, 'color'), (64, 68, 'brand')]}), 
("michelin pilot sport 4 ( 205/40 zr17 (84y) xl )", {'entities': [(0, 8, 'brand'), (9, 20, 'product'), (25, 45, 'size')]}), 
("lyhyt mekko ikks bq30155 75", {'entities': [(0, 11, 'product'), (12, 16, 'brand')]}), 
("gabrielle accessories jewellery earrings hoops kulta pilgrim", {'entities': [(22, 31, 'product'), (32, 40, 'product'), (47, 52, 'color'), (53, 60, 'brand')]}), 
("hammered juomalasi 50 cl amber", {'entities': [(9, 18, 'product'), (19, 24, 'size')]}), 
("healing moisture tamanu cream shampoo 50ml", {'entities': [(8, 16, 'product'), (24, 29, 'product'), (30, 37, 'product'), (38, 42, 'size')]}), 
("montana collection by sbg silmälasit mm594 e", {'entities': [(0, 25, 'brand'), (26, 36, 'product')]}), 
("drawstring dress polvipituinen mekko sininen lee jeans", {'entities': [(11, 16, 'product'), (17, 36, 'product'), (37, 44, 'color'), (45, 54, 'brand')]}), 
("healing moisture tamanu cream shampoo 750ml", {'entities': [(24, 29, 'product'), (30, 37, 'product'), (38, 43, 'size')]}), 
("synchro skin self refreshing foundation 30 gr no. 110", {'entities': [(29, 39, 'product'), (40, 45, 'size')]}), 
("slfmyla florenta ls aop midi dress b polvipituinen mekko oranssi selected femme", {'entities': [(29, 34, 'product'), (37, 56, 'product'), (57, 64, 'color'), (65, 79, 'brand')]}), 
("rodebjer kellman cotton pitkähihainen pusero paita valkoinen rodebjer", {'entities': [(0, 8, 'brand'), (17, 23, 'material'), (24, 37, 'product'), (38, 44, 'product'), (45, 50, 'product'), (51, 60, 'color'), (61, 69, 'brand')]}), 
("wiley x aurinkolasit kingpin polarized ackng09", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("luannegz skirt ma19 polvipituinen hame sininen gestuz", {'entities': [(9, 14, 'product'), (34, 38, 'product'), (39, 46, 'color'), (47, 53, 'brand')]}), 
("montana collection by sbg silmälasit mm594 f", {'entities': [(0, 25, 'brand'), (26, 36, 'product'), (43, 44, 'gender')]}), 
("michelin pilot super sport ( 295/35 zr20 (101y) k1 )", {'entities': [(0, 8, 'brand'), (9, 26, 'product'), (29, 50, 'size')]}), 
("bx 2.0 crew svetari collegepaita sininen adidas originals", {'entities': [(12, 19, 'product'), (20, 32, 'product'), (33, 40, 'color'), (41, 57, 'brand')]}), 
("taylor coat outerwear faux fur ruskea stand studio", {'entities': [(12, 21, 'product'), (7, 11, 'product'), (31, 37, 'color'), (38, 50, 'brand')]}), 
("sephina wrap dress polvipituinen mekko monivärinen/kuvioitu just female", {'entities': [(13, 18, 'product'), (19, 38, 'product'), (39, 59, 'color'), (60, 71, 'brand')]}), 
("oakley aurinkolasit oo9208 radar ev path 920896", {'entities': [(0, 6, 'brand'), (7, 19, 'product')]}), 
("healing moisture tamanu cream shampoo, 1000ml", {'entities': [(24, 29, 'product'), (30, 37, 'product'), (39, 45, 'size')]}), 
("pollie top blouses short sleeved keltainen notes du nord", {'entities': [(11, 18, 'product'), (18, 32, 'product'), (33, 42, 'color'), (43, 56, 'brand')]}), 
("rime riippuvalaisin ø 37 cm valkoinen", {'entities': [(5, 19, 'product'), (22, 27, 'size'), (28, 37, 'color')]}), 
("healing moisture tamanu cream shampoo, 300ml", {'entities': [(24, 29, 'product'), (30, 37, 'product'), (39, 44, 'size')]}), 
("healing moisture kukui nut conditioner 50ml", {'entities': [(27, 38, 'product'), (39, 43, 'size'), (23, 26, 'material')]}), 
("vredestein comtrac 2 all season (205/65 r16c 107/105t )", {'entities': [(0, 10, 'brand'), (39, 44, 'size')]}), 
("polaroid aurinkolasit pld 2068/s/x polarized 807/uc", {'entities': [(0, 8, 'brand'), (9, 21, 'product')]}), 
("ahrex tp650 26 degree bent", {'entities': [(0, 5, 'brand'), (22, 26, 'product')]}), 
("matador mp62 all weather evo ( 185/60 r15 88t xl )", {'entities': [(0, 7, 'brand'), (8, 28, 'product'), (31, 48, 'size')]}), 
("matador mps 400 variant all weather 2 ( 205/75 r16c 110/108r 8pr )", {'entities': [(0, 7, 'brand'), (40, 64, 'size')]}), 
("emilio pucci aurinkolasit ep0008 56b", {'entities': [(0, 12, 'brand'), (13, 25, 'product')]}), 
("healing moisture kukui nut conditioner, 1000ml", {'entities': [(23, 26, 'material'), (27, 38, 'product'), (40, 46, 'size')]}), 
("square lth 35mm accessories belts classic belts ruskea calvin klein", {'entities': [(48, 54, 'color'), (28, 33, 'product'), (11, 15, 'size'), (55, 67, 'brand')]}), 
("black opium, edp 30ml", {'entities': [(0, 5, 'color'), (17, 21, 'size')]}), 
("cherry dog sock underwear socks regular socks sininen happy socks", {'entities': [(26, 31, 'product'), (46, 53, 'color'), (54, 65, 'brand')]}), 
("eva lg kumisaappaat kengät punainen tretorn", {'entities': [(36, 43, 'brand'), (20, 26, 'product'), (27, 35, 'color'), (4, 6, 'size')]}), 
("pixi eye reflections shadow palette natural beauty", {'entities': [(0, 4, 'brand')]}), 
("shoes tennarit sneakerit kengät ruskea gulliver", {'entities': [(0, 5, 'product'), (39, 47, 'brand'), (32, 38, 'color'), (25, 31, 'product'), (15, 24, 'product'), (6, 14, 'product')]}), 
("daisy kakkuvati, ø 22 cm faded army", {'entities': [(19, 24, 'size'), (6, 15, 'product')]}), 
("wide maljakko 40 cm, smoked pearl", {'entities': [(14, 19, 'size'), (5, 13, 'product')]}), 
("icons pin point  pussilakana 220x220 cm blue", {'entities': [(29, 39, 'size'), (40, 44, 'color')]}), 
("wan c/w katto/seinävalaisin, valkoinen", {'entities': [(29, 38, 'color'), (8, 13, 'product')]}), 
("pampa mariposa bf chair, crude oak/black", {'entities': [(35, 40, 'color'), (25, 34, 'material')]}), 
("veluxe pearl antiqued beauty women makeup eyes eyeshadow   not palettes monivärinen/kuvioitu m.a.c.", {'entities': [(93, 99, 'brand'), (29, 34, 'gender'), (47, 56, 'product'), (72, 83, 'color')]}), 
("prodesign silmälasit 5640 5534", {'entities': [(10, 20, 'product'), (0, 9, 'brand')]}), 
("running moomin muumitarjotin pyöreä 38 cm", {'entities': [(36, 41, 'size')]}), 
("g star raw aurinkolasit g star raw gs603s 001", {'entities': [(11, 23, 'product'), (0, 10, 'brand')]}), 
("black opium, edp 50ml", {'entities': [(0, 5, 'color'), (17, 21, 'size')]}), 
("sandals 96861 shoes summer shoes flat sandals ruskea carla f", {'entities': [(0, 7, 'product'), (14, 19, 'product'), (46, 52, 'color'), (53, 60, 'brand')]}), 
("ray ban blaze shooter rb3581n 153/7v", {'entities': [(0, 7, 'brand')]}), 
("superliner le khol, 111 urban grey", {'entities': [(30, 34, 'color')]}), 
("rayban blaze shooter rb3581n 90387j", {'entities': [(0, 7, 'brand')]}), 
("hmlisam hoodie svetari collegepaita sininen hummel", {'entities': [(36, 43, 'color'), (44, 50, 'brand'), (8, 22, 'product')]}), 
("huawei freebuds 3 bluetooth vastamelukuulokkeet punainen", {'entities': [(48, 56, 'color'), (28, 47, 'product'), (0, 6, 'brand')]}), 
("icons pin point  tyynyliina 65x65 cm blue", {'entities': [(28, 36, 'size'), (37, 41, 'color'), (17, 27, 'product')]}), 
("kengät clarks  un tynamo turn", {'entities': [(7, 13, 'brand'), (0, 6, 'product')]}), 
("pants woven kapeat housut pillilahkeet musta esprit casual", {'entities': [(39, 44, 'color'), (45, 58, 'brand'), (19, 25, 'product'), (0, 5, 'product')]}), 
("polo ralph lauren silmälasit ph2126 5534", {'entities': [(18, 28, 'product'), (0, 4, 'brand')]}), 
("russell   iconic tackle twill crewneck   keltainen   s", {'entities': [(41, 50, 'color'), (53, 54, 'size'), (0, 7, 'brand')]}), 
("day london cb bags small shoulder bags   crossbody bags keltainen day et", {'entities': [(56, 65, 'color'), (41, 55, 'product'), (14, 18, 'product'), (66, 72, 'brand'), (19, 24, 'size')]}), 
("jacksonville t shirts & tops sleeveless valkoinen american vintage", {'entities': [(40, 49, 'color'), (24, 28, 'product'), (13, 21, 'product'), (50, 66, 'brand')]}), 
("street tray m matalavartiset sneakerit tennarit valkoinen ecco", {'entities': [(48, 57, 'color'), (39, 47, 'product'), (29, 38, 'product'), (58, 62, 'brand'), (12, 13, 'size')]}), 
("stella mccartney aurinkolasit sk0030s kids 008", {'entities': [(17, 29, 'product'), (0, 16, 'brand')]}), 
("michelin alpin 5 ( 215/60 r17 100h xl )", {'entities': [(0, 8, 'brand'), (26, 29, 'size')]}), 
("jimmy choo silmälasit jc242 086", {'entities': [(0, 10, 'brand'), (11, 21, 'product')]}), 
("sandaalit juicy couture  wisp", {'entities': [(10, 23, 'brand'), (0, 9, 'product')]}), 
("kate spade silmälasit amelinda ht8", {'entities': [(11, 21, 'color'), (0, 10, 'brand')]}), 
("sandaalit camper  karolina", {'entities': [(0, 9, 'product'), (10, 16, 'brand')]}), 
("jimmy choo aurinkolasit ben/s 807", {'entities': [(0, 10, 'brand'), (11, 23, 'product')]}), 
("coach silmälasit hc6127u 5535", {'entities': [(6, 16, 'product'), (0, 5, 'brand')]}), 
("superdry silmälasit sdo lincoln 108", {'entities': [(9, 19, 'product'), (0, 8, 'brand')]}), 
("cebe aurinkolasit queenstown cbs031", {'entities': [(5, 17, 'product'), (0, 4, 'brand')]}), 
("festina 99999 miesten kello f16826 b sininen/teräs �44 mm", {'entities': [(14, 21, 'gender'), (22, 27, 'product'), (37, 44, 'color'), (0, 7, 'brand'), (52, 57, 'size'), (45, 50, 'material')]}), 
("regular chino shorts shorts chinos shorts musta morris", {'entities': [(42, 47, 'color'), (48, 54, 'brand'), (14, 20, 'product')]}), 
("hackett silmälasit heb178 684", {'entities': [(8, 18, 'product'), (0, 7, 'brand')]}), 
("seventh street silmälasit 7a540 dq2", {'entities': [(15, 25, 'product'), (0, 14, 'brand')]}), 
("ray ban marshal rb3648 002/58", {'entities': [(0, 7, 'brand')]}), 
("sandaalit dorking  summer", {'entities': [(0, 9, 'product')]}), 
("screwdivision d over sweat shirt svetari collegepaita musta diesel", {'entities': [(21, 32, 'product'), (54, 59, 'color'), (60, 66, 'brand')]}), 
("superliner le khol, 116 forest green", {'entities': [(24, 36, 'color')]}), 
("swarovski aurinkolasit sk0203 90w", {'entities': [(10, 22, 'product'), (0, 9, 'brand')]}), 
("seventh street silmälasit 7a539 2m2", {'entities': [(15, 25, 'product'), (0, 14, 'brand')]}), 
("ray ban rb3558 002/8g", {'entities': [(0, 7, 'brand')]}), 
("bandon print polo men polos short sleeved punainen peak performance", {'entities': [(42, 50, 'color'), (13, 17, 'brand'), (28, 41, 'product')]}), 
("boyleg maillot uimapuku uima asut sininen seafolly", {'entities': [(34, 41, 'color'), (42, 50, 'brand'), (15, 23, 'product')]}), 
("paita selma", {'entities': [(0, 5, 'product')]}), 
("ciel small ear accessories jewellery earrings studs kulta sn� of sweden", {'entities': [(52, 57, 'material'), (27, 45, 'product'), (58, 71, 'brand')]}), 
("prada silmälasit pr65rv vic1o1", {'entities': [(6, 16, 'product'), (0, 5, 'brand')]}), 
("perf tee m night night t shirts short sleeved musta björn borg", {'entities': [(52, 62, 'brand'), (23, 32, 'product'), (32, 45, 'product'), (46, 51, 'color'), (9, 10, 'size')]}), 
("manhattan huopa steel blue dusty ocean", {'entities': [(10, 15, 'product'), (22, 32, 'color')]}), 
("boston candlestick/vase, 21,5 cm", {'entities': [(25, 32, 'size'), (19, 23, 'product'), (7, 18, 'product')]}), 
("polo ralph lauren aurinkolasit ph4099 564873", {'entities': [(18, 30, 'product'), (0, 17, 'brand')]}), 
("paita allie", {'entities': [(0, 5, 'product')]}), 
("superstay 24h foundation 30ml, fawn", {'entities': [(25, 29, 'size')]}), 
("elara dress polvipituinen mekko sininen by malina", {'entities': [(40, 49, 'brand'), (32, 39, 'color'), (26, 31, 'product'), (6, 11, 'product')]}), 
("prada silmälasit pr10vv uf31o1", {'entities': [(6, 16, 'product'), (0, 5, 'brand')]}), 
("wiley x silmälasit bounce kids yfbnc02", {'entities': [(8, 18, 'product'), (0, 7, 'brand')]}), 
("jazz 149 kylpyhuoneenmatto 70x160cm", {'entities': [(27, 35, 'size'), (9, 26, 'product')]}), 
("bulb riippuvalaisin sr1, kirkas johto", {'entities': [(0, 4, 'product')]}), 
("chrissline jatkosarja micro 100 lediä", {'entities': [(28, 36, 'material')]}), 
("mercedes silmälasit m 6042 d", {'entities': [(0, 8, 'brand'), (9, 19, 'product')]}), 
("dior aurinkolasit black tie 247s vk6/e8", {'entities': [(5, 17, 'product'), (18, 23, 'color'), (0, 4, 'brand')]}), 
("extra dimension rich core beauty women makeup eyes eyeshadow   not palettes monivärinen/kuvioitu m.a.c.", {'entities': [(97, 103, 'brand'), (33, 38, 'gender'), (51, 60, 'product'), (76, 87, 'color')]}), 
("vibratissimo play gel vegan 50ml", {'entities': [(28, 32, 'size'), (22, 27, 'material'), (13, 21, 'product'), (0, 12, 'product')]}), 
("tiffany & co. aurinkolasit tf3068 61419s", {'entities': [(14, 26, 'product'), (0, 13, 'brand')]}), 
("alpina aurinkolasit nylos hr a8635331", {'entities': [(7, 19, 'product'), (0, 6, 'brand')]}), 
("silk top regular w/elastic band t shirts & tops sleeveless vaaleanpunainen rosemunde", {'entities': [(75, 84, 'brand'), (59, 74, 'color'), (43, 47, 'product'), (32, 40, 'product'), (0, 5, 'material')]}), 
("lyhyt mekko armani exchange  helabina", {'entities': [(6, 11, 'product'), (12, 27, 'brand')]}), 
("tommy hilfiger silmälasit th 1494 900", {'entities': [(0, 14, 'brand'), (15, 25, 'product')]}), 
("chopard silmälasit vchb52g 579", {'entities': [(0, 7, 'brand'), (8, 18, 'product')]}), 
("chloe aurinkolasit ce 158s 856", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("ladies thin ankle sock lingerie hosiery socks harmaa decoy", {'entities': [(40, 45, 'product'), (0, 6, 'gender'), (53, 58, 'brand'), (46, 52, 'color')]}), 
("casio casio sport miesten kello mtd 1060d 7a2 hopea/teräs ø45.5 mm", {'entities': [(59, 66, 'size'), (52, 57, 'material'), (46, 51, 'color'), (26, 31, 'product'), (18, 25, 'gender'), (0, 5, 'brand')]}), 
("superstay 24h foundation 30ml, light beige", {'entities': [(31, 42, 'color'), (25, 29, 'size')]}), 
("superstay 24h foundation 30ml, sun beige", {'entities': [(31, 40, 'color'), (25, 29, 'size')]}), 
("3v cr2025 lithium paristo philips, 1 kpl", {'entities': [(25, 32, 'brand'), (10, 16, 'material'), (0, 2, 'size')]}), 
("michelin pilot alpin 5 ( 265/50 r20 111v xl, suv )", {'entities': [(0, 8, 'brand'), (25, 43, 'size')]}), 
("kengät dr martens  1460", {'entities': [(0, 6, 'product'), (7, 17, 'brand')]}), 
("smoke juomalasi 35 cl", {'entities': [(6, 15, 'product'), (16, 21, 'size')]}), 
("tray table, d70 h45cm, galv", {'entities': [(17, 21, 'size'), (0, 10, 'product')]}), 
("tiffany & co. silmälasit tf2168 8208", {'entities': [(14, 24, 'product'), (0, 13, 'brand')]}), 
("amazonia shampoo 200ml", {'entities': [(9, 16, 'product'), (17, 22, 'size')]}), 
("nokian nordman 7 suv ( 245/65 r17 111t xl, nastarengas  )", {'entities': [(0, 6, 'brand'), (22, 40, 'size')]}), 
("amazonia repairing balm 100ml", {'entities': [(9, 23, 'product'), (24, 29, 'size')]}), 
("alpina aurinkolasit sibiria a8316337", {'entities': [(7, 19, 'product'), (0, 6, 'brand')]}), 
("tyttöjen sandaalit acebo's  sarino", {'entities': [(9, 18, 'product'), (0, 8, 'gender'), (19, 26, 'brand')]}), 
("sc louise pitkähihainen paita musta soyaconcept", {'entities': [(36, 47, 'brand'), (30, 35, 'color'), (24, 29, 'product')]}), 
("pörröinen tekoturkistakki", {'entities': [(10, 25, 'product')]}), 
("amazonia mask 250ml", {'entities': [(14, 19, 'size'), (9, 13, 'product')]}), 
("sandaalit le temps des cerises  poly", {'entities': [(0, 9, 'product'), (10, 30, 'brand')]}), 
("ginna blouses short sleeved musta custommade", {'entities': [(6, 13, 'product'), (28, 33, 'color'), (34, 44, 'brand')]}), 
("bering ceramic naisten kello 11429 787 sininen/teräs ø29 mm", {'entities': [(54, 59, 'size'), (47, 52, 'material'), (39, 46, 'color'), (23, 28, 'product'), (15, 22, 'gender'), (0, 6, 'brand')]}), 
("tom ford aurinkolasit ft0613 01c", {'entities': [(9, 21, 'product'), (0, 8, 'brand')]}), 
("karl lagerfeld silmälasit kl 1003 616", {'entities': [(15, 25, 'product'), (0, 14, 'brand')]}), 
("first instinct, deostick 75g", {'entities': [(25, 28, 'size'), (16, 24, 'product')]}), 
("silver essence, edp 100ml", {'entities': [(20, 25, 'size')]}), 
("berlin armchair loose cover , caleido beige", {'entities': [(30, 43, 'color'), (7, 27, 'product')]}), 
("lastenkengät hummel  stadil poly boot mid jr", {'entities': [(0, 6, 'gender'), (6, 12, 'product'), (33, 37, 'product'), (13, 19, 'brand')]}), 
("crullé 19034 c2", {'entities': [(0, 6, 'brand')]}), 
("chloe aurinkolasit ce 114sd asian fit 776", {'entities': [(6, 18, 'product'), (0, 5, 'brand')]}), 
("institutional logo waist pack bumbag vyölaukku laukku musta calvin klein", {'entities': [(60, 72, 'brand'), (54, 59, 'color'), (47, 53, 'product')]}), 
("marc jacobs aurinkolasit marc 168/s ghp/ic", {'entities': [(0, 11, 'brand'), (12, 24, 'product')]}), 
("moncler silmälasit ml5046 092", {'entities': [(0, 7, 'brand'), (8, 18, 'product')]}),
("Hp Powerful desktop with NVIDIA GeForce GTX 1650 video card 256 SSD storage", {'entities':[(0,2,'brand'),(12,19,'product'),(25,31,'brand'),(32,43,'product'),(49,59,'product'),(64,67,'product'),(60,63,'size')]}), 
("porsche design silmälasit p8326 c", {'entities': [(0, 14, 'brand'), (15, 25, 'product')]}), 
("blake nora blazer bleiseri puvun takki ruskea mos mosh", {'entities': [(11, 17, 'product'), (33, 38, 'product'), (18, 32, 'product'), (39, 45, 'color'), (46, 54, 'brand')]}), 
("flexagon energy tr 2.0 shoes sport shoes training shoes  golf/tennis/fitness musta reebok performance", {'entities': [(83, 101, 'brand'), (77, 82, 'color'), (29, 40, 'product'), (23, 28, 'product')]}), 
("rey flat ballerinat ruskea filippa k", {'entities': [(20, 26, 'color'), (27, 36, 'brand')]}), 
("satin stretch   lotte bc pitkähihainen pusero paita ruskea sand", {'entities': [(52, 58, 'color'), (59, 63, 'brand'), (39, 51, 'product')]}), 
("kengät supra  noiz", {'entities': [(0, 6, 'product'), (7, 12, 'brand')]}), 
("marc jacobs marc 441/f j5g", {'entities': [(0, 11, 'brand')]}), 
("quay australia aurinkolasit qu 000435 omen milkytort/blu", {'entities': [(15, 27, 'product'), (0, 14, 'brand')]}), 
("polaroid pld 4073/f/s 086/la", {'entities': [(0, 8, 'brand')]}), 
("bobbi brown silmalasit the hemsley ryw", {'entities': [(12, 22, 'product'), (0, 11, 'brand')]}), 
("paita fondato", {'entities': [(0, 5, 'product')]}), 
("salvatore ferragamo aurinkolasit sf 189s 742", {'entities': [(20, 32, 'product'), (0, 19, 'brand')]}), 
("hair cond& groom aid formula 133 hoitoaine hiustenhoito nude kiehl's", {'entities': [(61, 68, 'brand'), (56, 60, 'color'), (43, 55, 'product')]}), 
("binasz jeans suorat farkut beige saint tropez", {'entities': [(20, 26, 'product'), (7, 12, 'product'), (33, 45, 'brand')]}), 
("woms boots shoes boots ankle boots ankle boot   flat musta tamaris", {'entities': [(5, 10, 'product'), (11, 16, 'product'), (59, 66, 'brand'), (53, 58, 'color')]}), 
("darling lace lingerie bras & tops soft bras beige love stories", {'entities': [(22, 33, 'product'), (50, 62, 'brand')]}), 
("danish design dress naisten kello iv15q1175 hopea/nahka ø30 mm", {'entities': [(57, 62, 'size'), (50, 55, 'material'), (44, 49, 'color'), (28, 33, 'product'), (20, 27, 'gender'), (0, 13, 'brand')]}), 
("tiffany & co. aurinkolasit tf3070 60019s", {'entities': [(0, 13, 'brand'), (14, 26, 'product')]}), 
("prodesign silmälasit 1791 9045", {'entities': [(0, 9, 'brand'), (10, 20, 'product')]}), 
("be fabulous fine cream conditi r hoitoaine hiukset nude revlon professional", {'entities': [(56, 75, 'brand'), (51, 55, 'color'), (33, 50, 'product'), (17, 22, 'material')]}), 
("sleepover set accessories travel accessories kermanvärinen love stories", {'entities': [(59, 71, 'brand')]}), 
("kengät les tropéziennes par m belarbi  paco", {'entities': [(0, 6, 'product'), (7, 37, 'brand')]}), 
("tjw soft touch v neck sweater neulepaita valkoinen tommy jeans", {'entities': [(51, 62, 'brand'), (41, 50, 'color'), (30, 40, 'product'), (22, 29, 'product')]}), 
("etnia barcelona aurinkolasit bonanova sun ywhv", {'entities': [(0, 15, 'brand'), (16, 28, 'product')]}), 
("siliävä tennispaita roger kent vaaleansininen::laivastonsininen", {'entities': [(8, 19, 'product'), (31, 45, 'color'), (47, 63, 'color'), (20, 30, 'brand')]}), 
("otto swim pants seersucker shortsit ruskea liewood", {'entities': [(5, 15, 'product'), (36, 42, 'color'), (43, 50, 'brand'), (27, 35, 'product')]}), 
("persol aurinkolasit po3170s 9041q8", {'entities': [(7, 19, 'product'), (0, 6, 'brand')]}), 
("swiss military hanowa miesten kello 06 3308.04.007 musta/teräs ø44", {'entities': [(57, 62, 'material'), (51, 56, 'color'), (30, 35, 'product'), (22, 29, 'gender'), (0, 14, 'brand')]}), 
("t paidat pitkillä hihoilla moony mood  fipolo", {'entities': [(0, 8, 'product'), (27, 37, 'brand')]}), 
("polo ralph lauren   custom fit denim shirt   sininen   xl", {'entities': [(45, 52, 'color'), (55, 57, 'size'), (0, 10, 'brand'), (37, 42, 'product')]}), 
("guess 99999 naisten kello w1231l2 hopea/kullansävytetty teräs ø38", {'entities': [(63, 65, 'size'), (56, 61, 'material'), (34, 39, 'color'), (20, 25, 'color'), (12, 19, 'gender'), (0, 5, 'brand')]}), 
("jbl clip 3 bluetooth kaiutin harmaa", {'entities': [(0, 3, 'brand'), (29, 35, 'color'), (11, 28, 'product')]}), 
("olivia toppatakki vaaleanpunainen marmar cph", {'entities': [(34, 44, 'brand'), (18, 33, 'color'), (7, 17, 'product')]}), 
("padded parka parka takki musta rains", {'entities': [(25, 30, 'color'), (19, 24, 'product'), (31, 36, 'brand')]}), 
("icons original  käsipyyhe 50x100 cm black", {'entities': [(36, 41, 'color'), (26, 35, 'size'), (16, 25, 'product')]}), 
("l'eau d'issey pour homme, shower gel 200ml", {'entities': [(26, 36, 'product'), (37, 42, 'size')]}), 
("vintage, edt 125ml", {'entities': [(13, 18, 'size')]}), 
("tonymoly i'm hand cream avocado 30ml beauty men skin care body hand cream nude tonymoly", {'entities': [(32, 36, 'size'), (44, 47, 'gender'), (13, 23, 'product'), (24, 31, 'material'), (0, 8, 'brand'), (74, 78, 'color')]}),
("3176   latia pitkähihainen paita sininen sand", {'entities': [(41, 45, 'brand'), (33, 40, 'color'), (27, 32, 'product')]}), 
("nautica bfd 100 miesten kello n15107g musta/kumi ø49 mm", {'entities': [(50, 55, 'size'), (44, 49, 'material'), (38, 43, 'color'), (24, 29, 'product'), (16, 23, 'gender'), (0, 7, 'brand')]}), 
("satisfyer love triangle black", {'entities': [(24, 29, 'color'), (0, 9, 'brand')]}), 
("oakley aurinkolasit oo9271 flak 2.0 asian fit 927135", {'entities': [(0, 6, 'brand'), (7, 19, 'product')]}), 
("nautica nsr miesten kello n12640g musta/kumi ø45 mm", {'entities': [(40, 44, 'material'), (34, 39, 'color'), (20, 25, 'product'), (12, 19, 'gender'), (0, 7, 'brand')]}), 
("vera paita\r", {'entities': [(5, 10, 'product')]}), 
("belted midi length wrap dress polvipituinen mekko ruskea scotch & soda", {'entities': [(50, 56, 'color'), (44, 49, 'product'), (57, 70, 'brand')]}), 
("juomapullo 600ml purple leaves", {'entities': [(0, 10, 'product'), (17, 23, 'color'), (11, 16, 'size')]}), 
("midge mid bootcut wmn bootcut farkut sininen g star raw", {'entities': [(30, 36, 'product'), (37, 44, 'color'), (45, 55, 'brand')]}), 
("kengät andré  kilwal", {'entities': [(0, 6, 'product'), (7, 12, 'brand')]}), 
("icons original  kylpypyyhe 100x150 cm hazel brown", {'entities': [(44, 49, 'color'), (27, 37, 'size'), (16, 26, 'product')]}), 
("cazal silmälasit 1203 004", {'entities': [(6, 16, 'product'), (0, 5, 'brand')]}), 
("pai suoralahkeiset housut musta masai", {'entities': [(26, 31, 'color'), (19, 25, 'product'), (32, 37, 'brand')]}), 
("taitoa outerwear thermo outerwear thermo sets sininen reima", {'entities': [(46, 53, 'color'), (17, 33, 'product'), (54, 59, 'brand')]}), 
("olympéa, body lotion 200ml", {'entities': [(0, 7, 'brand'), (9, 20, 'product'), (21, 26, 'size')]}), 
("bvlgari silmälasit bv2217b 2033", {'entities': [(0, 7, 'brand'), (8, 18, 'product')]}), 
("bering ceramic naisten kello 11435 787 sininen/teräs 35 mm", {'entities': [(54, 59, 'size'), (47, 52, 'material'), (39, 46, 'color'), (23, 28, 'product'), (15, 22, 'gender'), (0, 6, 'brand')]}), 
("malachite necklace adj. 40 48 cm accessories jewellery necklaces dainty necklaces kulta pernille corydon", {'entities': [(24, 32, 'size'), (45, 54, 'product'), (55, 65, 'product'), (82, 87, 'material'), (88, 104, 'brand')]}),
("bolle aurinkolasit slate polarized 12425", {'entities': [(6, 18, 'product'), (0, 5, 'brand')]}), 
("art peili setti 23x48cm, harmaa", {'entities': [(16, 23, 'size'), (4, 9, 'product'), (25, 31, 'color')]}), 
("birgacr shirt pitkähihainen paita ruskea cream", {'entities': [(28, 33, 'product'), (8, 13, 'product'), (34, 40, 'color'), (41, 46, 'brand')]}), 
("simpo ja outerwear faux fur musta ichi", {'entities': [(28, 33, 'color'), (34, 38, 'brand'), (9, 18, 'product')]}), 
("fatma l_s mesh dress exp mekko monivärinen/kuvioitu the new", {'entities': [(31, 42, 'color'), (25, 30, 'product'), (52, 59, 'brand'), (10, 20, 'product')]}), 
("flawless finish mousse makeup 02 natural meikkivoide meikki elizabeth arden", {'entities': [(60, 75, 'brand'), (16, 22, 'product'), (33, 40, 'material')]}), 
("tri vita c30, 30ml", {'entities': [(14, 18, 'size')]}), 
("led yleisvalaisin lumo ip44 9w 900lm 27x6cm", {'entities': [(37, 43, 'size'), (31, 36, 'size'), (0, 3, 'product')]}), 
("matkalaukku david jones  chauvetto 40l", {'entities': [(35, 38, 'size'), (12, 23, 'brand'), (0, 11, 'product')]}), 
("flaska neo authentic 0,75l sininen", {'entities': [(21, 26, 'size')]}), 
("healing moisture moi moi hair masque 200ml", {'entities': [(37, 42, 'size'), (8, 16, 'product')]}), 
("volupte liquid balm 7 huultenhoito meikki vaaleanpunainen yves saint laurent", {'entities': [(58, 76, 'brand'), (42, 57, 'color'), (22, 34, 'product')]}), 
("healing moisture moi moi hair masque 50ml", {'entities': [(37, 41, 'size'), (8, 16, 'product')]}), 
("kengät jb martin  1falba", {'entities': [(0, 6, 'product'), (7, 16, 'brand')]}), 
("fendi silmälasit ff 0394 807", {'entities': [(6, 16, 'product'), (0, 5, 'brand')]}), 
("jimmy choo ave/s pef/ez\r", {'entities': [(0, 10, 'brand')]}), 
("6192621, knit   alexian o neck neulepaita pyöreö kaula aukko sininen solid", {'entities': [(61, 68, 'color'), (69, 74, 'brand'), (31, 41, 'product')]}), 
("carrera carrera 1021/s v81/2k", {'entities': [(0, 7, 'brand')]}), 
("toppi elsa\r", {'entities': [(6, 10, 'brand'), (0, 5, 'product')]}), 
("marc jacobs aurinkolasit marc 182/s 086/ha", {'entities': [(0, 11, 'brand'), (12, 24, 'product')]}), 
("healing moisture noni fruit leave in conditioner, 250ml", {'entities': [(50, 55, 'size'), (0, 16, 'product'), (37, 48, 'product')]}), 
("smartbuy collection silmälasit diao t 1414 001", {'entities': [(20, 30, 'product'), (0, 19, 'brand')]}), 
("rolex datejust 41 miesten kello 126333 0017 valkoinen/18k keltakultaa", {'entities': [(44, 53, 'color'), (54, 57, 'material'), (26, 31, 'product'), (18, 25, 'gender'), (0, 5, 'brand')]}), 
("pusero robin", {'entities': [(0, 6, 'product'), (7, 12, 'brand')]}), 
("smartbuy collection silmälasit roy 981e", {'entities': [(0, 8, 'brand'), (20, 30, 'product')]}), 
("colonia ebano, edc 100ml", {'entities': [(0, 13, 'brand'), (19, 24, 'size')]}), 
("kengät diesel  s kby", {'entities': [(0, 6, 'product'), (7, 13, 'brand')]}), 
("slflulu ls knit short cardigan noos neuletakki ruskea selected femme", {'entities': [(16, 30, 'product'), (36, 46, 'product'), (47, 53, 'color'), (54, 68, 'brand')]}), 
("delaware3 1 tiukat farkut sininen boss", {'entities': [(19, 25, 'product'), (26, 33, 'color'), (34, 38, 'brand')]}), 
("pusero univ tee 1 w", {'entities': [(0, 6, 'product'), (7, 15, 'brand')]}), 
("kietaisupusero kalviniw blouse", {'entities': [(0, 14, 'product'), (15, 30, 'brand')]}), 
("supermicro mtw bare core i7/5/3 q170 4x3.5 250w 64gb sata3 2gbe dp 1pcie in bare", {'entities': [(0, 10, 'brand')]}), 
("anna stretchtoppi", {'entities': [(5, 17, 'product'), (0, 4, 'brand')]}), 
("reppu herschel  retreat", {'entities': [(0, 5, 'product'), (6, 14, 'brand')]}), 
("fossil tailor naisten kello es4052 valkoinen / nahka ø34 mm", {'entities': [(14, 21, 'gender'), (22, 27, 'product'), (35, 44, 'color'), (47, 52, 'material'), (53, 59, 'size'), (0, 13, 'brand')]}), 
("marc jacobs silmälasit mj 508 8np", {'entities': [(0, 11, 'brand'), (12, 22, 'product')]}), 
("nokian rotiiva at ( lt235/75 r15 116/113s 10pr )", {'entities': [(0, 17, 'brand'), (20, 41, 'size')]}), 
("armour l2 ( 20.5  25 16pr tl )", {'entities': [(0, 6, 'brand'), (12, 28, 'size')]}), 
("boss by hugo boss silmälasit boss 0948 807", {'entities': [(18, 28, 'product'), (29, 33, 'brand'), (8, 17, 'brand'), (0, 4, 'brand')]}), 
("kengät marc o'polo  neretta", {'entities': [(0, 6, 'product'), (7, 18, 'brand')]}), 
("korkokengät andré  jasmine", {'entities': [(0, 11, 'product'), (19, 26, 'brand')]}), 
("lastenkengät new balance  997", {'entities': [(0, 12, 'product'), (13, 24, 'brand')]}), 
("d1. cotton rib knit hat accessories headwear beanies musta gant", {'entities': [(59, 63, 'brand'), (53, 58, 'color'), (20, 23, 'product'), (24, 35, 'product'), (36, 44, 'product'), (4, 10, 'material')]}), 
("t shirt rn t shirts short sleeved valkoinen boss", {'entities': [(0, 7, 'product'), (11, 19, 'product'), (34, 43, 'color'), (44, 48, 'brand')]}), 
("shorts shorts leather shorts musta sofie schnoor", {'entities': [(0, 6, 'product'), (14, 21, 'material'), (29, 34, 'color'), (35, 48, 'brand')]}), 
("longines symphonette naisten kello l2.305.4.71.6 hopea/teräs ø16 mm", {'entities': [(21, 28, 'gender'), (29, 34, 'product'), (49, 60, 'color'), (61, 67, 'size'), (0, 8, 'brand')]}), 
("gömma säilytyspurkki small, valkoinen", {'entities': [(6, 20, 'product'), (21, 26, 'size'), (28, 37, 'color'), (0, 5, 'brand')]}), 
("colonia club, shower gel 200ml", {'entities': [(0, 12, 'brand'), (14, 24, 'product'), (25, 30, 'size')]}), 
("colonia club, after shave balm 100ml", {'entities': [(0, 12, 'brand'), (14, 30, 'product'), (31, 36, 'size')]}), 
("feeling fine top t shirts & tops long sleeved valkoinen odd molly", {'entities': [(13, 16, 'product'), (17, 32, 'product'), (46, 55, 'color'), (56, 65, 'brand')]}), 
("wvngoag4 shoes sport shoes running shoes harmaa new balance", {'entities': [(9, 14, 'product'), (15, 26, 'product'), (27, 40, 'product'), (41, 47, 'color'), (48, 59, 'brand')]}), 
("colonia club, after shave lotion 100ml", {'entities': [(0, 12, 'brand'), (14, 32, 'product'), (33, 38, 'size')]}), 
("retrosuperfuture aurinkolasit classic kids 218", {'entities': [(17, 29, 'product'), (0, 16, 'brand')]}), 
("rae accessories headwear beanies harmaa re:designed est 2003", {'entities': [(4, 15, 'product'), (16, 24, 'product'), (33, 39, 'color'), (40, 55, 'brand')]}), 
("tessa knitted sweater", {'entities': [(14, 21, 'product'), (0, 5, 'brand')]}), 
("ecodenta brilliant whitening valkaiseva hammastahna 100ml", {'entities': [(40, 51, 'product'), (52, 57, 'size'), (0, 8, 'brand')]}), 
("day essence neulepaita vaaleanpunainen day birger et mikkelsen", {'entities': [(12, 22, 'product'), (23, 38, 'color'), (39, 62, 'brand')]}), 
("glo ball w1 seinävalaisin", {'entities': [(12, 25, 'product'), (0, 8, 'brand')]}), 
("kontio whitepaw classic ( 205/75 r15 97r ww 65mm )", {'entities': [(0, 6, 'brand'), (26, 48, 'size')]}), 
("olkalaukut karl lagerfeld  k/odina hobo", {'entities': [(0, 10, 'product'), (11, 25, 'brand')]}), 
("kate spade silmälasit camberly 0t4", {'entities': [(0, 10, 'brand'), (11, 21, 'product')]}), 
("colonia pura, edc 100ml", {'entities': [(0, 12, 'brand'), (18, 23, 'size')]}), 
("lastenkengät citrouille et compagnie  limette", {'entities': [(0, 12, 'product'), (38, 45, 'brand')]}), 
("ray ban tech aurinkolasit rb4210 wayfarer light ray 612213", {'entities': [(0, 7, 'brand'), (13, 25, 'product')]}), 
("chloe aurinkolasit ce 137s 767", {'entities': [(0, 5, 'brand'), (6, 17, 'product')]}), 
("chloe silmälasit ce 2705 321", {'entities': [(0, 5, 'brand'), (6, 16, 'product')]}), 
("zac posen silmälasit humphrey silver", {'entities': [(10, 20, 'product'), (30, 36, 'color'), (0, 9, 'brand')]}), 
("pusero jdynora life s/s anglaise top", {'entities': [(0, 6, 'product'), (7, 14, 'brand'), (20, 23, 'size')]}), 
("icon crew neulepaita pyöreä kaula aukko harmaa abercrombie & fitch", {'entities': [(10, 20, 'product'), (40, 46, 'color'), (47, 66, 'brand')]}), 
("breguet classique miesten kello 5140bb 12 9w6 hopea/nahka ø40 mm", {'entities': [(18, 25, 'gender'), (26, 31, 'product'), (46, 57, 'material'), (58, 64, 'size'), (0, 17, 'brand')]}), 
("dragon alliance aurinkolasit dr post up 001", {'entities': [(16, 28, 'product'), (0, 15, 'brand')]}), 
("acorelle kylmävahaliuskat vartalolle 20 kpl", {'entities': [(37, 43, 'size'), (9, 25, 'product'), (0, 8, 'brand')]}), 
("smartbuy collection silmälasit kaya a84", {'entities': [(0, 8, 'brand'), (20, 30, 'product')]}), 
("puma pu0001s 006", {'entities': [(0, 4, 'brand')]}), 
("eric maccoat trenssi takki sininen les deux", {'entities': [(8, 12, 'product'), (27, 34, 'color'), (35, 43, 'brand')]}), 
("colette knit skirt polvipituinen hame monivärinen/kuvioitu morris lady", {'entities': [(13, 18, 'product'), (33, 37, 'product'), (59, 70, 'brand')]}), 
("dragon alliance aurinkolasit dr d1otg bonus 029", {'entities': [(0, 15, 'brand'), (16, 28, 'product')]}), 
("giti gsr225 (245/70 r19.5 136/134m 16pr )", {'entities': [(0, 4, 'brand'), (20, 25, 'size')]}), 
("apple cto cto:z0y3 mbp 16in tb 2.4ghz i9 32gb 1tb 4gb silver se btop", {'entities': [(0, 6, 'brand'), (54, 60, 'color'), (22, 53, 'size'), (19, 22, 'product')]}), 
("guess aurinkolasit gu 6947 06b", {'entities': [(0, 5, 'brand'), (5, 18, 'product')]}), 
("guess aurinkolasit gu 6947 06b", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("\xa0essential skinny chi housut beige tommy hilfiger", {'entities': [(34, 49, 'brand'), (24, 29, 'color'), (30, 32, 'product')]}), 
("gramicci gramicci pants khaki xl", {'entities': [(0, 8, 'brand'), (18, 23, 'product'), (36, 38, 'size'), (27, 34, 'color')]}), 
("nicola bikinit vaaleanpunainen molo", {'entities': [(7, 14, 'product'), (15, 30, 'color'), (31, 35, 'brand')]}), 
("ralph by ralph lauren aurinkolasit ra5252 57497b", {'entities': [(9, 21, 'brand'), (21, 34, 'product')]}), 
("riley sweater", {'entities': [(6, 13, 'product'), (0, 5, 'brand')]}), 
("oakley aurinkolasit oo9271 flak 2.0 asian fit 927103", {'entities': [(0, 6, 'brand'), (6, 19, 'product')]}), 
("boss by hugo boss silmälasit boss 1075 2m4", {'entities': [(8, 17, 'brand'), (18, 28, 'product')]}), 
("invicta excursion miesten kello 15315 hopea/teräs\xa049.5 mm", {'entities': [(0, 17, 'brand'), (18, 25, 'gender'), (26, 31, 'product'), (38, 43, 'color'), (44, 49, 'material'), (50, 57, 'size')]}), 
("pusero laura", {'entities': [(0, 6, 'product'), (7, 12, 'brand')]}), 
("hangable cosmetic black toilettilaukut musta pipol's bazaar", {'entities': [(17, 23, 'color'), (24, 38, 'product'), (38, 44, 'color'), (44, 59, 'brand')]}), 
("timberland silmälasit tb1597 091", {'entities': [(0, 10, 'brand'), (11, 21, 'product')]}), 
("y3441 00 shoes boots ankle boots ankle boot   flat musta rieker", {'entities': [(9, 14, 'product'), (15, 20, 'product'), (51, 56, 'color'), (57, 63, 'brand')]}), 
("flail black olkalaukku laukku musta eastpak", {'entities': [(5, 11, 'color'), (30, 35, 'color'), (36, 43, 'brand'), (23, 29, 'product'), (12, 22, 'product')]}), 
("mymagicwand nubbed attachment   pink", {'entities': [(32, 36, 'color'), (0, 11, 'brand')]}), 
("persol silmälasit po3227v 24", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("hammer tassel necklace", {'entities': [(14, 22, 'product'), (0, 13, 'brand')]}), 
("karl lagerfeld silmälasit kl 852 033", {'entities': [(0, 15, 'brand'), (15, 25, 'product')]}), 
("pusero bonn", {'entities': [(0, 6, 'product'), (7, 11, 'brand')]}), 
("nokian hakkapeliitta r2 suv ( 275/50 r20 113r xl, pohjoismainen kitkarengas )", {'entities': [(29, 47, 'size'), (0, 6, 'brand')]}), 
("nahkatakki naf naf xa0 chao", {'entities': [(5, 10, 'product'), (0, 5, 'material'), (11, 24, 'brand')]}), 
("pressed mineral foundation, 8g, rose beige", {'entities': [(0, 26, 'brand'), (28, 30, 'size'), (32, 42, 'color')]}), 
("mcq silmälasit mcq0027 k1i", {'entities': [(0, 3, 'brand'), (4, 14, 'product')]}), 
("barry stretch nylon vuorillinen takki topattu takki musta j. lindeberg", {'entities': [(14, 19, 'material'), (32, 37, 'product'), (52, 57, 'color')]}), 
("nike aurinkolasit essential chaser m ev0998 207", {'entities': [(0, 4, 'brand'), (4, 17, 'product')]}), 
("adidas originals   gazelle   musta   us 5", {'entities': [(0, 7, 'brand'), (29, 34, 'color'), (37, 41, 'size')]}), 
("robo winter paisley paita rento casual harmaa matinique", {'entities': [(39, 45, 'color'), (0, 19, 'brand')]}), 
("graphic t shirt t shirts short sleeved sininen fred perry", {'entities': [(8, 15, 'product'), (16, 24, 'product'), (39, 46, 'color'), (47, 57, 'brand')]}), 
("guess silmälasit gu 3052 075", {'entities': [(0, 5, 'brand'), (6, 16, 'product')]}), 
("rolex datejust 36 naisten kello 126234 0014 hopea/teräs 36 mm", {'entities': [(26, 31, 'product'), (44, 49, 'color'), (50, 55, 'material'), (56, 61, 'size'), (18, 25, 'gender'), (0, 5, 'brand')]}), 
("tods silmälasit to05038 047", {'entities': [(0, 4, 'brand'), (4, 15, 'product')]}), 
("\xa0ilyna blouses short sleeved valkoinen boss", {'entities': [(7, 15, 'product'), (29, 38, 'color'), (39, 43, 'brand')]}), 
("goodride radial sl369 a/t (lt315/70 r17 121/118r 10pr )", {'entities': [(0, 8, 'brand'), (36, 39, 'size')]}), 
("paita bonnie", {'entities': [(0, 5, 'product'), (6, 12, 'brand')]}), 
("cormack matalavartiset sneakerit tennarit valkoinen lyle & scott", {'entities': [(52, 64, 'brand'), (41, 51, 'color'), (33, 41, 'product'), (23, 33, 'product')]}), 
("nike silmälasit 5015 259", {'entities': [(0, 5, 'brand'), (5, 16, 'product')]}), 
("pressed mineral foundation, 8g, soft beige", {'entities': [(0, 26, 'brand'), (28, 30, 'size'), (37, 42, 'color')]}), 
("kengät tuk pointed creepers", {'entities': [(0, 6, 'product'), (19, 27, 'brand')]}), 
("kumho portran kc53 ( 195 r15c 106/104r 8pr )", {'entities': [(0, 5, 'brand'), (20, 42, 'size')]}), 
("jaguar silmälasit\xa0", {'entities': [(0, 6, 'brand'), (7, 18, 'product')]}), 
("yokohama bluearth van ry55 ( 205/70 r15c 106/104s bluearth )", {'entities': [(29, 49, 'size'), (0, 8, 'brand')]}), 
("ermenegildo zegna aurinkolasit ez0137 02e", {'entities': [(0, 17, 'brand'), (18, 30, 'product')]}), 
("ray ban silmälasit tech rx8901 carbon fibre 5263", {'entities': [(0, 8, 'brand'), (8, 18, 'product')]}), 
("healing moisture kukui nut conditioner, 250ml", {'entities': [(23, 26, 'material'), (27, 38, 'product'), (40, 45, 'size')]}), 
("iris slim joulutähti 80 cm hopeakulta", {'entities': [(10, 20, 'product'), (21, 26, 'size'), (27, 38, 'color')]}), 
("l/s batwing tee t shirts long sleeved t shirts punainen levi's", {'entities': [(16, 24, 'product'), (25, 46, 'product'), (47, 55, 'color'), (56, 62, 'brand')]}), 
("cotton body t shirts & tops bodies oranssi adidas originals", {'entities': [(0, 6, 'material'), (12, 27, 'product'), (35, 42, 'color'), (43, 59, 'brand')]}), 
("bunaken pöytäliina vihreä  vaaleanpunainen", {'entities': [(8, 18, 'product'), (19, 43, 'color')]}), 
("lastenkengät emu wallaby mini metalic", {'entities': [(13, 16, 'brand'), (30, 37, 'color'), (25, 29, 'size')]}), 
("nicole miller silmälasit nm columbus c02", {'entities': [(0, 13, 'brand'), (14, 24, 'product')]}), 
("healing moisture kukui nut conditioner, 750ml", {'entities': [(23, 26, 'material'), (27, 38, 'product'), (40, 45, 'size')]}), 
("liza accessories headwear hats vihreä baum und pferdgarten", {'entities': [(17, 25, 'product'), (26, 30, 'product'), (31, 37, 'color'), (38, 58, 'brand')]}), 
("sc marica t shirts & tops long sleeved ruskea soyaconcept", {'entities': [(10, 25, 'product'), (39, 45, 'color'), (46, 57, 'brand')]}), 
("stockholm veitsi 23 cm", {'entities': [(10, 16, 'product'), (17, 22, 'size')]}), 
("3 paria sukkia, joissa liukuestepohjat ja eläinkuviointi", {'entities': [(8, 14, 'product')]}), 
("matador mp62 all weather evo ( 155/65 r14 75t )", {'entities': [(0, 7, 'brand'), (8, 12, 'product'), (31, 45, 'size')]}), 
("hiker next generation women's drymaxx shell jacket outerwear sport jackets musta halti", {'entities': [(22, 29, 'gender'), (44, 50, 'product'), (75, 80, 'color'), (81, 86, 'brand')]}), 
("healing style curl define 125g", {'entities': [(26, 30, 'size'), (0, 13, 'product')]}), 
("slim farkut diesel thommer", {'entities': [(0, 11, 'product'), (12, 18, 'brand')]}), 
("isla tank", {'entities': []}), 
("miu miu silmälasit mu05rv 1ab1o1", {'entities': [(0, 7, 'brand'), (8, 18, 'product')]}), 
("asta povi scarf seasonal col. huivi keltainen becksöndergaard", {'entities': [(10, 15, 'product'), (30, 35, 'product'), (36, 45, 'color'), (46, 61, 'brand')]}), 
("healing color care color preserving shampoo 50ml", {'entities': [(36, 43, 'product'), (44, 48, 'size')]}), 
("oakley aurinkolasit oo9154 half jacket 2.0 xl polarized 915467", {'entities': [(0, 6, 'brand'), (7, 19, 'product'), (27, 38, 'product')]}), 
("healing color care color preserving shampoo, 1000ml", {'entities': [(36, 43, 'product'), (45, 51, 'size')]}), 
("pelican hat accessories headwear hats harmaa sways", {'entities': [(33, 37, 'product'), (38, 44, 'color'), (45, 50, 'brand')]}), 
("oakley silmälasit ox5040 wingspan 504001", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("viking protech newgen ( 245/45 r19 102y xl )", {'entities': [(0, 6, 'brand'), (23, 42, 'size')]}), 
("matador mp92 sibir snow ( 245/70 r16 107t, suv )", {'entities': [(0, 7, 'brand'), (26, 40, 'size')]}), 
("brio labyrintti puinen", {'entities': [(0, 4, 'brand'), (5, 14, 'product'), (15, 21, 'color')]}), 
("matador mp92 sibir snow ( 205/60 r16 92h )", {'entities': [(0, 7, 'brand'), (26, 39, 'size')]}), 
("kengät adidas x_plr", {'entities': [(0, 6, 'product'), (7, 13, 'brand')]}), 
("chino housut / porkkanahousut eleven paris charlie", {'entities': [(0, 12, 'product'), (15, 29, 'product'), (30, 42, 'brand')]}), 
("d1. bc print slim bd paita bisnes sininen gant", {'entities': [(21, 26, 'product'), (34, 41, 'color'), (42, 46, 'brand')]}), 
("healing color care color preserving shampoo, 300ml", {'entities': [(19, 43, 'product'), (45, 50, 'size')]}), 
("tyttöjen sandaalit gbb cocorikoo", {'entities': [(0, 8, 'gender'), (9, 18, 'product'), (19, 22, 'brand')]}), 
("healing color care color preserving shampoo, 750ml", {'entities': [(19, 43, 'product'), (45, 50, 'size')]}), 
("polaroid pld d334 086", {'entities': [(0, 8, 'brand')]}), 
("nike säärisuojat mercurial lite   valkoinen/sininen/musta", {'entities': [(0, 4, 'brand'), (5, 31, 'product'), (34, 57, 'color')]}), 
("maxxis vansmart a/s al2 ( 235/65 r16c 115/113t )", {'entities': [(0, 6, 'brand'), (26, 45, 'size')]}), 
("guess silmälasit gu 2667 083", {'entities': [(0, 5, 'brand'), (6, 16, 'product')]}), 
("hipster lingerie panties hipsters/boyshorts/brazilian harmaa calvin klein", {'entities': [(8, 16, 'product'), (17, 24, 'product'), (54, 60, 'color'), (61, 73, 'brand')]}), 
("baloo   bodystocking bodies long sleeved vihreä hust & claire", {'entities': [(41, 47, 'color'), (48, 61, 'brand')]}), 
("kengät etro borchie", {'entities': [(0, 6, 'product'), (7, 11, 'brand')]}), 
("wiley x aurinkolasit compass cccmp01", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("peacock single earring left / gold accessories jewellery earrings studs kulta julie sandlau", {'entities': [(15, 22, 'product'), (47, 56, 'product'), (57, 65, 'product'), (72, 77, 'color'), (78, 91, 'brand')]}), 
("diesel aurinkolasit dl0253 52n", {'entities': [(0, 6, 'brand'), (7, 19, 'product')]}), 
("vickycr shirt pitkähihainen paita valkoinen cream", {'entities': [(28, 33, 'product'), (34, 43, 'color')]}), 
("healing color care color preserving conditioner 50ml", {'entities': [(36, 47, 'product'), (48, 52, 'size')]}), 
("polaroid aurinkolasit a8418 r1h", {'entities': [(0, 8, 'brand'), (9, 21, 'product')]}), 
("healing color care color preserving conditioner, 1000ml", {'entities': [(36, 47, 'product'), (49, 55, 'size')]}), 
("goodride radial sl369 a/t ( 225/70 r15 100t )", {'entities': [(0, 8, 'brand'), (28, 42, 'size')]}), 
("fendi silmälasit ff m0005 scl", {'entities': [(0, 5, 'brand'), (6, 16, 'product')]}), 
("6219   star napoli normal bleiseri puvun takki harmaa sand", {'entities': [(26, 34, 'product'), (35, 46, 'product'), (47, 53, 'color'), (54, 58, 'brand')]}), 
("blouses woven pitkähihainen pusero paita valkoinen esprit collection", {'entities': [(0, 7, 'product'), (28, 40, 'product'), (41, 50, 'color'), (51, 57, 'brand')]}), 
("dress long sleeve polvipituinen mekko oranssi noa noa", {'entities': [(0, 5, 'product'), (32, 37, 'product'), (38, 45, 'color'), (46, 53, 'brand')]}), 
("got2b twisted anti frizz styling spray 200 ml", {'entities': [(25, 38, 'product'), (39, 45, 'size')]}), 
("croon silmälasit leesbril madonna havana red transparent", {'entities': [(0, 5, 'brand'), (6, 16, 'product'), (41, 56, 'color')]}), 
("alliance farm pro 70 ( 520/70 r30 145a8 tl kaksoistunnus\xa0 145b )", {'entities': [(0, 8, 'brand'), (23, 42, 'size')]}), 
("poikien sandaalit andré\xa0 milan", {'entities': [(8, 17, 'product'), (18, 23, 'brand')]}), 
("paradise swimsuit uimapuku uima asut monivärinen/kuvioitu notes du nord", {'entities': [(9, 17, 'product'), (18, 26, 'product'), (27, 36, 'product'), (37, 48, 'color'), (49, 57, 'color'), (58, 71, 'brand')]}), 
("aj pöytälamppu musta", {'entities': [(3, 14, 'product'), (15, 20, 'color')]}), 
("intex swan ride on", {'entities': [(0, 5, 'brand')]}), 
("beyond perf. foundation + concealer, 6,5 buttermilk", {'entities': [(13, 35, 'product'), (41, 51, 'color')]}), 
("guess gu7459 01b", {'entities': [(0, 6, 'brand')]}), 
("can't stop won't stop contour concealer peitevoide meikki nyx professional makeup", {'entities': [(22, 39, 'product'), (40, 57, 'product'), (58, 81, 'brand')]}), 
("paita objsifka top 107", {'entities': [(0, 5, 'product')]}), 
("korkokengät rupert sanderson\xa0 clipper", {'entities': [(0, 11, 'product'), (12, 28, 'brand')]}), 
("ray ban silmälasit rb2132f new wayfarer asian fit polarized 901/58", {'entities': [(0, 7, 'brand'), (8, 18, 'product')]}), 
("bvlgari aurinkolasit bv6104 20144z", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("isabel alushousut brief tangat ruskea love stories", {'entities': [(7, 17, 'product'), (18, 30, 'product'), (31, 37, 'color'), (38, 50, 'brand')]}), 
("cr7 boys trunk 2 pack alushousut musta cr7", {'entities': [(0, 3, 'brand'), (22, 32, 'product'), (33, 38, 'color'), (39, 42, 'brand')]}), 
("seha agro 10 ( 420/70 r24 130a8 tl )", {'entities': [(0, 4, 'brand'), (15, 33, 'size')]}), 
("carlisle turf saver ( 18x8.50  8 74a4 4pr tl nhs )", {'entities': [(0, 8, 'brand')]}), 
("kengät ippon vintage\xa0 beach prince", {'entities': [(0, 6, 'product'), (7, 20, 'brand')]}), 
("high lengths mascara, black", {'entities': [(13, 20, 'product'), (22, 27, 'color')]}), 
("self sun face tinted lotion 50ml", {'entities': [(21, 27, 'product'), (28, 32, 'size')]}), 
("cargo loose suoralahkeiset housut vihreä please jeans", {'entities': [(27, 33, 'product'), (34, 40, 'color'), (41, 53, 'brand')]}), 
("rally sneakers matalavartiset sneakerit tennarit musta fitflop", {'entities': [(6, 14, 'product'), (40, 48, 'product'), (49, 54, 'color'), (55, 62, 'brand')]}), 
("ceat lift pro ( 460/70 r24 159a8 tl kaksoistunnus\xa0 159b, t.r.a. i3 )", {'entities': [(0, 4, 'brand'), (16, 34, 'size')]}), 
("farkut rex", {'entities': [(0, 6, 'product')]}), 
("isadora double ended eye shadow applicator", {'entities': [(0, 7, 'brand'), (21, 42, 'product')]}), 
("day dream on messinki", {'entities': [(13, 21, 'material')]}), 
("ceat farmax r1 ( 15.5  38 137a6 10pr tt )", {'entities': [(0, 4, 'brand'), (17, 38, 'size')]}), 
("radley silmälasit rdo natallia 003", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("vredestein v 61 ( 250/65  14.5 121a8 12pr tl )", {'entities': [(0, 10, 'brand'), (18, 43, 'size')]}), 
("superdefense moisturizer spf20 50ml (very dry/comb)", {'entities': [(13, 24, 'product'), (31, 35, 'size')]}), 
("quickliner for eyes, blue grey", {'entities': [(0, 19, 'product'), (21, 30, 'color')]}), 
("vogue eyewear silmälasit vo4094 5138", {'entities': [(0, 13, 'brand'), (14, 24, 'product')]}), 
("tennarit ua classic slip on", {'entities': [(0, 8, 'product')]}), 
("dolce & gabbana aurinkolasit dg2213 02/73", {'entities': [(0, 15, 'brand'), (16, 28, 'product')]}), 
("viceiw blazer bleiseri puvun takki ruskea inwear", {'entities': [(7, 13, 'product'), (14, 22, 'product'), (23, 34, 'product'), (35, 41, 'color'), (42, 48, 'brand')]}), 
("washed linen pussilakana tuplapeittoon, navy blue", {'entities': [(13, 38, 'product'), (40, 49, 'color')]}), 
("starmaxx tr 110 ( 200/70 r16 94a8 tl )", {'entities': [(0, 8, 'brand'), (24, 27, 'size')]}), 
("kate spade aurinkolasit kahli/s wr7/ha", {'entities': [(0, 10, 'brand'), (11, 23, 'product')]}), 
("lompakot le tanneur\xa0 martin", {'entities': [(0, 8, 'product'), (9, 19, 'brand')]}), 
("kengät creative recreation\xa0 cesario lo xvi", {'entities': [(0, 6, 'product'), (7, 26, 'brand')]}), 
("volume million lashes fatale mascara ripsiväri maskara meikki musta l'oréal paris", {'entities': [(29, 36, 'product'), (37, 46, 'product'), (47, 54, 'product'), (62, 67, 'color'), (68, 81, 'brand')]}), 
("elevated corporate cap accessories headwear caps sininen tommy hilfiger", {'entities': [(35, 48, 'product'), (49, 56, 'color'), (57, 71, 'brand')]}), 
("tiger of sweden jeans jet c housut black", {'entities': [(0, 15, 'brand'), (16, 21, 'product'), (28, 34, 'product'), (35, 40, 'color')]}), 
("ua classic slip on  tennarit", {'entities': [(20, 28, 'product')]}), 
("vredestein traxion versa ( 340/80 r18 143a8 tl )", {'entities': [(0, 10, 'brand'), (27, 45, 'size')]}), 
("lottecr plain twill   coco fit skinny farkut cream", {'entities': [(0, 7, 'brand'), (38, 44, 'product')]}), 
("lastenkengät le coq sportif  courtset gs", {'entities': [(0, 12, 'product'), (29, 40, 'brand')]}), 
("perfect cleanser beauty women skin care face cleansers milk cleanser nude oskia", {'entities': [(24, 29, 'gender'), (8, 16, 'product'), (40, 54, 'product'), (74, 79, 'brand')]}), 
("adidas originals aurinkolasit aom007 009.120", {'entities': [(0, 6, 'brand'), (17, 29, 'product')]}), 
("lyhythihainen poolopaita lacoste  ph5462 slim", {'entities': [(25, 32, 'brand'), (14, 24, 'product')]}), 
("colonia essenza, after shave balm 100ml", {'entities': [(17, 33, 'brand'), (34, 39, 'size'), (0, 15, 'brand')]}), 
("kaappi, valkoinen", {'entities': [(0, 6, 'product'), (8, 17, 'color')]}), 
("paita irene", {'entities': [(0, 5, 'product'), (6, 11, 'brand')]}), 
("kengät marc o'polo  catania", {'entities': [(7, 18, 'brand'), (0, 6, 'product')]}), 
("smith aurinkolasit lowdown steel 003/1c", {'entities': [(6, 18, 'product'), (0, 5, 'brand')]}), 
("kelis pu trousers", {'entities': [(9, 17, 'product'), (0, 5, 'brand'), (6, 8, 'material')]}), 
("supermicro x10drh ct o xeon5 c612 eatx vga 2x10gbe sas3 ddr4 ipmi reta in cpnt", {'entities': [(0, 10, 'brand')]}), 
("pusero jdynora life s/s anglaise top", {'entities': [(0, 6, 'product'), (20, 23, 'size'), (33, 36, 'product'), (7, 14, 'brand')]}), 
("lightly lined demi lingerie bras & tops padded bras kermanvärinen calvin klein", {'entities': [(28, 39, 'product'), (40, 51, 'product'), (52, 65, 'color'), (66, 78, 'brand')]}), 
("uvex aurinkolasit blaze iii 5320462210", {'entities': [(5, 17, 'product'), (0, 4, 'brand')]}), 
("vicky top t shirts & tops sleeveless valkoinen kari traa", {'entities': [(10, 25, 'product'), (37, 46, 'color'), (47, 56, 'brand')]}), 
("pusero elisa basic", {'entities': [(0, 6, 'product'), (7, 12, 'brand')]}), 
("smartbuy collection silmälasit caroline cp165d", {'entities': [(20, 30, 'product'), (0, 19, 'brand')]}), 
("colonia essenza, after shave lotion 100ml", {'entities': [(0, 15, 'brand'), (17, 35, 'product'), (36, 41, 'size')]}), 
("julbo aurinkolasit starwind j75431147", {'entities': [(6, 18, 'product'), (0, 5, 'brand')]}), 
("lastenkengät gola  quota ii", {'entities': [(0, 12, 'product'), (13, 17, 'brand')]}), 
("balmain aurinkolasit bl 2114 c03", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("korkokengät emma go  joelle", {'entities': [(0, 11, 'product'), (12, 19, 'brand')]}), 
("t shirt ss t shirts & tops short sleeved musta rosemunde", {'entities': [(0, 7, 'product'), (11, 26, 'product'), (41, 46, 'color'), (47, 56, 'brand')]}), 
("kengät new balance  373", {'entities': [(0, 6, 'product'), (7, 18, 'brand')]}), 
("guess aurinkolasit gu 7646 10g", {'entities': [(6, 18, 'product'), (0, 5, 'brand')]}), 
("paita juel 146", {'entities': [(0, 5, 'product'), (6, 10, 'brand'), (11, 14, 'size')]}), 
("fayy chelsea shoes chelsea boots ruskea gant", {'entities': [(13, 18, 'product'), (27, 32, 'product'), (33, 39, 'color'), (40, 44, 'brand')]}), 
("kengät moma  cusna nero", {'entities': [(0, 6, 'product'), (7, 11, 'brand'), (19, 23, 'color')]}), 
("dana buchman silmälasit vivian golden brown", {'entities': [(13, 23, 'product'), (31, 43, 'color'), (0, 12, 'brand'), (24, 30, 'brand')]}), 
("paita jdyrosalina l/s shirt wvn", {'entities': [(0, 5, 'product'), (18, 21, 'size'), (22, 27, 'product'), (6, 17, 'brand')]}), 
("paita hepburn iw50", {'entities': [(0, 5, 'product'), (6, 13, 'brand')]}), 
("zac posen aurinkolasit luciana black", {'entities': [(10, 22, 'product'), (31, 36, 'color'), (0, 9, 'brand')]}), 
("longchamp aurinkolasit lo606s 429", {'entities': [(10, 22, 'product'), (0, 9, 'brand')]}), 
("vogue eyewear silmälasit vo5053 ligh t& shine w44", {'entities': [(14, 24, 'product'), (0, 5, 'brand')]}), 
("sava trenta 2 ( 195/70 r15c 104/102r 8pr )", {'entities': [(0, 4, 'brand'), (16, 39, 'size')]}), 
("channi mekko molo", {'entities': [(7, 12, 'product'), (0, 6, 'brand'), (13, 17, 'brand')]}), 
("gucci aurinkolasit gg0535s 005", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("smartbuy collection aurinkolasit barrow street jst 43 m02", {'entities': [(0, 8, 'brand'), (20, 32, 'product')]}), 
("polaroid aurinkolasit p8302 polarized kih/jb", {'entities': [(9, 21, 'product'), (0, 8, 'brand')]}), 
("guess aurinkolasit gu 7643 10c", {'entities': [(6, 18, 'product'), (0, 5, 'brand')]}), 
("smartbuy collection silmälasit metropolitan avenue jsv 058 012", {'entities': [(0, 8, 'brand'), (20, 30, 'product')]}), 
("adidas originals aurinkolasit aom000 078.055", {'entities': [(17, 29, 'product'), (0, 6, 'brand')]}), 
("saint laurent silmälasit sl 306 00", {'entities': [(0, 13, 'brand'), (14, 24, 'product')]}), 
("stella mccartney silmälasit sc0218o 002", {'entities': [(17, 27, 'product'), (0, 16, 'brand')]}), 
("colonia quercia, edc 100ml", {'entities': [(21, 26, 'size'), (0, 15, 'brand')]}), 
("yokohama bluearth van ry55 ( 225/75 r16c 121/120r bluearth )", {'entities': [(0, 8, 'brand'), (29, 49, 'size')]}), 
("silk top regular w/rev vintage lace t shirts & tops sleeveless vihreä rosemunde", {'entities': [(0, 4, 'material'), (36, 51, 'product'), (63, 69, 'color'), (70, 79, 'brand')]}), 
("pusero carblaire life ss teg tee", {'entities': [(0, 6, 'product'), (22, 24, 'size'), (7, 16, 'brand')]}), 
("chloe aurinkolasit ce 132s 261", {'entities': [(6, 18, 'product'), (0, 5, 'brand')]}), 
("adidas originals   superstar   valkoinen   us 5,5", {'entities': [(31, 40, 'color'), (43, 49, 'size'), (0, 6, 'brand')]}), 
("sc erna polvipituinen hame sininen soyaconcept", {'entities': [(27, 34, 'color'), (22, 26, 'product'), (35, 46, 'brand')]}), 
("dana buchman silmälasit taren caramel tortoise", {'entities': [(13, 23, 'product'), (0, 12, 'brand'), (30, 46, 'color')]}), 
("active sports woven shorts b shortsit musta puma", {'entities': [(20, 26, 'product'), (29, 37, 'product'), (38, 43, 'color'), (44, 48, 'brand')]}), 
("renata polvipituinen\xa0hame\xa0musta\xa0fall winter spring summer", {'entities': [(32, 57, 'brand'), (21, 25, 'product'), (26, 31, 'color')]}), 
("\n  salo m technical\n  polo polos short sleeved sininen halti", {'entities': [(47, 54, 'color'), (22, 26, 'product'), (3, 7, 'brand')]}), 
("tennarit ml850cd", {'entities': [(0, 8, 'product')]}), 
("g9 orange shiny", {'entities': [(3, 9, 'color')]}), 
("diesel aurinkolasit dl0076 90g", {'entities': [(0, 6, 'brand'), (7, 19, 'product')]}), 
("dotti tyyny 50x50cm, mix", {'entities': [(6, 11, 'product'), (12, 19, 'size')]}), 
("kengät jonak\xa0 didlaneo", {'entities': [(0, 6, 'product'), (7, 12, 'brand')]}), 
("lyhyt mekko see u soon\xa0 santine", {'entities': [(6, 11, 'product'), (12, 22, 'brand')]}), 
("dsquared2 silmälasit dq5126 072", {'entities': [(10, 20, 'product'), (0, 9, 'brand')]}), 
("stussy   basic tee   musta   m", {'entities': [(0, 6, 'brand'), (15, 18, 'product'), (21, 26, 'color'), (29, 30, 'size')]}), 
("kengät grisport\xa0 nero dakar 10309d109g", {'entities': [(0, 6, 'product'), (7, 15, 'brand')]}), 
("vredestein aw ( 15.0/55  17 134a8 tl )", {'entities': [(0, 10, 'brand'), (16, 35, 'size')]}), 
("ryders aurinkolasit trapper r02502a", {'entities': [(7, 19, 'product'), (0, 6, 'brand')]}), 
("farah silmälasit fho 1011 113", {'entities': [(6, 16, 'product'), (0, 5, 'brand')]}), 
("f 7301 korolliset sandaalit ruskea wonders", {'entities': [(28, 34, 'color'), (18, 27, 'product'), (35, 42, 'brand')]}), 
("g4 pro white mirror", {'entities': [(7, 12, 'color'), (13, 19, 'product'), (0, 2, 'brand')]}), 
("tennarit\xa0ctalymsi", {'entities': [(0, 8, 'product')]}), 
("sukat\xa0bamboo\xa0socks,\xa05\xa0paria", {'entities': [(0, 5, 'brand'), (13, 18, 'product'), (6, 12, 'material')]}), 
("cazal\xa0silmälasit\xa04267\xa0004", {'entities': [(0, 5, 'brand'), (6, 16, 'product')]}), 
("13294\xa0bags\xa0small\xa0shoulder\xa0bags\xa0 \xa0crossbody\xa0bags\xa0kermanvärinen\xa0depeche", {'entities': [(11, 16, 'size'), (48, 61, 'color'), (62, 69, 'brand')]}), 
("madina\xa0w\xa0tee\xa0t shirts\xa0&\xa0tops\xa0short sleeved\xa0musta\xa08848\xa0altitude", {'entities': [(0, 6, 'brand'), (43, 48, 'color'), (9, 12, 'product'), (13, 21, 'product'), (24, 28, 'product')]}), 
("superdry\xa0aurinkolasit\xa0sds\xa0yatomi\xa0006", {'entities': [(0, 8, 'brand'), (9, 21, 'product')]}), 
("burberry\xa0aurinkolasit\xa0be4181\xa0300187", {'entities': [(0, 8, 'brand'), (9, 21, 'product')]}), 
("ray\xa0kattovalaisin,\xa0large,\xa0valkoinen", {'entities': [(0, 17, 'brand'), (19, 24, 'size'), (26, 35, 'color')]}), 
("paradigm aurinkolasit 19 30 rose gold", {'entities': [(9, 21, 'product'), (0, 8, 'brand'), (28, 37, 'color')]}), 
("general tractor v.ply ( 20.8  38 151a6 10pr tt )", {'entities': [(0, 7, 'brand'), (24, 45, 'size')]}), 
("sola dress polvipituinen mekko kermanvärinen\xa0résumé", {'entities': [(5, 10, 'product'), (25, 30, 'product'), (31, 44, 'color'), (45, 51, 'brand')]}), 
("helix teekannu ruostumaton teräs 1 l", {'entities': [(6, 14, 'product'), (33, 36, 'size'), (15, 32, 'material')]}), 
("false lash effect velvet volume mascara, black", {'entities': [(32, 39, 'product'), (41, 46, 'color')]}), 
("false lash effect velvet volume mascara, black/br", {'entities': [(32, 39, 'product'), (41, 49, 'color')]}), 
("rok merino gloves musta l/xl", {'entities': [(11, 17, 'product'), (18, 23, 'color'), (24, 28, 'size'), (4, 10, 'brand')]}), 
("diesel silmälasit dl5067 001", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("small eye shadow beauty women makeup eyes eyeshadow   not palettes ruskea m.a.c.", {'entities': [(0, 5, 'size'), (74, 80, 'brand'), (24, 29, 'gender'), (6, 16, 'product'), (42, 51, 'product')]}), 
("kengät victoria\xa0 berlin piel", {'entities': [(0, 6, 'product'), (7, 15, 'brand')]}), 
("capsule wool tights sukkahousut valkoinen mp denmark", {'entities': [(32, 41, 'color'), (13, 19, 'product'), (20, 31, 'product'), (8, 12, 'material'), (42, 52, 'brand')]}), 
("norden 2 osainen veitsisetti", {'entities': [(17, 28, 'product')]}), 
("marshmallow hearts bath & shower gel 500 ml", {'entities': [(19, 36, 'product'), (37, 43, 'size')]}), 
("rodebjer nevaeh leveälahkeiset housut ruskea rodebjer", {'entities': [(38, 44, 'color'), (31, 37, 'product'), (45, 53, 'brand'), (0, 8, 'brand')]}), 
("wet n wild angled liner brush", {'entities': [(11, 29, 'product'), (0, 10, 'brand')]}), 
("stella, shw, brief bikinialaosa sininen zizzi", {'entities': [(32, 39, 'color'), (19, 31, 'product'), (40, 45, 'brand')]}), 
("mom jean jeans mom jeans sininen calvin klein jeans", {'entities': [(33, 45, 'brand'), (25, 32, 'color'), (9, 14, 'product'), (15, 24, 'product')]}), 
("g15 black mat", {'entities': [(4, 9, 'color')]}), 
("colby sneaker matalavartiset sneakerit tennarit musta michael kors shoes", {'entities': [(39, 47, 'product'), (29, 38, 'product'), (48, 53, 'color'), (54, 66, 'brand'), (67, 72, 'product')]}), 
("d bazer farkut sininen diesel men", {'entities': [(23, 29, 'brand'), (30, 33, 'gender'), (15, 22, 'color'), (8, 14, 'product')]}), 
("lecco 80 t shirts short sleeved valkoinen boss", {'entities': [(32, 41, 'color'), (9, 17, 'product'), (42, 46, 'brand')]}), 
("senoa style sähkögrilli jalustalla", {'entities': [(12, 23, 'product')]}), 
("tommy girl neon brights, edt 50ml", {'entities': [(0, 10, 'brand'), (29, 33, 'size')]}), 
("mini foodprocessor 0,95 litraa punainen", {'entities': [(5, 18, 'product'), (19, 30, 'size'), (31, 39, 'color')]}), 
("sp700+ maidonvaahdotin, jossa 13 ohjelmaa", {'entities': []}), 
("aromatics elixir, perfume spray 100ml", {'entities': [(0, 16, 'brand'), (18, 31, 'product'), (32, 37, 'size')]}), 
("ozka agro 10 traction ( 480/70 r30 141a8 tl )", {'entities': [(0, 4, 'brand'), (24, 43, 'size')]}), 
("g3 smaragd", {'entities': [(0, 10, 'brand')]}), 
("clara velvet skirt polvipituinen hame oranssi just female", {'entities': [(38, 45, 'color'), (13, 18, 'product'), (33, 37, 'product'), (19, 32, 'size'), (46, 57, 'brand'), (6, 12, 'material')]}), 
("leivänpaahdin cube, 2 viipaleelle", {'entities': [(20, 33, 'size'), (0, 18, 'brand')]}), 
("bkt tr 135 ( 12.4  36 127a6 8pr tt )", {'entities': [(0, 3, 'brand'), (13, 33, 'size')]}), 
("g2 cuba libre", {'entities': []}), 
("aromatics elixir, perfume spray 10ml", {'entities': [(0, 16, 'brand'), (18, 31, 'product'), (32, 36, 'size')]}), 
("korkokengät etro  3074\xa0", {'entities': [(0, 11, 'product'), (12, 16, 'brand')]}), 
("sandaalit pikolinos  algar w0x", {'entities': [(0, 9, 'product'), (10, 19, 'brand')]}), 
("kengät les tropéziennes par m belarbi  amelie", {'entities': [(0, 6, 'product'), (7, 23, 'brand')]}), 
("kengät everlast  evl shadow mid", {'entities': [(0, 6, 'product'), (7, 15, 'brand')]}), 
("farah aurinkolasit fhs 5015 001", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("ceat farmax r85 ( 380/85 r34 137a8 tl kaksoistunnus  137b )", {'entities': [(0, 4, 'brand'), (18, 36, 'size')]}), 
("diesel silmälasit dl5050 b05", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("gi2 dejavu angel violet", {'entities': [(0, 16, 'brand'), (17, 23, 'color')]}), 
("nonogarden t shirts & tops short sleeved sininen american vintage", {'entities': [(41, 48, 'color'), (11, 26, 'product'), (49, 65, 'brand')]}), 
("crewneck tshirt t shirts short sleeved valkoinen champion", {'entities': [(40, 49, 'color'), (9, 16, 'product'), (17, 25, 'product'), (50, 58, 'brand')]}), 
("classic chino suoralahkeiset housut vaaleanpunainen gant", {'entities': [(52, 56, 'brand'), (36, 51, 'color'), (29, 35, 'product')]}), 
("all about shadow single, purple pumps beauty women makeup eyes eyeshadow   not palettes liila clinique", {'entities': [(94, 102, 'brand'), (45, 50, 'gender'), (63, 72, 'product'), (88, 93, 'color')]}), 
("g12 titan polarized", {'entities': [(0, 9, 'brand')]}), 
("mitas ts06 ( 6.50/80  15 94a8 6pr tt kaksoistunnus  82a8 )", {'entities': [(0, 5, 'brand'), (13, 35, 'size')]}), 
("scoop teepurkki mittalusikalla", {'entities': [(6, 15, 'product'), (0, 5, 'brand')]}), 
("lyhythihainen t paita adidas  trefoil t shirt", {'entities': [(14, 21, 'product'), (38, 45, 'product'), (22, 28, 'brand')]}), 
("wool slipper infant aamutossut sisäkengät harmaa hummel", {'entities': [(0, 4, 'material'), (5, 12, 'product'), (20, 30, 'product'), (31, 41, 'product'), (42, 48, 'color'), (49, 55, 'brand')]}), 
("olkalaukut desigual  mex kiev mini", {'entities': [(11, 19, 'brand'), (0, 10, 'product')]}), 
("mitas fl08 ( 18x7.00  8 121a5 14pr tt nhs )", {'entities': [(0, 5, 'brand')]}), 
("mid rise true skinny jeans skinny farkut sininen gap", {'entities': [(41, 48, 'color'), (49, 52, 'brand'), (27, 40, 'product'), (14, 26, 'product')]}), 
("metsästäjän sukat 37 39", {'entities': [(12, 17, 'product'), (18, 23, 'size'), (0, 11, 'brand')]}), 
("aromatics elixir, perfume spray 25ml", {'entities': [(0, 16, 'brand'), (18, 31, 'product'), (32, 36, 'size')]}), 
("bisgaard jacob tennarit sneakerit kengät vihreä bisgaard", {'entities': [(15, 23, 'product'), (34, 40, 'product'), (41, 47, 'color'), (0, 8, 'brand')]}), 
("bvlgari aurinkolasit bv5047q 202273", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("le klint 6 varjostin 21 cm, muovi", {'entities': [(1, 9, 'brand'), (22, 27, 'size'), (29, 34, 'material')]}), 
("general tractor v.ply ( 14.9  28 137a6 12pr tt )", {'entities': [(0, 7, 'brand'), (24, 45, 'size')]}), 
("diesel silmälasit dl5045 090", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("clubman classic vanilla after shave 177 ml", {'entities': [(36, 42, 'size'), (24, 35, 'product'), (0, 7, 'brand')]}), 
("cordoba rug 160x230 cm, ivory", {'entities': [(8, 11, 'product'), (12, 22, 'size'), (0, 7, 'brand'), (24, 29, 'color')]}), 
("refine tyynynpäällinen raid", {'entities': [(7, 22, 'product')]}), 
("bkt maglift lip ( 15x4.50  8 109a5 tl )", {'entities': [(0, 3, 'brand'), (18, 36, 'size')]}), 
("lyhythihainen t paita gant  le mans block", {'entities': [(14, 21, 'product'), (22, 26, 'brand'), (31, 35, 'gender')]}), 
("day paris bag bags small shoulder bags   crossbody bags musta day et", {'entities': [(56, 61, 'color'), (0, 9, 'brand'), (10, 13, 'product'), (14, 18, 'product')]}), 
("komfort xxl kolgrillsvagn", {'entities': [(0, 25, 'brand')]}), 
("jane top t shirts & tops short sleeved valkoinen ida sjöstedt", {'entities': [(9, 17, 'product'), (20, 24, 'product'), (39, 48, 'color'), (49, 61, 'brand')]}), 
("classic veitsitukki setti 7 osaa", {'entities': [(0, 7, 'brand'), (8, 19, 'product')]}), 
("aromatics elixir, perfume spray 45ml", {'entities': [(0, 16, 'brand'), (18, 31, 'product'), (32, 36, 'size')]}), 
("puma silmälasit pj0029o kids 001", {'entities': [(5, 15, 'product'), (0, 4, 'brand')]}), 
("g12 black shiny polarized", {'entities': [(0, 3, 'brand'), (4, 9, 'color')]}), 
("suoja box kaasugrilliin", {'entities': [(0, 9, 'product')]}), 
("verpan fun 1dm riippuvalaisin", {'entities': [(0, 10, 'brand'), (15, 29, 'product'), (11, 14, 'size')]}), 
("lyhyt mekko betty london\xa0 lady", {'entities': [(6, 11, 'product'), (11, 30, 'brand')]}), 
("coat rack naulakko, musta/rauta", {'entities': [(0, 18, 'product'), (20, 25, 'color'), (26, 31, 'material')]}), 
("chelsea pöytävalaisin 31x48cm, musta", {'entities': [(0, 7, 'brand'), (8, 21, 'product'), (22, 29, 'size'), (31, 36, 'color')]}), 
("reppu vans \xa0 wm realm plus backpa", {'entities': [(6, 10, 'brand'), (30, 36, 'product')]}), 
("lash sensational waterproof mascara, black", {'entities': [(0, 16, 'brand'), (28, 35, 'product'), (37, 42, 'color')]}), 
("tilda dress polvipituinen mekko sininen boob", {'entities': [(40, 43, 'brand'), (26, 31, 'product'), (6, 11, 'product'), (32, 39, 'color')]}), 
("origami pöytävalaisin, valkoinen", {'entities': [(0, 7, 'brand'), (8, 21, 'product'), (23, 32, 'color')]}), 
("absolute woman, edt 40ml", {'entities': [(0, 14, 'brand'), (20, 24, 'size')]}), 
("moss kynttilänjalka 8.5 cm blue mirage", {'entities': [(0, 4, 'brand'), (5, 19, 'product'), (20, 26, 'size'), (27, 38, 'color')]}), 
("le parfum, deospray 100ml", {'entities': [(0, 9, 'brand'), (10, 19, 'product'), (20, 25, 'size')]}), 
("vredestein flotation + ( 400/70  20 150a8 tl )", {'entities': [(0, 10, 'brand'), (25, 43, 'size')]}), 
("vg10 leipäveitsi 20cm", {'entities': [(5, 16, 'product'), (17, 21, 'size')]}), 
("mitas td 19 ( 12.4  24 120a6 8pr tt kaksoistun...", {'entities': [(0, 5, 'brand'), (14, 34, 'size')]}), 
("diesel silmälasit dl5050 050", {'entities': [(0, 6, 'brand'), (6, 17, 'product'), (17, 28, 'size')]}), 
("kate spade aurinkolasit avaline2/s ht8/0j", {'entities': [(0, 10, 'brand'), (11, 23, 'product')]}), 
("ozka agro 10 traction ( 280/85 r24 115a8 tl ka...", {'entities': [(0, 4, 'brand'), (24, 43, 'size')]}), 
("gloryfy gi15 st pauli havanna shiny", {'entities': [(0, 13, 'brand')]}), 
("select warm shortsit w/lycra musta", {'entities': [(0, 11, 'brand'), (28, 34, 'color')]}), 
("retrosuperfuture aurinkolasit drew mama panna ...", {'entities': [(0, 16, 'brand'), (17, 29, 'product')]}), 
("superdry silmälasit sdo yumi 104", {'entities': [(0, 8, 'brand'), (9, 19, 'product')]}), 
("sophie aurora dress maksimekko juhlamekko sininen bruuns bazaar", {'entities': [(50, 63, 'brand'), (14, 19, 'product'), (20, 30, 'product'), (42, 49, 'color')]}), 
("norr leipäveitsi 21 cm", {'entities': [(0, 4, 'brand'), (4, 16, 'product'), (17, 22, 'size')]}), 
("essential for her, edt 20ml", {'entities': [(0, 17, 'brand'), (23, 27, 'size')]}), 
("So & Co New York Madison Naisten kello 5220.3", {'entities': [(0, 16, 'brand'), (25, 32, 'gender'), (33, 38, 'product')]}),
("Globen Lighting Cube Pöytävalaisin, Musta", {'entities': [(20, 34, 'product'), (36, 41, 'color'), (0, 6, 'brand')]}), 
("Long Boots 4835 Korkeavartiset Saapikkaat Musta Billi Bi", {'entities': [(42, 47, 'color'), (48, 56, 'brand'), (31, 41, 'product'), (5, 10, 'product')]}),
("Invicta Bolt Naisten kello 28936 Harmaa/Kullansävytetty teräs", {'entities': [(0, 12, 'brand'), (13, 20, 'gender'), (21, 26, 'product'), (33, 55, 'color'), (56, 61, 'material')]}),
("BigSize Me Stringit", {'entities': [(0, 10, 'brand'), (11, 19, 'product')]}), 
("INTEL Xeon Scalable 8256 3.80GHZ FC LGA3647 16.5M Cache 10.4GT/sec Box CPU", {'entities': [(0, 5, 'brand'), (25, 32, 'size'), (44, 49, 'size'), (67, 74, 'product')]}),
("essential for her, edt 40ml", {'entities': [(0, 17, 'brand'), (23, 27, 'size'), (18, 22, 'product')]}), 
("ff veitsitukki 5 veistä valkoinen", {'entities': [(0, 2, 'brand'), (3, 14, 'product'), (17, 23, 'product'), (24, 33, 'color')]}), 
("gi21 dante red", {'entities': [(11, 14, 'color')]}), 
("lyhythihainen t paita botd \xa0 estoila", {'entities': [(14, 21, 'product'), (22, 25, 'brand')]}), 
("kengät big star\xa0 shoes big top ff274343", {'entities': [(0, 6, 'product'), (6, 15, 'brand'), (17, 22, 'product')]}), 
("starmaxx tr 110 ( 340/85 r38 133a8 tl kaksoistunnus  130b", {'entities': [(0, 8, 'brand'), (17, 36, 'size')]}), 
("dana buchman silmälasit poppi black", {'entities': [(0, 12, 'brand'), (13, 23, 'product'), (30, 35, 'color')]}), 
("roberto cavalli silmälasit rc 5052 foiano 016", {'entities': [(0, 15, 'brand'), (16, 26, 'product')]}), 
("roberto cavalli silmälasit rc 5053 follonica a56", {'entities': [(0, 15, 'brand'), (16, 27, 'product')]}), 
("femme, edp 100ml", {'entities': [(0, 5, 'brand'), (7, 10, 'product'), (11, 16, 'size')]}), 
("paradise bay, edp 85ml", {'entities': [(0, 12, 'brand'), (14, 17, 'product'), (18, 22, 'size')]}), 
("rannekoru a1859 leather bracelet black", {'entities': [(0, 9, 'brand'), (15, 23, 'material'), (23, 32, 'product'), (32, 38, 'color')]}), 
("striped flower pussilakana 220x220cm, lt aqua", {'entities': [(0, 14, 'brand'), (15, 26, 'product'), (27, 36, 'size'), (38, 45, 'color')]}), 
("funktion 4 matala kattila ja kansi 140 cl", {'entities': [(0, 8, 'brand'), (17, 25, 'product'), (29, 35, 'product'), (35, 41, 'size')]}), 
("mosakb tie dress polvipituinen mekko monivärinen/kuvioitu karen by simonsen", {'entities': [(0, 10, 'brand'), (11, 16, 'product'), (31, 36, 'product'), (37, 48, 'color')]}), 
("bitz kulho ø 14 cm musta musta meripihka", {'entities': [(0, 10, 'brand'), (13, 18, 'size'), (19, 24, 'color'), (24, 30, 'color'), (31, 40, 'product')]}), 
("cultor rd 01 ( 320/85 r32 142a8 tl kaksoistunnus  142b", {'entities': [(0, 6, 'brand'), (14, 33, 'size')]}), 
("dbkd oblong ruukku/maljakko small, mole", {'entities': [(0, 11, 'brand'), (12, 18, 'product'), (19, 27, 'product'), (27, 33, 'size')]}), 
("poikien sandaalit geox\xa0 jr sandal strada", {'entities': [(0, 7, 'gender'), (8, 17, 'product'), (18, 26, 'brand'), (27, 33, 'product')]}), 
("moncler aurinkolasit ml0073 01a", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("police aurinkolasit spl997 origins lite 3 0722", {'entities': [(0, 6, 'brand'), (7, 19, 'product')]}), 
("gap logo joggers collegehousut olohousut harmaa gap", {'entities': [(0, 3, 'brand'), (31, 40, 'product'), (41, 47, 'color')]}), 
("umbra dress maksimekko juhlamekko vaaleanpunainen résumé", {'entities': [(0, 5, 'brand'), (6, 11, 'product'), (17, 22, 'product'), (34, 49, 'color')]}), 
("harvey jacket outerwear jackets anoraks sininen lexington clothing", {'entities': [(0, 6, 'brand'), (6, 13, 'product'), (24, 31, 'product'), (40, 47, 'color'), (48, 66, 'brand')]}), 
("classic veitsitukki setti 9 osaa musta", {'entities': [(0, 7, 'brand'), (8, 19, 'product'), (26, 32, 'size'), (33, 38, 'color')]}), 
("kokkiveitsi red", {'entities': [(0, 11, 'product'), (12, 15, 'color')]}), 
("balleriinat ikks\xa0 anabel", {'entities': [(12, 16, 'brand'), (0, 11, 'product')]}), 
("bleiseri le temps des cerises\xa0 mimo", {'entities': [(9, 29, 'brand'), (0, 8, 'product')]}), 
("tiffany & co. silmälasit tf2175 8134", {'entities': [(0, 13, 'brand'), (14, 24, 'product')]}), 
("cazal silmälasit 4268 003", {'entities': [(0, 5, 'brand'), (6, 16, 'product')]}), 
("timberland silmälasit tb1652 002", {'entities': [(0, 10, 'brand'), (11, 21, 'product')]}), 
("coolmax saumaton musta/pinkki 34   36", {'entities': [(0, 7, 'brand'), (8, 16, 'product'), (17, 29, 'color'), (30, 37, 'size')]}), 
("urban mini talvikengät tarroilla musta ecco", {'entities': [(39, 43, 'brand'), (33, 38, 'color'), (11, 22, 'product'), (6, 10, 'size')]}), 
("stockholm vase 22,5 cm, lignum", {'entities': [(0, 9, 'brand'), (24, 30, 'product'), (10, 14, 'product'), (15, 23, 'size')]}), 
("idun hanskat käsineet ruskea hestra", {'entities': [(0, 4, 'brand'), (13, 21, 'product'), (22, 28, 'color')]}), 
("beauty, shower gel 200ml", {'entities': [(0, 6, 'brand'), (8, 18, 'product'), (19, 24, 'size')]}), 
("lyhythihainen t paita vans\xa0 flying v crew tee", {'entities': [(22, 26, 'brand'), (0, 13, 'product'), (14, 21, 'product')]}), 
("brow defining kit 1 set", {'entities': []}), 
("extreme length sealer 50ml", {'entities': [(0, 21, 'product'), (22, 26, 'size')]}), 
("prada aurinkolasit pr53vs 1ab1i0", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("lyhythihainen t paita desigual\xa0 under", {'entities': [(0, 13, 'product'), (14, 21, 'product'), (32, 37, 'brand')]}), 
("police silmälasit vk079 lapis jr 2 kids 06qs", {'entities': [(0, 6, 'brand'), (7, 17, 'product'), (30, 34, 'size')]}), 
("coolmax musta 3436", {'entities': [(0, 7, 'product'), (8, 13, 'color')]}), 
("extreme length primer 150ml", {'entities': [(15, 21, 'product'), (0, 14, 'size'), (22, 27, 'size')]}), 
("frizz dismiss conditioner 250ml", {'entities': [(0, 25, 'product'), (26, 31, 'size')]}), 
("lean back loungestol oak, rusty rose velvet", {'entities': [(26, 43, 'color'), (0, 24, 'product')]}), 
("sweater forest aop dots svetari collegepaita\xa0beige lindex", {'entities': [(51, 57, 'brand'), (0, 23, 'product'), (24, 31, 'product'), (45, 50, 'color')]}), 
("frizz dismiss conditioner 500ml", {'entities': [(0, 25, 'product'), (26, 31, 'size')]}), 
("wella sp balance scalp shampoo 250 ml", {'entities': [(0, 5, 'brand'), (6, 30, 'product'), (31, 37, 'size')]}), 
("vogue eyewear silmälasit vo5224 2636", {'entities': [(0, 5, 'brand'), (6, 24, 'product')]}), 
("adi core 1/4 z svetari collegepaita musta adidas golf", {'entities': [(36, 41, 'color'), (15, 22, 'product'), (23, 35, 'product'), (42, 52, 'brand')]}), 
("korkokengät irregular choice \xa0 marma ladies", {'entities': [(12, 27, 'brand'), (0, 11, 'product')]}), 
("emilio pucci aurinkolasit ep0138 01a", {'entities': [(0, 12, 'brand'), (13, 25, 'product')]}), 
("biolage smoothproof serum 89ml", {'entities': [(0, 25, 'product'), (26, 30, 'size')]}), 
("cugy lyhyt hame vaaleanpunainen vanessa bruno", {'entities': [(32, 45, 'brand'), (16, 31, 'color'), (11, 15, 'product')]}), 
("roar skirt polvipituinen hame musta lollys laundry", {'entities': [(35, 50, 'brand'), (30, 34, 'color'), (11, 29, 'product'), (0, 10, 'product')]}), 
("zonnebril alensa retro black", {'entities': [(10, 16, 'brand'), (0, 9, 'product'), (17, 28, 'color')]}), 
("traverse shorts cumin l", {'entities': [(0, 15, 'product'), (16, 21, 'color'), (22, 23, 'size')]}), 
("flairville low laceshoes shoes business laced shoes ruskea gant", {'entities': [(59, 63, 'brand'), (0, 24, 'product'), (31, 51, 'product'), (52, 58, 'color')]}), 
("cartier panthere de cartier naisten kello hpi01296 valkoinen/nahka", {'entities': [(0, 7, 'brand'), (51, 66, 'color'), (8, 27, 'product'), (28, 35, 'gender')]}), 
("starmaxx tr 40 ( 6.00  19 93a6 6pr tt )", {'entities': [(0, 8, 'brand'), (17, 36, 'size')]}), 
("bkt tr 135 ( 12.4  24 121a6 8pr tt )", {'entities': [(0, 3, 'brand'), (13, 33, 'size')]}), 
("max mara mm 1410 wr9", {'entities': [(0, 11, 'brand')]}), 
("kengät redskins\xa0 malvino", {'entities': [(7, 15, 'brand'), (0, 6, 'product')]}), 
("kengät nike\xa0 air vapormax flyknit 3 aj6900 004", {'entities': [(0, 6, 'product'), (7, 11, 'brand'), (13, 35, 'product')]}), 
("persol aurinkolasit po2455s 107471", {'entities': [(0, 6, 'brand'), (7, 19, 'product')]}), 
("janesca nahkatakki musta boss casual wear", {'entities': [(19, 24, 'color'), (30, 41, 'product'), (25, 29, 'brand'), (0, 18, 'product')]}), 
("treeline glove tan l", {'entities': [(0, 18, 'product'), (19, 20, 'size')]}), 
("pullover long sleeve kilpikonnakaulus poolopaita punainen taifun", {'entities': [(58, 64, 'brand'), (49, 57, 'color'), (38, 48, 'product'), (21, 37, 'product'), (0, 20, 'product')]}), 
("m. cotton merino basic sweater neulepaita pyöreä kaula aukko sininen filippa k", {'entities': [(69, 78, 'brand'), (0, 30, 'product'), (31, 41, 'product'), (42, 60, 'product'), (61, 68, 'color')]}), 
("malhotra mrl mtf 222 ( 4.50 16 73a6 4pr tt kaksoistunnus  65a8", {'entities': [(0, 8, 'brand'), (28, 30, 'size')]}), 
("malhotra mrl ml2 455 ( 10  16.5 134a2 10pr tl kaksoistunnus  126a3 )", {'entities': [(0, 12, 'brand'), (23, 44, 'size')]}), 
("bobo regular c nk top ss t shirts & tops short sleeved sininen tommy hilfiger", {'entities': [(63, 77, 'brand'), (55, 62, 'color'), (36, 55, 'product'), (0, 33, 'product')]}), 
("lr gilmore t shirts & tops long sleeved monivärinen/kuvioitu levete room", {'entities': [(0, 10, 'brand'), (11, 39, 'product'), (40, 60, 'color')]}), 
("cultor as front 08 ( 6.50 16 91a6 8pr tt kaksoistunnus  83a8 )", {'entities': [(0, 6, 'brand'), (26, 28, 'size')]}), 
("apple cto cto:z0xz mbp 16in tb 2.4ghz i9 16gb 1tb 8gb space grey se btop", {'entities': [(0, 5, 'brand'), (19, 22, 'product'), (60, 64, 'color'), (23, 53, 'size')]}), 
("pusero juni", {'entities': [(0, 6, 'product'), (7, 11, 'brand')]}), 
("pressed mineral foundation, 8g, tawnee", {'entities': [(0, 26, 'brand'), (28, 30, 'size'), (32, 38, 'color')]}), 
("pressed mineral foundation, 8g, warm beige", {'entities': [(0, 26, 'brand'), (28, 30, 'size'), (37, 42, 'color')]}), 
("kids cascade ruffle t shirt t shirts sleeveless keltainen gap", {'entities': [(20, 27, 'product'), (28, 36, 'product'), (58, 61, 'brand'), (48, 57, 'color')]}), 
("cast crepe chukka nyörisaappaat musta royal republiq", {'entities': [(18, 31, 'product'), (38, 52, 'brand'), (32, 37, 'color')]}), 
("guess aurinkolasit gu 7627 01b", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("babblarna hakku puuta", {'entities': [(16, 21, 'material'), (0, 9, 'brand')]}), 
("mineral radiance creme powder foundation, 7g, barely beige", {'entities': [(42, 44, 'size'), (53, 58, 'color'), (23, 40, 'product')]}), 
("only onlmaylee mada faux leather dress p", {'entities': [(33, 38, 'product'), (25, 32, 'material'), (5, 14, 'brand')]}), 
("medium lr boot jeans leveälahkeiset farkut sininen hollister", {'entities': [(36, 42, 'product'), (43, 50, 'color'), (15, 20, 'product'), (10, 14, 'product'), (0, 6, 'size'), (51, 60, 'brand')]}), 
("Orange & Bergamot Three Wick Candle Tuoksukynttilä Nude Molton Brown", {'entities': [(36, 50, 'product'), (29, 35, 'product'), (56, 68, 'brand'), (0, 17, 'material')]}), 
("Bluetooth kaiutin Nikkei Submarine", {'entities': [(10, 17, 'product'), (18, 34, 'brand')]}), 
("UA HG Armour Racer Tank, White", {'entities': [(3, 12, 'brand'), (25, 30, 'color')]}),
("Julie Polvipituinen Hame Punainen Baum Und Pferdgarten", {'entities': [(20, 24, 'product'), (25, 33, 'color'), (34, 54, 'brand')]}),
("Emerald Charm Chain Choker", {'entities': [(20, 26, 'product'), (0, 13, 'brand')]}),
("ArchitectMade Gemini Kynttilänjalka, Ruostumaton teräs", {'entities': [(21, 35, 'product'), (0, 20, 'brand'), (49, 54, 'material')]}),
("Cisco Catalyst 3650 48 Port Mgig 2X40G Uplink Lan Base In Cpnt", {'entities': [(0, 5, 'brand'), (23, 27, 'product'), (33, 38, 'size')]}),
("Blouses Woven Pitkähihainen Pusero Paita Keltainen Esprit Collection", {'entities': [(51, 57, 'brand'), (35, 40, 'product'), (28, 34, 'product'), (41, 50, 'color')]}), 
("mineral radiance creme powder foundation, 7g, coffee", {'entities': [(23, 40, 'product'), (42, 44, 'size'), (45, 52, 'color')]}), 
("mineral radiance creme powder foundation, 7g, rose beige", {'entities': [(23, 40, 'product'), (42, 44, 'size'), (46, 56, 'color')]}), 
("m 4201 shoes heels pumps classic musta wonders", {'entities': [(7, 12, 'product'), (33, 38, 'color'), (13, 24, 'brand')]}), 
("cn ss tee logo flag t shirts short sleeved sininen tommy hilfiger", {'entities': [(51, 65, 'brand'), (42, 50, 'color'), (20, 28, 'product')]}), 
("asics   gel nimbus 22 platinum     us 8", {'entities': [(0, 30, 'brand'), (35, 39, 'size')]}), 
("neubau silmälasit t069 mike 6000", {'entities': [(0, 6, 'brand'), (7, 18, 'product')]}), 
("chloe aurinkolasit ce 139s 805", {'entities': [(0, 6, 'brand'), (6, 18, 'product')]}), 
("kengät timberland\xa0 london square 6in boot", {'entities': [(33, 36, 'size'), (37, 41, 'product'), (7, 32, 'brand'), (0, 6, 'product')]}), 
("paita bonnie", {'entities': [(0, 5, 'product'), (6, 12, 'brand')]}), 
("tammy shirt lyhythihainen paita beige filippa k", {'entities': [(6, 11, 'product'), (38, 47, 'brand'), (32, 37, 'color')]}), 
("sukat, 5 paria", {'entities': [(0, 5, 'product')]}), 
("spy aurinkolasit toddy 6700000000055", {'entities': [(0, 3, 'brand'), (4, 16, 'product')]}), 
("apple cto cto:z0y3 mbp 16in tb 2.4ghz i9 64gb 8tb 8gb silver se btop", {'entities': [(0, 5, 'brand'), (19, 22, 'product'), (23, 53, 'size'), (54, 60, 'color')]}), 
("carolina herrera silmälasit vhe734k 752y", {'entities': [(0, 16, 'brand'), (17, 27, 'product')]}), 
("t shirt rn t shirts short sleeved musta boss", {'entities': [(0, 7, 'product'), (11, 19, 'product'), (34, 39, 'color'), (40, 44, 'brand')]}), 
("porsche design aurinkolasit p8600 c/v713", {'entities': [(0, 7, 'brand'), (15, 27, 'product')]}), 
("nokian hakkapeliitta 8 suv ( 255/45 r20 105t xl nastarengas\xa0 )", {'entities': [(0, 6, 'brand'), (29, 46, 'size'), (53, 59, 'product')]}), 
("grant stretch twill pants puvun housut suorat housut sininen j. lindeberg", {'entities': [(61, 73, 'brand'), (53, 60, 'color'), (32, 38, 'product'), (20, 25, 'product')]}), 
("cotton front logo t shirt t shirts short sleeved sininen calvin klein", {'entities': [(18, 25, 'product'), (26, 34, 'product'), (49, 56, 'color'), (57, 69, 'brand'), (0, 6, 'material')]}), 
("pusero juni", {'entities': [(0, 6, 'product'), (7, 11, 'brand')]}), 
("w lokka ls shirt pitkähihainen paita helly hansen", {'entities': [(37, 49, 'brand'), (31, 36, 'product'), (11, 16, 'product'), (0, 1, 'gender')]}), 
("guess gu7621 52f", {'entities': [(0, 5, 'brand')]}), 
("spy aurinkolasit arcylon polarized 6800000000042", {'entities': [(0, 3, 'brand'), (4, 16, 'product')]}), 
("mineral radiance creme powder foundation, 7g, tawnee", {'entities': [(23, 40, 'product'), (42, 44, 'size'), (46, 52, 'color')]}), 
("michelin multibib ( 320/65 r16 107d tl )", {'entities': [(0, 8, 'brand'), (20, 38, 'size')]}), 
("apple cto cto:z0y0 mbp 16in tb 2.4ghz i9 32gb 1tb 4gb space grey se btop", {'entities': [(0, 5, 'brand'), (19, 22, 'product'), (60, 64, 'color'), (23, 53, 'size')]}), 
("dragon alliance aurinkolasit dr d1otg bonus\xa0", {'entities': [(16, 28, 'product'), (0, 15, 'brand')]}), 
("mary dew manizer® liquid highlighter korostus varjostus contouring meikki the balm", {'entities': [(25, 36, 'product'), (0, 17, 'brand')]}), 
("woms boots shoes boots ankle boots ankle boot   heel musta tamaris", {'entities': [(59, 66, 'brand'), (53, 58, 'color'), (5, 10, 'product'), (11, 16, 'product')]}), 
("ralph lauren aurinkolasit rl8179 57926h", {'entities': [(0, 12, 'brand'), (13, 25, 'product')]}), 
("silje polvipituinen hame ruskea baum und pferdgarten", {'entities': [(25, 31, 'color'), (32, 52, 'brand'), (20, 24, 'product')]}), 
("fabelab kuu mobiili soittorasialla", {'entities': [(0, 7, 'brand'), (12, 19, 'product'), (20, 34, 'product')]}), 
("neubau silmälasit t038 felix 9040", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("emporio armani aurinkolasit ea2086 30156g", {'entities': [(0, 14, 'brand'), (15, 27, 'product')]}), 
("mineral radiance creme powder foundation, 7g, warm beige", {'entities': [(17, 40, 'product'), (42, 44, 'size'), (51, 56, 'color')]}), 
("mineral radiance creme powder, 7g, honey", {'entities': [(17, 29, 'product'), (31, 33, 'size')]}), 
("mineral radiance creme powder, 7g, neutral", {'entities': [(17, 29, 'product'), (31, 33, 'size')]}), 
("hewlett packard enterprise xl230k gen10 xeon g 6143 rear kit in chip", {'entities': [(0, 15, 'brand')]}), 
("mineral radiance creme powder, 7g, toffee", {'entities': [(17, 29, 'product'), (31, 33, 'size')]}), 
("spy aurinkolasit rocky polarized 6700000000086", {'entities': [(0, 3, 'brand'), (4, 16, 'product')]}), 
("toppatakki only  onlshelly", {'entities': [(0, 10, 'product'), (11, 15, 'brand')]}), 
("adidas originals aurinkolasit aor017 070.009", {'entities': [(0, 6, 'brand'), (17, 29, 'product')]}), 
("Intel Core i7 9700 processor NVIDIA GeForce RTX 2060 16GB RAM and 512GB M.2 SSD", {'entities':[(76,79,'product'),(66,71,'size'),(58,61,'product'),(53,57,'size'),(36,47,'product'),(29,35,'brand'),(19,28,'product'),(0,5,'brand')]}),
("serengeti aurinkolasit renzo 8626", {'entities': [(0, 9, 'brand'), (10, 22, 'product')]}), 
("a rose dream ear accessories jewellery earrings studs hopea efva attling", {'entities': [(29, 38, 'product'), (39, 47, 'product'), (54, 59, 'material'), (0, 16, 'brand'), (17, 28, 'product')]}), 
("vans   checker sweater   musta   m", {'entities': [(0, 4, 'brand'), (15, 22, 'product'), (25, 30, 'color')]}), 
("ashley jersey tee t shirts & tops short sleeved ruskea lexington clothing", {'entities': [(48, 54, 'color'), (18, 33, 'product'), (55, 73, 'brand')]}), 
("wild knit raw polvipituinen mekko beige moshi moshi mind", {'entities': [(28, 33, 'product'), (34, 39, 'color'), (40, 56, 'brand')]}), 
("jordan lattiavalaisin, messinki", {'entities': [(7, 21, 'product'), (0, 6, 'brand')]}), 
("retrosuperfuture aurinkolasit tuttolente lucia black is6d n26", {'entities': [(17, 29, 'product'), (47, 52, 'color'), (0, 16, 'brand')]}), 
("kenneth cole slim miesten kello kc9335 hopea/teräs ø44 mm", {'entities': [(51, 57, 'size'), (39, 50, 'material'), (26, 31, 'product'), (18, 25, 'gender'), (0, 12, 'brand')]}), 
("t paidat pitkillä hihoilla morgan  tpois", {'entities': [(0, 8, 'product'), (35, 40, 'brand')]}), 
("salvatore ferragamo silmälasit sf 2777 424", {'entities': [(0, 19, 'brand'), (20, 30, 'product')]}), 
("smartbuy collection silmälasit hughes 970e", {'entities': [(0, 8, 'brand'), (20, 30, 'product')]}), 
("etched leaves  lasinalunen 30x23 cm 4 kpl pakkaus tummansininen", {'entities': [(27, 35, 'size'), (36, 41, 'size'), (49, 63, 'color'), (15, 26, 'product'), (0, 13, 'brand')]}), 
("diesel silmälasit dl5249 054", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("kampa bambua", {'entities': [(0, 5, 'product'), (6, 12, 'material')]}), 
("paita jrbinta 3/4 blouse", {'entities': [(0, 5, 'product'), (18, 24, 'product'), (6, 13, 'brand'), (14, 17, 'size')]}), 
("kengät pepe jeans  adam saba", {'entities': [(0, 6, 'product'), (12, 17, 'material'), (19, 28, 'brand')]}), 
("d1. gant sports washbag toilettilaukut musta gant", {'entities': [(45, 49, 'brand'), (39, 44, 'color'), (24, 38, 'product'), (16, 23, 'product')]}), 
("supermicro mbd x9srw f o sngl skt intl c602 chipset", {'entities': [(0, 10, 'brand'), (44, 51, 'product')]}), 
("boot shoes boots ankle boots ankle boot   heel valkoinen sofie schnoor", {'entities': [(0, 4, 'product'), (5, 10, 'product'), (11, 16, 'product'), (17, 28, 'product'), (47, 56, 'color'), (57, 70, 'brand')]}), 
("prada Aurinkolasit pr08xs 2au6s1", {'entities': [(6, 18, 'product'), (0, 5, 'brand')]}), 
("paita", {'entities': [(0, 5, 'product')]}), 
("ermenegildo zegna silmälasit ez5009 001", {'entities': [(0, 17, 'brand'), (18, 28, 'product')]}), 
("paita juel 146", {'entities': [(0, 5, 'product'), (6, 10, 'brand'), (11, 14, 'size')]}), 
("liu jo aurinkolasit lj110s 711", {'entities': [(0, 6, 'brand'), (7, 19, 'product')]}), 
("kengät redskins  vervil", {'entities': [(0, 6, 'product'), (17, 23, 'brand')]}), 
("tunika cora", {'entities': [(0, 6, 'product'), (7, 11, 'brand')]}), 
("kengät ippon vintage  dolly magic", {'entities': [(0, 6, 'product'), (22, 33, 'brand')]}), 
("adidas originals silmälasit aor010o 009.000", {'entities': [(0, 6, 'brand'), (17, 27, 'product')]}), 
("jimmy choo silmälasit jc188 fn1", {'entities': [(0, 10, 'brand'), (11, 21, 'product')]}), 
("paita jrbinta 3/4 blouse", {'entities': [(6, 13, 'brand'), (0, 5, 'product'), (18, 24, 'product'), (14, 17, 'size')]}), 
("tag heuer silmälasit th 0555 005", {'entities': [(10, 20, 'product'), (0, 9, 'brand')]}), 
("triangle fixed bikiniyläosa bikiniliivit sininen tommy hilfiger", {'entities': [(49, 63, 'brand'), (41, 48, 'color'), (15, 27, 'product'), (28, 40, 'product')]}), 
("nexen winguard wt1 ( 215/65 r16c 109/107r 8pr )", {'entities': [(0, 5, 'brand'), (21, 44, 'size')]}), 
("realistnen dildo real love 7", {'entities': [(27, 28, 'size'), (11, 16, 'brand')]}), 
("nike aurinkolasit outrider e ev1086 820", {'entities': [(5, 17, 'product'), (0, 4, 'brand')]}), 
("colonia pura, edc 50ml", {'entities': [(0, 12, 'brand'), (18, 22, 'size')]}), 
("maxtrek trek m900 ice ( 235/60 r18 107s, nastarengas  )", {'entities': [(0, 7, 'brand'), (24, 39, 'size'), (41, 52, 'product')]}), 
("tom ford aurinkolasit ft0735 h 01w", {'entities': [(0, 8, 'brand'), (9, 21, 'product')]}), 
("smartbuy collection silmälasit keira l139c", {'entities': [(0, 8, 'brand'), (20, 30, 'product')]}), 
("burberry aurinkolasit be3092q 12437j", {'entities': [(0, 8, 'brand'), (9, 21, 'product')]}), 
("brizaiw dress polvipituinen mekko beige inwear", {'entities': [(28, 33, 'product'), (34, 39, 'color'), (40, 46, 'brand'), (8, 13, 'product')]}), 
("lastenkengät vans  td old skool v", {'entities': [(0, 12, 'product'), (13, 17, 'brand')]}), 
("startech brocade 10g sfpp sr comp   sfp+ module   mm transceiver in accs", {'entities': [(0, 16, 'brand'), (53, 64, 'product'), (17, 20, 'size')]}), 
("adidas originals aurinkolasit aor016 bhs.021", {'entities': [(17, 29, 'product'), (0, 6, 'brand')]}), 
("paita jdyenya 3/4 placket puff top", {'entities': [(0, 5, 'product'), (6, 13, 'brand'), (14, 17, 'size'), (31, 34, 'product')]}), 
("raidallinen neulemekko alba moda laivastonsininen::valkoinen::punainen", {'entities': [(12, 22, 'product'), (23, 32, 'brand'), (33, 70, 'color')]}), 
("zac posen aurinkolasit arroh black", {'entities': [(0, 9, 'brand'), (10, 22, 'product')]}), 
("pusero r.y.v. tee", {'entities': [(0, 6, 'product'), (7, 17, 'brand')]}), 
("sculpted western env bags small shoulder bags   crossbody bags musta calvin klein", {'entities': [(21, 25, 'product'), (32, 45, 'product'), (26, 31, 'size'), (48, 62, 'product'), (63, 68, 'color'), (69, 81, 'brand')]}), 
("bvlgari aurinkolasit swarovski sk 5126 032", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("boss by hugo boss aurinkolasit boss 1073/s 003/ir", {'entities': [(8, 17, 'brand'), (18, 30, 'product')]}), 
("epson workforce es 50", {'entities': [(0, 5, 'brand')]}), 
("jaeger lecoultre master ultra thin naisten kello 1252520 hopea/nahka", {'entities': [(0, 16, 'brand'), (35, 42, 'gender'), (43, 48, 'product'), (57, 62, 'color'), (63, 68, 'material')]}), 
("neubau silmälasit t047 frank 6230", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("lastenkengät beppi\xa0 lovuvette", {'entities': [(6, 12, 'product'), (13, 29, 'brand')]}), 
("emilio pucci silmälasit ep2690 278", {'entities': [(0, 12, 'brand'), (13, 23, 'product')]}), 
("boots shoes boots ankle boots ankle boot   heel harmaa tamaris", {'entities': [(0, 5, 'product'), (6, 11, 'product'), (55, 62, 'brand')]}), 
("lip pencil, brick huulikynä meikki punainen m.a.c.", {'entities': [(4, 10, 'product'), (12, 17, 'color'), (18, 27, 'product'), (35, 43, 'color'), (44, 50, 'brand')]}), 
("mont blanc silmälasit mb0010o 003", {'entities': [(0, 10, 'brand'), (10, 21, 'product')]}), 
("guess aurinkolasit gu 7612 83z", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("Cluse La Boheme Naisten kello CW0101201013 Hopea/Teräs Ø38 mm", {'entities': [(16, 23, 'gender'), (24, 29, 'product'), (43, 54, 'color'), (55, 61, 'size'), (0, 15, 'brand')]}),
("Sara bikini brief", {'entities': [(0, 4, 'brand'), (5, 11, 'product')]}),
("porsche design silmälasit p8256 b", {'entities': [(15, 26, 'product')]}), 
("liu jo silmälasit lj2676 609", {'entities': [(0, 7, 'brand'), (7, 17, 'product')]}), 
("mera polvipituinen mekko monivärinen/kuvioitu stella nova", {'entities': [(18, 24, 'product'), (46, 57, 'brand')]}), 
("tom ford aurinkolasit ft0717 simona 52f", {'entities': [(0, 8, 'brand'), (9, 21, 'product')]}), 
("neubau silmälasit t058 theo 3530", {'entities': [(0, 6, 'brand'), (7, 18, 'product')]}), 
("agira carshoe loaferit matalat kengät musta morris", {'entities': [(31, 37, 'product'), (37, 43, 'color'), (44, 50, 'brand')]}), 
("tommy hilfiger silmälasit th 1554 pjp", {'entities': [(0, 14, 'brand'), (15, 25, 'product')]}), 
("black diamond heavyweight screentap", {'entities': [(0, 13, 'brand')]}), 
("healing style design foam, 200ml", {'entities': [(0, 13, 'brand'), (21, 25, 'product'), (27, 32, 'size')]}), 
("healing style sculpting paste, 100ml", {'entities': [(0, 13, 'brand'), (14, 29, 'product'), (31, 36, 'size')]}), 
("ck silmälasit 5469 714", {'entities': [(0, 2, 'brand'), (3, 13, 'product')]}), 
("wpg4035 ultimate bikini kit 1 set", {'entities': [(17, 27, 'product')]}), 
("montana collection by sbg aurinkolasit\xa0", {'entities': [(0, 25, 'brand'), (26, 38, 'product')]}), 
("neubau silmälasit t068 jan 6200", {'entities': [(0, 6, 'brand'), (7, 17, 'product')]}), 
("apple cto cto:z0y1 mbp 16in tb 2.4ghz i9 16gb 2tb 4gb silver se btop", {'entities': [(0, 5, 'brand'), (18, 22, 'product'), (54, 61, 'color'), (23, 53, 'size')]}), 
("mustad l87", {'entities': [(0, 6, 'color')]}), 
("shunga organica massage oil\xa0 natural", {'entities': [(0, 6, 'brand'), (16, 27, 'product')]}), 
("adidas originals dc dress", {'entities': [(20, 25, 'product'), (0, 6, 'brand')]}), 
("mcq silmälasit mcq0027 rl7", {'entities': [(0, 3, 'brand'), (4, 14, 'product')]}), 
("ralph by ralph lauren silmälasit ra7107 5785", {'entities': [(22, 32, 'product'), (9, 21, 'brand')]}), 
("ermenegildo zegna aurinkolasit ez0138 02e", {'entities': [(0, 17, 'brand'), (17, 31, 'product')]}), 
("polaroid pld 2074/s/x kj1/m9", {'entities': [(0, 8, 'brand')]}), 
("eloise top t shirts & tops long sleeved sininen filippa k", {'entities': [(0, 7, 'brand'), (11, 19, 'product'), (39, 47, 'color'), (27, 39, 'product')]}), 
("neubau silmälasit t021 edmund 7940", {'entities': [(0, 6, 'brand'), (6, 18, 'product')]}), 
("hewlett packard enterprise xl1x0r gen10 xeon s 4108 kit in chip", {'entities': [(0, 15, 'brand')]}), 
("kengät andré\xa0 elba30037 fd pike tube 5mm", {'entities': [(0, 6, 'product'), (6, 12, 'brand'), (37, 40, 'size')]}), 
("guess silmälasit gu 2698 083", {'entities': [(0, 5, 'brand'), (6, 16, 'product')]}), 
("kenda k774f budds creek ( 80/100 21 tl 51m )", {'entities': [(0, 6, 'brand'), (26, 42, 'size')]}), 
("guess aurinkolasit gm 0801 10b", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("vandogz boots ms20 shoes boots ankle boots ankle boot   heel ruskea gestuz", {'entities': [(0, 7, 'brand'), (9, 13, 'product'), (19, 24, 'product'), (60, 67, 'color')]}), 
("guess aurinkolasit gm 0800 10z", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("pressed mineral foundation, 8g, coffee", {'entities': [(28, 30, 'size'), (32, 38, 'product'), (0, 26, 'brand')]}), 
("jorcourier tee ls crew neck jr t shirts long sleeved t shirts punainen jack & j s", {'entities': [(71, 81, 'brand'), (62, 70, 'color'), (31, 39, 'product')]}), 
("prt ls placket detail blouse pitkähihainen pusero paita punainen calvin klein", {'entities': [(65, 77, 'brand'), (55, 64, 'color'), (22, 28, 'product')]}), 
("pressed mineral foundation, 8g, neutral", {'entities': [(0, 26, 'brand'), (28, 30, 'size')]}), 
("swedese lamino nojatuoli maalattu pyökki, musta talja", {'entities': [(0, 14, 'brand'), (42, 47, 'color'), (15, 24, 'product')]}), 
("liu jo silmälasit lj2707 424", {'entities': [(0, 7, 'brand'), (7, 17, 'product')]}), 
("käsiläpsy valkoinen", {'entities': [(10, 19, 'color')]}), 
("carhartt wip   w´ deming jacket   monivärinen   xs", {'entities': [(0, 12, 'brand'), (18, 31, 'product'), (34, 45, 'color'), (48, 50, 'size')]}), 
("paolo päiväpeitto 260x270 cm kermainen beige", {'entities': [(6, 17, 'product'), (18, 28, 'size'), (29, 44, 'color')]}), 
("2nd zsazsa polvipituinen hame musta 2ndday", {'entities': [(11, 29, 'product'), (30, 35, 'color'), (36, 42, 'brand')]}), 
("svetari suncoo pablo", {'entities': [(0, 7, 'product'), (8, 14, 'brand')]}), 
("cayuga purjehduskengät kengät ruskea sebago", {'entities': [(7, 22, 'product'), (23, 29, 'product'), (30, 36, 'color'), (37, 43, 'brand')]}), 
("calvin klein silmälasit ck18540 415", {'entities': [(0, 12, 'brand'), (13, 23, 'product')]}), 
("to go water bottle accessories water bottles vihreä design letters", {'entities': [(6, 18, 'product'), (37, 44, 'product'), (45, 51, 'color'), (52, 66, 'brand')]}), 
("hugo boss hg 1061 003", {'entities': [(0, 9, 'brand')]}), 
("croon silmälasit leesbril classic black shiny", {'entities': [(0, 5, 'brand'), (6, 16, 'product'), (34, 39, 'color')]}), 
("tiger jb 5 svetari collegepaita vihreä kenzo", {'entities': [(11, 18, 'product'), (19, 31, 'product'), (32, 38, 'color'), (39, 44, 'brand')]}), 
("healing color care color preserving conditioner, 250ml", {'entities': [(36, 47, 'product'), (49, 54, 'size')]}), 
("seventh street silmälasit 7a024 086", {'entities': [(0, 14, 'brand'), (15, 25, 'product')]}), 
("warm lining 81923 shoes boots ankle boots ankle boot   flat musta carla f", {'entities': [(0, 11, 'material'), (18, 23, 'product'), (24, 29, 'product'), (30, 41, 'product'), (60, 65, 'color'), (66, 73, 'brand')]}), 
("guess aurinkolasit gu 3026 91w", {'entities': [(0, 5, 'brand'), (6, 18, 'product')]}), 
("healing color care color preserving conditioner, 750ml", {'entities': [(36, 47, 'product'), (49, 54, 'size')]}), 
("adele tee t shirts & tops short sleeved oranssi minus", {'entities': [(10, 25, 'product'), (40, 47, 'color'), (48, 53, 'brand')]}), 
("solar eclipse all over schoolbag accessories bags backpacks vaaleanpunainen bobo choses", {'entities': [(23, 32, 'product'), (50, 59, 'product'), (60, 75, 'color'), (76, 87, 'brand')]}), 
("1 million edt   50 ml", {'entities': [(16, 21, 'size')]}), 
("illusione male edt   50 ml", {'entities': [(10, 14, 'gender'), (21, 26, 'size')]}), 
("donna edt   50 ml", {'entities': [(0, 5, 'product'), (12, 17, 'size')]}), 
("nike mercurial vapor 13 pro ic dream speed 3   valkoinen/musta", {'entities': [(0, 4, 'brand'), (5, 20, 'product'), (47, 62, 'color')]}), 
("chopard happy sport naisten kello 278590 3001 valkoinen/kumi ø30 mm", {'entities': [(0, 7, 'brand'), (20, 27, 'gender'), (28, 33, 'product'), (46, 60, 'color'), (62, 67, 'size')]}), 
("orangerie boxer short lingerie panties hipsters/boyshorts/brazilian musta chantelle", {'entities': [(31, 38, 'product'), (68, 73, 'color'), (74, 83, 'brand')]}), 
("keratin healing oil lustrous shampoo 50ml", {'entities': [(16, 19, 'product'), (29, 36, 'product'), (37, 41, 'size')]}), 
("piece baseball cap accessories headwear caps valkoinen les deux", {'entities': [(6, 18, 'product'), (40, 44, 'product'), (45, 54, 'color'), (55, 63, 'brand')]}), 
("Bridgestone VT01 R ( 130/90 16 RF TL 73H takapyörä )", {'entities': [(0, 11, 'brand'), (21, 36, 'size'), (41, 50, 'product')]}), 
("Blouse Long Sleeve Pitkähihainen Pusero Paita Monivärinen/Kuvioitu Gerry Weber", {'entities': [(67, 78, 'brand'), (40, 45, 'product'), (33, 39, 'product'), (0, 6, 'product'), (46, 66, 'color')]}), 
("Seletti Apina Valaisin, Köysimalli, Ulkokäyttöön, Musta", {'entities': [(50, 55, 'color'), (0, 7, 'brand'), (14, 22, 'product')]}),
("Esprit 99999 Naisten kello ES1L163M0105 Vihreä/Kullansävytetty", {'entities': [(0, 6, 'brand'), (13, 20, 'gender'), (21, 26, 'product'), (40, 62, 'color')]}),
("So & Co New York Madison Naisten kello 5019.4", {'entities': [(25, 32, 'gender'), (33, 38, 'product'), (0, 16, 'brand')]}),
("keratin healing oil lustrous shampoo, 300ml", {'entities': [(16, 19, 'product'), (29, 36, 'product'), (38, 43, 'size')]}), 
("keratin healing oil lustrous shampoo, 950ml", {'entities': [(16, 19, 'product'), (29, 36, 'product'), (38, 43, 'size')]}), 
("lastenkengät nike roshe one print junior", {'entities': [(13, 17, 'brand')]}), 
("keratin healing oil hair treatment 100ml", {'entities': [(16, 19, 'product'), (35, 40, 'size')]}), 
("roberto cavalli aurinkolasit rc 1063 giannutri 01b", {'entities': [(0, 15, 'brand'), (16, 28, 'product')]}), 
("lasten talvisaappaat aigle neervei", {'entities': [(7, 20, 'product')]}), 
("nike silmälasit 5016 445", {'entities': [(0, 4, 'brand'), (5, 15, 'product')]}), 
("kengät tosca blu fraser", {'entities': [(0, 6, 'product'), (7, 16, 'brand')]}), 
("maxxis arctictrekker np3 ( 175/70 r14 88t xl, nastarengas )", {'entities': [(0, 6, 'brand'), (27, 44, 'size')]}), 
("keratin healing oil hair treatment 185ml", {'entities': [(16, 19, 'product'), (35, 40, 'size')]}), 
("nike säärisuojat mercurial lite   sininen/sininen/punainen", {'entities': [(0, 4, 'brand'), (5, 16, 'product'), (34, 58, 'color')]}), 
("adidas originals aurinkolasit aor009 076.026", {'entities': [(0, 16, 'brand'), (17, 29, 'product')]}), 
("keratin healing oil hair treatment 50ml", {'entities': [(16, 19, 'product'), (35, 39, 'size')]}), 
("keratin healing oil lustrous conditioner 50ml", {'entities': [(16, 19, 'product'), (29, 40, 'product'), (41, 45, 'size')]}), 
("perfect edp   50 ml", {'entities': [(14, 19, 'size')]}), 
("persol aurinkolasit po3215s 95/31", {'entities': [(0, 6, 'brand'), (7, 19, 'product')]}), 
("bottega veneta edp   50 ml", {'entities': [(21, 26, 'size'), (0, 14, 'brand'), (15, 18, 'product')]}), 
("vuarnet aurinkolasit vl1709 polarized 0001 2622", {'entities': [(0, 7, 'brand'), (8, 20, 'product')]}), 
("prada linea rossa aurinkolasit ps52ts dg05s0", {'entities': [(0, 5, 'brand'), (18, 30, 'product')]}), 
("be delicious duo edp 50 ml, deostick 75 ml  ", {'entities': [(37, 42, 'size'), (21, 26, 'size')]}), 
("w hh logo hoodie huppari vaaleanpunainen helly hansen", {'entities': [(0, 1, 'gender'), (10, 16, 'product'), (17, 24, 'product'), (25, 40, 'color'), (41, 53, 'brand')]}), 
("keratin healing oil lustrous conditioner 950ml", {'entities': [(16, 19, 'product'), (29, 40, 'product'), (41, 46, 'size')]}), 
("keratin healing oil lustrous conditioner, 250ml", {'entities': [(16, 19, 'product'), (29, 40, 'product'), (42, 47, 'size')]}), 
("healing nourish stimulating shampoo, 1000ml", {'entities': [(28, 35, 'product'), (37, 43, 'size')]}), 
("healing nourish stimulating shampoo, 300ml", {'entities': [(28, 35, 'product'), (37, 42, 'size')]}), 
("plisse jersey skirt polvipituinen hame musta mos mosh", {'entities': [(7, 13, 'product'), (14, 19, 'product'), (20, 38, 'product'), (39, 44, 'color'), (45, 53, 'brand')]}), 
("viiru & pesonen lasten huopa tervehdys", {'entities': [(23, 28, 'product'), (0,15,'brand'), (16,22,'age_group')]}), 
("Viiru & Pesonen's Children's Blanket Greetings",{'entities':[(0,15,'brand'), (17,27,'age_group'), (28,35,'product')]}),
("huawei freebuds 3 bluetooth vastamelukuulokkeet punainen", {'entities': [(48, 56, 'color'), (28, 47, 'product'), (0, 6, 'brand'), (7,15,'product'), (18,27,'product')]}), 
("Vila viOfficiel New L/S Top blouse Body toted blouse stretch jersey Long sleeves and pyre pntie with narrow spring knitting border length approx. 63 cm in size S/M Vila 16.99€ 07 home and garden Ellos", {'entities':[(196,201,'provider'), (176,195,'provider_category'), (170,176,'price'), (164,169,'brand'), (162,163,'size'), (160,161,'size'), (0,4,'brand'), (24,34,'product'), (147,152,'size'), (60,68,'material'), (53,60,'material'), (46,52,'product')]}),
("SENOA Style Shäkägrilli on a pedestal Severin 131.43€ {COLOR: (metal), 07 home and garden Taste shop", {'entities':[(0,5, 'brand'), (38,45,'brand'), (61,66,'color'), (12,23,'product'), (73,92,'provider_category'), (93,102,'provider'), (47,52,'price')]}),
("Intel Core i7 9700 processor NVIDIA GeForce RTX 2060 2060 16GB RAM and 512GB M.2 SSD", {'entities':[(0,5,'brand'), (19,28,'product'), (29,35,'brand'), (58,61,'product'), (76,79,'product'), (66,71,'size'), (53,57,'size'), (36,47,'product')]}),
("'Viiru & Pesonen lasten huopa Tervehdys'",{'entities':[(1,16,'brand'),(17,23,'age_group'),(24,29,'product')]}),
("intel Core i7 9700  suoritin NVIDIA GeForce RTX 2060 2060 16 Gt RAM muistia ja 512 Gt M.2 SSD", {'entities':[(0,5,'brand'), (29,35,'brand'), (20,28,'product'), (36,47,'product'), (64,67,'product'), (90,93,'product'), (79,85,'size'), (58,63,'size')]}),
("Lenovo 17inch laptop Intel Core i7, Turbo Boost up to 4.5GHz with 128MB. 500GB SSD storage lenovo 1276€ 04 elektroniikka Tokmanni",{'entities':[(0,6,'brand'), (7,13,'size'), (14,20,'product'), (21,31,'product'), (54,60,'size'), (66,71,'size'), (73,78,'size'), (91,97,'brand'), (98,102,'price'), (104,120,'provider_category'), (121,129,'provider'), (79,82,'product')]}),
("Samsung 15inch kannettava tietokona Intel Core i3, Turbo Boost up to 3.8GHz, 120GB SSD storage samsung 996€ 04 elektroniikka Elisa ",{'entities':[(0,7,'brand'), (126,131,'provider'), (109,125,'provider_category'), (104,107,'price'), (84,87,'product'), (78,83,'size'), (70,76,'size'), (37,50,'product'), (15,35,'product'), (8,14,'size'), (95,102,'brand')]}),
("Tefal So Tasty paistinpannu 24 cm	Nyttv ja kestv paistinpannu, jolla loihdit herkullisimmat annokset. Tefal 22.9€	07 koti ja puutarha	Band Company",{'entities':[(28,33,'size'), (108,112,'price'), (134,146,'provider'), (114,133,'provider_category'), (102,107,'brand'), (0,5,'brand'), (49,61,'product')]}),
("Rocks Off Silhoutte   DARK DESIRES KIT silicone stimulator. Rocks Off 39.95€ 01 Adults kaalimato", {'entities':[(87,96,'provider'), (77,86,'provider_category'), (70,75,'price'), (60,69,'brand'), (0,9,'brand'), (22,38,'product'), (39,47,'material'), (48,58,'product')]}),
("Rocks Off Silhoutte   DARK DESIRES KIT silikonista valmistetun stimulaattorin. Rocks Off 39.95€ 01 aikuiset SinCity", {'entities':[(39,50,'material'), (79,88,'brand'), (0,9,'brand'), (22,38,'product'), (63,77,'product'), (108,115,'provider'), (96,107,'provider_category'), (89,94,'price')]}),
("Bibs tutti 6 18 kk Tuttien klassikko.  tutit lohduttaneet pikkulapsia. Luonnonkumi tuntuu samettisen pehmeÃ¤ltÃ¤, joustavalta ja luontevalta lapsen imeÃ¤. Kilpi on valmistettu Polypropeenista BPA  ja PVC vapaa. Bibs 3.5€ 08 vauvatarvikkeet ja lastentarvikkeet	Wauva ", {'entities':[(260,266,'provider'), (221,259,'provider_category'), (216,219,'price'), (211,215,'brand'), (0,4,'brand'), (19,26,'product'), (39,44,'product'), (58,69,'age_group'), (141,147,'age_group'), (176,195,'material'), (200,203,'material'), (71,82,'material')]})]

# TAKING ENGLISH LANGUAGE PACKAGE IN SPACY AND CREATING FUNCTION TO: 
# CUSTOMIZING THE ENTITY BY SUPPLYING THE ENTITY THAT WE WANT TO PREDICT TO THE PIPLINES
# TRAINING THE MODEL WITH BACTCH SIZE (COMPUNDING) AND LOSS OPTIMIZER AS 'ADAM'( DEFAULT OF SPACY MODEL)

nlp = spacy.blank('en')
Labels = ['brand', 'price', 'product', 'material', 'size', 'color', 'gender', 'age_group', 'provider', 'provider_category',]
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
        ner=nlp.get_pipe('ner')
        for label in Labels:
            ner.add_label(label)
        
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[1])
            #gold = GoldParse(doc, entities=annotation)
    pipe_exceptions = ["entity_linker", "trf_wordpiecer", "trf_tok2vec"]
    disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(disable_pipes):
        sizes = compounding(1.0, 4.0, 1.001)
        optimizer = nlp.begin_training()
        for itn in tqdm(range(100)):
            random.shuffle(train_data)
            losses ={}
            batches = minibatch(train_data, size=sizes)
            for batch in batches:
                text, annotations = zip(*batch)
                nlp.update(text, annotations, drop = 0.20, sgd= optimizer, losses = losses)
            # print('      Iteration Progress : ' + str(itn), losses)

            
# TRANING MODEL
import warnings
warnings.filterwarnings("ignore")
train_model(train_data)
import datetime
now1 = datetime.datetime.now()
print('Model trainind complete in :', now - now1)
print('********************************************')



from spacy import displacy
text_statement='Samsung 14inch laptop, i7 processor, 8gb ram, 256gb ssd, 990.8€ 04 elektroniikka Gigantti '
doc= nlp(text_statement)
print('As example: ', text_statement)
for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(nlp(doc.text), style='ent', jupyter=True)

sentance= input('Enter you text here : ')
doc= nlp(sentence)
for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(nlp(doc.text), style='ent', jupyter=True)
while senetence not in ['Complete', 'COMPLETE', 'complete', 'DONE', 'Done', 'done', 'Over', 'OVER', 'over', 'CLOSE', 'Close', 'close']:
    print('hint: type -- close, complete or done')
    doc= nlp(sentence)
    for ent in doc.ents:
      print(ent.text, ent.start_char, ent.end_char, ent.label_)
    displacy.render(nlp(doc.text), style='ent', jupyter=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')